In [3]:
# bookmyshow_prototype.py
# Run this cell in Jupyter Notebook. It will write bookmyshow.html, start a local HTTP server
# and open the page in your browser.
# Uses only standard libraries.

import os
import http.server
import socketserver
import threading
import webbrowser
import time
from pathlib import Path

HTML_FILENAME = "bookmyshow.html"
PORT = 8000

html_code = r"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>BookMyShow Prototype</title>
<style>
  :root{
    --primary:#E50914; /* BookMyShow-like red */
    --dark:#222;
    --muted:#666;
    --success:#2ecc71;
    --danger:#e74c3c;
    --white:#ffffff;
    --light-bg:#f8f9fa;
    --card-shadow: 0 8px 24px rgba(0,0,0,0.12);
    --hover-shadow: 0 16px 40px rgba(0,0,0,0.18);
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  }
  * {
    box-sizing: border-box;
  }
  html,body{
    height:100%;
    margin:0;
    padding:0;
    background:var(--light-bg);
    color:var(--dark);
    overflow-x: hidden;
  }
  body {
    display: flex;
    flex-direction: column;
    min-height: 100vh;
  }
  .container{
    width: 100%;
    max-width: 100%;
    margin: 0;
    padding: 0;
    flex: 1;
  }
  header{
    display:flex;
    align-items:center;
    justify-content:space-between;
    padding: 16px 5%;
    background: white;
    box-shadow: 0 2px 10px rgba(0,0,0,0.08);
    position: sticky;
    top: 0;
    z-index: 100;
  }
  .brand{
    display:flex;
    gap:16px;
    align-items:center;
  }
  .logo{
    width:52px;height:52px;border-radius:8px;background:var(--primary);display:flex;align-items:center;justify-content:center;color:#fff;font-weight:bold;font-size:20px;
    box-shadow:var(--card-shadow);
  }
  nav a{
    margin-right:20px;color:var(--dark);text-decoration:none;font-weight:600;
    padding: 10px 16px;
    border-radius: 8px;
    transition: all 0.3s ease;
    font-size: 16px;
  }
  nav a:hover{
    background: #f5f5f5;
    transform: translateY(-2px);
  }
  nav a.active{color:var(--primary); background: #ffeaea; box-shadow: 0 4px 12px rgba(229, 9, 20, 0.15);}
  .hero{
    display:flex;
    gap:30px;
    align-items:flex-start;
    margin-top:0;
    padding: 30px 5%;
    background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%);
    min-height: 500px;
  }
  .card{
    background:#fff;border-radius:16px;padding:24px;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    border: 1px solid rgba(0,0,0,0.05);
  }
  .card:hover {
    box-shadow: var(--hover-shadow);
    transform: translateY(-5px);
  }
  /* LARGER MOVIE GRID STYLES */
  .movie-grid{
    display: grid;
    grid-template-columns: repeat(auto-fill, minmax(320px, 1fr));
    gap: 30px;
    padding: 30px 5%;
  }
  .movie{
    border-radius:16px;overflow:hidden;background:#fff;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    position: relative;
  }
  .movie:hover {
    transform: translateY(-8px);
    box-shadow: var(--hover-shadow);
  }
  .poster{
    width:100%;height:480px;background-size:cover;background-position:center;
    transition: transform 0.6s ease;
  }
  .movie:hover .poster {
    transform: scale(1.08);
  }
  .movie-body{padding:24px;}
  .btn{
    display:inline-block;
    padding:14px 24px;
    border-radius:10px;
    text-decoration:none;
    color:#fff;
    background:var(--primary);
    border:none;
    cursor:pointer;
    font-weight:700;
    font-size: 16px;
    transition: all 0.3s ease;
    text-align: center;
  }
  .btn:hover {
    background: #c40811;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(229, 9, 20, 0.35);
  }
  .btn.secondary{background:#6c757d;color:#fff}
  .btn.secondary:hover {
    background: #5a6268;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(0, 0, 0, 0.2);
  }
  .btn-large {
    padding: 16px 28px;
    font-size: 17px;
  }
  .muted{color:var(--muted);font-size:16px}
  .small{font-size:14px}
  footer{
    margin-top:40px;
    padding: 30px 5%;
    text-align:center;
    color:var(--muted);
    background: white;
    border-top: 1px solid #eee;
  }
  /* Seat map */
  .seat-map-container {
    padding: 20px 5%;
  }
  .seat-map{display:grid;gap:10px; margin: 0 auto; max-width: 800px;}
  .seat-row{display:flex;gap:10px;align-items:center; justify-content: center;}
  .seat{
    width:44px;height:44px;border-radius:8px;background:#e6e6e6;display:flex;align-items:center;justify-content:center;cursor:pointer;user-select:none;
    transition: all 0.3s ease;
    font-weight: 600;
    font-size: 14px;
  }
  .seat.available{background:#b6f0c8; color: #155724;} /* available green-ish */
  .seat.available:hover{background:#9ae6b2; transform: scale(1.15); box-shadow: 0 4px 8px rgba(0,0,0,0.15);}
  .seat.booked{background:#ccc; cursor:not-allowed; opacity:0.6; color: #666;}
  .seat.selected{background:#ff7070; color: white;} /* selected red-ish */
  .seat.selected:hover{background:#ff5252; transform: scale(1.15); box-shadow: 0 4px 8px rgba(255, 80, 80, 0.3);}
  .seat.plate{background:#ffd9d9}
  .legend{
    display:flex;
    gap:20px;
    align-items:center;
    margin-top:12px;
    justify-content: center;
    flex-wrap: wrap;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    max-width: 800px;
    margin: 20px auto;
  }
  .legend .box{
    width:22px;height:22px;border-radius:6px;display:inline-block;margin-right:8px;vertical-align:middle
  }
  .flex{display:flex;gap:16px;align-items:center}
  .spacer{height:16px}
  .summary{
    margin-top:16px;
    padding:20px;
    background:#fafafa;
    border-radius:12px;
    border: 1px solid #eee;
  }
  input[type="text"], input[type="email"], select {
    padding:12px 16px;
    border-radius:8px;
    border:1px solid #ddd;
    width:100%;
    transition: all 0.3s ease;
    font-size: 15px;
    background: white;
  }
  input[type="text"]:focus, input[type="email"]:focus, select:focus {
    border-color: var(--primary);
    outline: none;
    box-shadow: 0 0 0 3px rgba(229, 9, 20, 0.15);
    transform: translateY(-2px);
  }
  label{
    display:block;
    margin-bottom:8px;
    font-weight:600;
    font-size: 15px;
  }
  .row{
    display:flex;
    gap:20px;
    flex-wrap:wrap;
    padding: 20px 5%;
  }
  .page-section {
    padding: 40px 5%;
  }
  /* small details */
  .topline{font-size:15px;color:var(--muted)}
  .badge{
    background:#f1f1f1;
    padding:8px 16px;
    border-radius:10px;
    font-size:15px;
    font-weight: 600;
  }
  .center{text-align:center}
  .hidden{display:none}
  .booking-list{
    margin-top:16px;
    padding: 0 5%;
  }
  .booking-item{
    padding:20px;
    border-radius:12px;
    background:#fff;
    margin-bottom:16px;
    box-shadow:var(--card-shadow);
    transition: all 0.3s ease;
    border-left: 4px solid var(--primary);
  }
  .booking-item:hover {
    box-shadow: var(--hover-shadow);
    transform: translateX(5px);
  }
  .footer-small{font-size:14px;color:var(--muted)}
  .input-inline{display:flex;gap:12px;align-items:center}
  .screen {
    background: linear-gradient(135deg, #333 0%, #555 100%);
    color: white;
    text-align: center;
    padding: 16px;
    margin: 30px auto;
    border-radius: 8px;
    font-weight: bold;
    letter-spacing: 3px;
    font-size: 18px;
    max-width: 800px;
    box-shadow: 0 8px 16px rgba(0,0,0,0.2);
    text-transform: uppercase;
  }
  .rating {
    display: inline-flex;
    align-items: center;
    background: #f1c40f;
    color: #333;
    padding: 6px 12px;
    border-radius: 8px;
    font-weight: bold;
    font-size: 15px;
    margin-left: 12px;
  }
  .movie-meta {
    display: flex;
    align-items: center;
    gap: 12px;
    margin-top: 12px;
    flex-wrap: wrap;
  }
  .genre-tag {
    background: #f0f0f0;
    padding: 6px 12px;
    border-radius: 8px;
    font-size: 14px;
    font-weight: 600;
  }
  .price-tag {
    color: var(--primary);
    font-weight: bold;
    font-size: 18px;
  }
  .notification {
    position: fixed;
    top: 30px;
    right: 30px;
    padding: 16px 24px;
    background: var(--success);
    color: white;
    border-radius: 10px;
    box-shadow: var(--hover-shadow);
    z-index: 1000;
    transform: translateX(150%);
    transition: transform 0.4s ease;
    font-weight: 600;
    max-width: 400px;
  }
  .notification.show {
    transform: translateX(0);
  }
  .notification.error {
    background: var(--danger);
  }
  .section-title {
    font-size: 42px;
    margin-bottom: 16px;
    color: var(--dark);
    font-weight: 800;
    text-align: center;
  }
  .section-subtitle {
    font-size: 20px;
    color: var(--muted);
    margin-bottom: 40px;
    max-width: 700px;
    text-align: center;
    margin-left: auto;
    margin-right: auto;
    line-height: 1.6;
  }
  .welcome-message {
    font-size: 24px;
    margin-bottom: 10px;
    color: var(--dark);
  }
  .welcome-subtitle {
    font-size: 18px;
    color: var(--muted);
    margin-bottom: 30px;
  }
  .hero-left {
    flex: 1;
    min-width: 0;
  }
  .hero-right {
    width: 480px;
    min-width: 480px;
  }
  .quick-actions {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 16px;
    margin-top: 20px;
  }
  .stats {
    display: flex;
    gap: 20px;
    margin-top: 30px;
  }
  .stat-item {
    text-align: center;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    flex: 1;
  }
  .stat-number {
    font-size: 28px;
    font-weight: 800;
    color: var(--primary);
    display: block;
  }
  .stat-label {
    font-size: 14px;
    color: var(--muted);
    margin-top: 5px;
  }
  /* LARGER MOVIE TITLE STYLES */
  .movie-title {
    font-size: 24px;
    font-weight: 800;
    margin-bottom: 8px;
    line-height: 1.2;
  }
  .movie-description {
    font-size: 16px;
    line-height: 1.6;
    color: var(--muted);
    margin-top: 12px;
  }
  .movie-actions {
    display: flex;
    gap: 12px;
    margin-top: 20px;
  }
  .movie-info {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-top: 16px;
  }
  .movie-duration {
    font-size: 15px;
    color: var(--muted);
  }
  /* UPCOMING MOVIE STYLES */
  .upcoming-movie {
    position: relative;
    overflow: hidden;
  }
  .upcoming-overlay {
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: linear-gradient(180deg, rgba(0,0,0,0.1) 0%, rgba(0,0,0,0.7) 100%);
    display: flex;
    flex-direction: column;
    justify-content: flex-end;
    padding: 24px;
    color: white;
  }
  .coming-soon-badge {
    background: var(--primary);
    color: white;
    padding: 10px 20px;
    border-radius: 25px;
    font-weight: 700;
    font-size: 16px;
    display: inline-block;
    margin-bottom: 12px;
  }
  @media (max-width: 1200px) {
    .hero-right {
      width: 400px;
      min-width: 400px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(300px, 1fr));
    }
  }
  @media (max-width: 992px) {
    .hero {
      flex-direction: column;
    }
    .hero-right {
      width: 100%;
      min-width: auto;
    }
    .quick-actions {
      grid-template-columns: 1fr;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
      gap: 20px;
    }
    .poster {
      height: 420px;
    }
  }
  @media (max-width: 768px) {
    header {
      flex-direction: column;
      gap: 16px;
      padding: 16px 20px;
    }
    nav {
      width: 100%;
      display: flex;
      justify-content: space-between;
    }
    nav a {
      margin-right: 0;
      flex: 1;
      text-align: center;
      font-size: 14px;
      padding: 8px 12px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(260px, 1fr));
      padding: 20px;
      gap: 16px;
    }
    .row {
      padding: 20px;
    }
    .page-section {
      padding: 30px 20px;
    }
    .section-title {
      font-size: 36px;
    }
    .section-subtitle {
      font-size: 18px;
    }
    .poster {
      height: 380px;
    }
  }
  @media (max-width: 480px) {
    .movie-grid {
      grid-template-columns: 1fr;
    }
    .poster {
      height: 500px;
    }
  }
</style>
</head>
<body>
<div class="container">
  <header>
    <div class="brand">
      <div class="logo">BMS</div>
      <div>
        <div style="font-weight:800;font-size:22px">BookMyShow Prototype</div>
        <div class="topline">Welcome, <strong>Avaneesh Thakur</strong></div>
      </div>
    </div>
    <nav>
      <a href="#" id="nav-home" class="active">Home</a>
      <a href="#" id="nav-showing">Currently Showing</a>
      <a href="#" id="nav-upcoming">Upcoming</a>
      <a href="#" id="nav-bookings">My Bookings</a>
      <a href="#" id="nav-logout">Logout</a>
    </nav>
  </header>

  <main style="flex: 1;">
    <!-- HOME -->
    <section id="page-home">
      <div class="hero">
        <div class="hero-left">
          <div class="card">
            <div class="welcome-message">Hello, Avaneesh Thakur 👋</div>
            <div class="welcome-subtitle">Welcome to your BookMyShow prototype. Choose a movie to book tickets. Prices shown in ₹ (INR)</div>
            
            <div class="stats">
              <div class="stat-item">
                <span class="stat-number">12</span>
                <span class="stat-label">Movies Available</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">5</span>
                <span class="stat-label">Cities</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">24/7</span>
                <span class="stat-label">Support</span>
              </div>
            </div>
            
            <div class="spacer"></div>
            <div class="quick-actions">
              <a class="btn btn-large" id="btn-see-showing">See Currently Showing</a>
              <a class="btn btn-large secondary" id="btn-see-upcoming">See Upcoming</a>
            </div>
          </div>
          <div style="height:20px"></div>
          <div class="card">
            <h3 style="margin-top:0; font-size: 24px;">Quick Booking</h3>
            <p class="muted" style="font-size: 16px;">Choose a movie and go directly to booking</p>
            <div style="display:flex;gap:12px;align-items:center;margin-top:16px;">
              <select id="quick-movie" style="flex: 1;">
                <option value="jawan" selected>Jawan (Currently Showing)</option>
                <option value="animal">Animal (Currently Showing)</option>
                <option value="salaar">Salaar (Currently Showing)</option>
                <option value="kalki">Kalki 2898 AD (Upcoming)</option>
                <option value="pushpa">Pushpa 2 (Upcoming)</option>
              </select>
              <a class="btn btn-large" id="quick-book">Book Now</a>
            </div>
          </div>
        </div>
        <div class="hero-right">
          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Currently Showing</h4>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.imgur.com/7Qb8Y8R.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Jawan</div>
                <div class="muted">Action • 2h 30m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="jawan" id="home-book-jawan">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.imgur.com/9Qb8Y8S.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Animal</div>
                <div class="muted">Action • 3h 21m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="animal" id="home-book-animal">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center">
              <div style="width:100px;height:140px;background-image:url('https://i.imgur.com/8Qb8Y8T.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Salaar</div>
                <div class="muted">Action • 2h 55m • Telugu/Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="salaar" id="home-book-salaar">Book Tickets</a>
                </div>
              </div>
            </div>
          </div>

          <div style="height:20px"></div>

          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Upcoming</h4>
            <div style="display:flex;gap:16px;flex-direction:column">
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.imgur.com/2QY8kPz.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Kalki 2898 AD</div>
                  <div class="muted small">Sci-Fi • December 2025</div>
                </div>
              </div>
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.imgur.com/8sQmN7R.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Pushpa 2</div>
                  <div class="muted small">Action • December 2025</div>
                </div>
              </div>
            </div>
          </div>

        </div>
      </div>
    </section>

    <!-- SHOWING - LARGER LAYOUT -->
    <section id="page-showing" class="hidden">
      <div class="page-section">
        <div class="section-title">Currently Showing</div>
        <div class="section-subtitle">Book tickets for the latest blockbusters in theaters now. Experience the biggest movies on the largest screens.</div>
        
        <div class="movie-grid" id="showing-grid">
          <!-- Movie: Jawan - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('Jawan-1.jpeg')"></div>
            <div class="movie-body">
              <div class="movie-title">Jawan <span class="rating">8.2/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 30m • Hindi • UA</div>
                <div class="price-tag">₹180 - ₹350</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="jawan" onclick="goToDetails('jawan')">View Details</a>
                <a class="btn secondary" onclick="goToBook('jawan')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Animal - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/9Qb8Y8S.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Animal <span class="rating">7.8/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Crime</span>
              </div>
              <div class="movie-description">
                A father, who is often away due to work, is unable to comprehend the intensity of his son's love and admiration.
              </div>
              <div class="movie-info">
                <div class="movie-duration">3h 21m • Hindi • A</div>
                <div class="price-tag">₹200 - ₹400</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="animal" onclick="goToDetails('animal')">View Details</a>
                <a class="btn secondary" onclick="goToBook('animal')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Salaar - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/8Qb8Y8T.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Salaar <span class="rating">8.5/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs in an epic showdown.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 55m • Telugu/Hindi • UA</div>
                <div class="price-tag">₹190 - ₹380</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="salaar" onclick="goToDetails('salaar')">View Details</a>
                <a class="btn secondary" onclick="goToBook('salaar')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Dunki -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/6Qb8Y8U.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Dunki <span class="rating">7.9/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Comedy</span>
                <span class="genre-tag">Adventure</span>
              </div>
              <div class="movie-description">
                A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called 'Donkey Flight'.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 41m • Hindi • UA</div>
                <div class="price-tag">₹170 - ₹320</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="dunki" onclick="goToDetails('dunki')">View Details</a>
                <a class="btn secondary" onclick="goToBook('dunki')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Fighter -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/7Qb8Y8V.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Fighter <span class="rating">8.1/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Patriotic</span>
              </div>
              <div class="movie-description">
                An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 26m • Hindi • UA</div>
                <div class="price-tag">₹190 - ₹360</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="fighter" onclick="goToDetails('fighter')">View Details</a>
                <a class="btn secondary" onclick="goToBook('fighter')">Book Now</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- UPCOMING - LARGER LAYOUT -->
    <section id="page-upcoming" class="hidden">
      <div class="page-section">
        <div class="section-title">Upcoming Movies</div>
        <div class="section-subtitle">Get ready for these exciting releases coming soon to theaters. Mark your calendars for the most anticipated films of 2025.</div>
        
        <div class="movie-grid">
          <!-- Kalki 2898 AD - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/2QY8kPz.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Kalki 2898 AD</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Sci-Fi • Epic • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience.
              </div>
            </div>
          </div>

          <!-- Pushpa 2 - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/8sQmN7R.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Pushpa 2: The Rule</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Drama • Crime</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Devara -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/9Qb8Y8W.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Devara</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Period • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Singham Again -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/1Qb8Y8X.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Singham Again</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Cop • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The third installment in the popular Singham franchise, featuring Rohit Shetty's cop universe with high-octane action and drama.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: War 2 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/3Qb8Y8Y.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">War 2</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Spy • Thriller</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Baahubali 3 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/4Qb8Y8Z.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Baahubali 3</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Epic • Fantasy • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- MOVIE DETAILS -->
    <section id="page-details" class="hidden">
      <div class="page-section">
        <div class="card" id="details-card">
          <div style="display:flex;gap:24px;align-items:flex-start">
            <div style="width:240px;height:340px;background-size:cover;background-position:center;border-radius:12px; box-shadow: var(--card-shadow);" id="details-poster"></div>
            <div style="flex:1">
              <h2 id="details-title" style="margin:0; font-size: 32px;">Movie Title</h2>
              <div class="muted" id="details-meta" style="font-size: 18px; margin: 10px 0;">Meta</div>
              <p id="details-desc" style="font-size: 16px; line-height: 1.6;">Description</p>
              <div style="margin-top:20px">
                <a class="btn btn-large" id="details-book">Book Tickets</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- BOOK TICKETS -->
    <section id="page-book" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;" id="book-title">Book Tickets</h2>
          <div class="row">
            <div style="flex:2; min-width: 500px;">
              <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
                <div>
                  <label>Show Timing</label>
                  <select id="show-timing">
                    <option>10:00 AM</option>
                    <option>01:30 PM</option>
                    <option>05:00 PM</option>
                    <option>08:30 PM</option>
                  </select>
                </div>

                <div>
                  <label>Seat Category</label>
                  <select id="seat-category">
                    <option value="platinum">Platinum (Rows A-B) — ₹350</option>
                    <option value="gold">Gold (Row C) — ₹250</option>
                    <option value="silver">Silver (Rows D-E) — ₹180</option>
                  </select>
                </div>
              </div>

              <div class="spacer"></div>

              <div style="margin-top:16px">
                <div style="font-weight:700; font-size: 20px;">Select Seats</div>
                <div class="muted" style="font-size: 16px;">Click seats to select/deselect</div>
                
                <div class="legend">
                  <div><span class="box" style="background:#b6f0c8"></span>Available</div>
                  <div><span class="box" style="background:#ff7070"></span>Selected</div>
                  <div><span class="box" style="background:#ccc"></span>Booked/Unavailable</div>
                </div>

                <div style="height:16px"></div>
                
                <div class="screen">SCREEN</div>

                <div class="seat-map card" id="seat-map" style="padding:20px"></div>

              </div>

            </div>

            <div style="flex:1; min-width: 350px;">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Booking Summary</h3>
                <div class="summary">
                  <div style="margin-bottom: 12px;">Movie: <strong id="sum-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Timing: <strong id="sum-time" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="sum-seats" style="float: right;">None</strong></div>
                  <div style="margin-bottom: 12px;">Price per seat: <strong id="sum-price" style="float: right;">₹0</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <span id="sum-total" style="float: right;">₹0</span></div>
                  <div style="height:20px"></div>
                  <label for="email-input">Email for Confirmation</label>
                  <input type="email" id="email-input" placeholder="you@example.com" />
                  <div style="height:16px"></div>
                  <div style="display:flex;gap:12px">
                    <a class="btn" id="proceed-payment" style="flex: 2;">Proceed to Payment</a>
                    <a class="btn secondary" id="clear-selection" style="flex: 1;">Clear</a>
                  </div>
                </div>
              </div>
            </div>
          </div>
        </div> 
      </div>
    </section>

    <!-- PAYMENT -->
    <section id="page-payment" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">Payment</h2>
          <div class="row">
            <div style="flex:1;min-width:500px">
              <div style="margin-bottom:20px">
                <label>Card / UPI Name</label>
                <input type="text" id="pay-name" placeholder="Avaneesh Thakur" value="Avaneesh Thakur" />
              </div>
              <div style="margin-bottom:20px">
                <label>Card / UPI Number</label>
                <input type="text" id="pay-number" placeholder="Enter 12-16 digit card or UPI id" />
              </div>
              <div style="display:grid; grid-template-columns: 1fr 1fr; gap: 16px;">
                <div>
                  <label>Expiry</label>
                  <input type="text" id="pay-expiry" placeholder="MM/YY" />
                </div>
                <div>
                  <label>CVV</label>
                  <input type="text" id="pay-cvv" placeholder="123" />
                </div>
              </div>
              <div style="height:20px"></div>
              <div class="muted">This is a simulated payment — no real transaction will occur.</div>
              <div style="height:20px"></div>
              <div style="display:flex;gap:12px">
                <a class="btn btn-large" id="confirm-pay">Pay & Confirm</a>
                <a class="btn secondary btn-large" id="back-to-book">Back</a>
              </div>
            </div>
            <div style="width:400px">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Payment Summary</h3>
                <div style="margin-top:16px">
                  <div style="margin-bottom: 12px;">Movie: <strong id="pay-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="pay-seats" style="float: right;">-</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <strong id="pay-total" style="float: right;">₹0</strong></div>
                  <div style="height:16px"></div>
                  <div class="muted">A confirmation email will be "sent" to the address you provided (simulated).</div>
                </div>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- CONFIRMATION -->
    <section id="page-confirm" class="hidden">
      <div class="page-section">
        <div class="card" style="max-width: 800px; margin: 0 auto;">
          <h2 style="font-size: 32px; text-align: center;">Booking Confirmed! 🎉</h2>
          <p style="font-size: 18px; text-align: center;">Thank you, <strong>Avaneesh Thakur</strong>. Your booking is confirmed.</p>
          <div class="card" style="background: #f8f9fa;">
            <div style="margin-bottom: 16px; font-size: 16px;">Movie: <strong id="conf-movie" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Timing: <strong id="conf-time" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Seats: <strong id="conf-seats" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Email: <strong id="conf-email" style="float: right;"></strong></div>
            <div style="height:12px; border-bottom: 1px solid #ddd; margin-bottom: 16px;"></div>
            <div style="font-weight:800;font-size:24px">Total Paid: <strong id="conf-total" style="float: right;"></strong></div>
          </div>
          <div style="height:20px"></div>
          <div style="display:flex;gap:12px; justify-content: center;">
            <a class="btn btn-large" id="conf-home">Return to Homepage</a>
            <a class="btn secondary btn-large" id="conf-bookings">My Bookings</a>
          </div>
        </div>
      </div>
    </section>

    <!-- MY BOOKINGS -->
    <section id="page-bookings" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">My Bookings</h2>
          <div id="bookings-list">
            <div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>
          </div>
        </div>
      </div>
    </section>

  </main>

  <footer>
    <div class="footer-small">Made by Avaneesh Thakur • BookMyShow Prototype • 2024</div>
  </footer>
</div>

<div id="notification" class="notification"></div>

<script>
// Update MOVIES object with December 2025 release dates
const MOVIES = {
  jawan: {
    id:'jawan',
    title:'Jawan',
    poster:'https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.bollywoodhungama.com%2Fmovie%2Fjawan%2Fphotos%2Fpage%2F4%2F&psig=AOvVaw3hne2QSyZrsIrLw3O2ayni&ust=1761935920521000&source=images&cd=vfe&opi=89978449&ved=0CBIQjRxqFwoTCNDh5vbIzJADFQAAAAAdAAAAABAE',
    meta:'Action • 2h 30m • Hindi',
    desc:'A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society, while keeping a promise made years ago. He comes up against a monstrous outlaw with no fear, who has caused extreme suffering to many.',
  },
  animal: {
    id:'animal',
    title:'Animal',
    poster:'https://i.imgur.com/9Qb8Y8S.jpg',
    meta:'Action • 3h 21m • Hindi',
    desc:'A father, who is often away due to work, is unable to comprehend the intensity of his son\'s love. Ironically, this fervent love and admiration for his father and family creates conflict between the father and son, which leads the son to become a threat to many.',
  },
  salaar: {
    id:'salaar',
    title:'Salaar',
    poster:'https://i.imgur.com/8Qb8Y8T.jpg',
    meta:'Action • 2h 55m • Telugu/Hindi',
    desc:'A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs. He is forced to become the saviour of the people when the city becomes a battleground for powerful and ruthless criminal forces.',
  },
  dunki: {
    id:'dunki',
    title:'Dunki',
    poster:'https://i.imgur.com/6Qb8Y8U.jpg',
    meta:'Drama • 2h 41m • Hindi',
    desc:'A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called Donkey Flight. The film explores themes of friendship, dreams, and the price of ambition.',
  },
  fighter: {
    id:'fighter',
    title:'Fighter',
    poster:'https://i.imgur.com/7Qb8Y8V.jpg',
    meta:'Action • 2h 26m • Hindi',
    desc:'An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet. The film showcases breathtaking aerial combat sequences and the personal sacrifices of our armed forces.',
  },
  kalki: {
    id:'kalki',
    title:'Kalki 2898 AD',
    poster:'https://i.imgur.com/2QY8kPz.jpg',
    meta:'Sci-Fi • December 2025',
    desc:'A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience. The film promises to redefine Indian cinema with its scale and visual effects.',
  },
  pushpa: {
    id:'pushpa',
    title:'Pushpa 2: The Rule',
    poster:'https://i.imgur.com/8sQmN7R.jpg',
    meta:'Action • December 2025',
    desc:'The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world. The film promises even more action and drama than its predecessor.',
  },
  devara: {
    id:'devara',
    title:'Devara',
    poster:'https://i.imgur.com/9Qb8Y8W.jpg',
    meta:'Action • December 2025',
    desc:'A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.',
  },
  singham: {
    id:'singham',
    title:'Singham Again',
    poster:'https://i.imgur.com/1Qb8Y8X.jpg',
    meta:'Action • December 2025',
    desc:'The third installment in the popular Singham franchise, featuring Rohit Shetty\'s cop universe with high-octane action and drama.',
  },
  war2: {
    id:'war2',
    title:'War 2',
    poster:'https://i.imgur.com/3Qb8Y8Y.jpg',
    meta:'Action • December 2025',
    desc:'The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.',
  },
  baahubali3: {
    id:'baahubali3',
    title:'Baahubali 3',
    poster:'https://i.imgur.com/4Qb8Y8Z.jpg',
    meta:'Epic • December 2025',
    desc:'The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.',
  }
};

// Rest of the JavaScript code remains exactly the same as previous version
// Only the MOVIES object has been updated with December 2025 dates

/* Single-page navigation */
const pages = {
  home: document.getElementById('page-home'),
  showing: document.getElementById('page-showing'),
  upcoming: document.getElementById('page-upcoming'),
  details: document.getElementById('page-details'),
  book: document.getElementById('page-book'),
  payment: document.getElementById('page-payment'),
  confirm: document.getElementById('page-confirm'),
  bookings: document.getElementById('page-bookings'),
};

function showPage(name){
  for(const p of Object.values(pages)) p.classList.add('hidden');
  pages[name].classList.remove('hidden');
  // navbar active states
  document.querySelectorAll('nav a').forEach(a=>a.classList.remove('active'));
  if(name==='home') document.getElementById('nav-home').classList.add('active');
  if(name==='showing') document.getElementById('nav-showing').classList.add('active');
  if(name==='upcoming') document.getElementById('nav-upcoming').classList.add('active');
  if(name==='bookings') document.getElementById('nav-bookings').classList.add('active');
}

/* Seat map state */
const ROWS = ['A','B','C','D','E'];
const SEATS_PER_ROW = 10;
let bookedSeats = {}; // { 'jawan_10:00_A3': true } to mark booked for simulation
let selectedSeats = []; // list of seat ids e.g., 'A3'
let currentMovie = 'jawan';
let currentTiming = '10:00 AM';

const PRICES = {
  platinum:350,
  gold:250,
  silver:180
};

function showNotification(message, isError = false) {
  const notification = document.getElementById('notification');
  notification.textContent = message;
  notification.className = 'notification';
  if (isError) {
    notification.classList.add('error');
  }
  notification.classList.add('show');
  
  setTimeout(() => {
    notification.classList.remove('show');
  }, 3000);
}

function init(){
  // nav handlers
  document.getElementById('nav-home').addEventListener('click', e=>{ e.preventDefault(); showPage('home')});
  document.getElementById('nav-showing').addEventListener('click', e=>{ e.preventDefault(); showPage('showing')});
  document.getElementById('nav-upcoming').addEventListener('click', e=>{ e.preventDefault(); showPage('upcoming')});
  document.getElementById('nav-bookings').addEventListener('click', e=>{ e.preventDefault(); renderBookings(); showPage('bookings')});
  document.getElementById('nav-logout').addEventListener('click', e=>{ e.preventDefault(); logout(); });

  document.getElementById('btn-see-showing').addEventListener('click', ()=>{ showPage('showing')});
  document.getElementById('btn-see-upcoming').addEventListener('click', ()=>{ showPage('upcoming')});
  document.getElementById('home-book-jawan').addEventListener('click', ()=> goToBook('jawan'));
  document.getElementById('home-book-animal').addEventListener('click', ()=> goToBook('animal'));
  document.getElementById('home-book-salaar').addEventListener('click', ()=> goToBook('salaar'));
  document.getElementById('quick-book').addEventListener('click', ()=> {
    const m = document.getElementById('quick-movie').value;
    goToBook(m);
  });

  // details/book buttons
  document.getElementById('details-book').addEventListener('click', ()=> goToBook(currentMovie));

  // book page controls
  document.getElementById('seat-category').addEventListener('change', updateSummary);
  document.getElementById('show-timing').addEventListener('change', ()=> { 
    currentTiming = document.getElementById('show-timing').value; 
    updateSummary();
    applyBookedVisuals();
  });
  document.getElementById('clear-selection').addEventListener('click', clearSelection);
  document.getElementById('proceed-payment').addEventListener('click', proceedToPayment);

  // payment page controls
  document.getElementById('confirm-pay').addEventListener('click', confirmPayment);
  document.getElementById('back-to-book').addEventListener('click', ()=> showPage('book'));

  // confirmation page buttons
  document.getElementById('conf-home').addEventListener('click', ()=> showPage('home'));
  document.getElementById('conf-bookings').addEventListener('click', ()=> { renderBookings(); showPage('bookings'); });

  // initial render
  renderSeatMap();
  populateDetails('jawan');
  updateSummary();
  restoreBookedFromLocal();
}

function goToDetails(movieId){
  currentMovie = movieId;
  populateDetails(movieId);
  showPage('details');
}
function goToBook(movieId){
  currentMovie = movieId;
  // reset selection
  clearSelection();
  populateDetails(movieId);
  showPage('book');
}

/* Populate details card */
function populateDetails(movieId){
  const m = MOVIES[movieId];
  document.getElementById('details-poster').style.backgroundImage = `url('${m.poster}')`;
  document.getElementById('details-title').innerText = m.title;
  document.getElementById('details-meta').innerText = m.meta;
  document.getElementById('details-desc').innerText = m.desc;
  document.getElementById('book-title').innerText = `Book Tickets - ${m.title}`;
  document.getElementById('sum-movie').innerText = m.title;
  document.getElementById('pay-movie').innerText = m.title;
}

/* Seat map rendering */
function renderSeatMap(){
  const container = document.getElementById('seat-map');
  container.innerHTML = '';
  for(const row of ROWS){
    const rowEl = document.createElement('div');
    rowEl.className='seat-row';
    const label = document.createElement('div');
    label.style.width='28px';
    label.style.fontWeight='700';
    label.innerText = row;
    rowEl.appendChild(label);
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const seatEl = document.createElement('div');
      seatEl.className='seat available';
      seatEl.id = `seat-${seatId}`;
      seatEl.innerText = i;
      seatEl.dataset.seat = seatId;
      // assign category styles visually but selection price uses mapping
      if(row==='A' || row==='B') seatEl.title='Platinum';
      else if(row==='C') seatEl.title='Gold';
      else seatEl.title='Silver';

      seatEl.addEventListener('click', ()=> onSeatClick(seatId));
      rowEl.appendChild(seatEl);
    }
    container.appendChild(rowEl);
  }
  // mark already booked seats (simulate)
  applyBookedVisuals();
}

/* simulate restoring booked seats (from localStorage bookings) */
function restoreBookedFromLocal(){
  const b = JSON.parse(localStorage.getItem('bookings') || '[]');
  // mark as booked for current movie/timeslot by id
  bookedSeats = {}; // reset
  for(const item of b){
    if(!item.movieId || !item.timing || !item.seats) continue;
    // for simplicity mark seats as booked for that movie across times
    item.seats.forEach(seat=>{
      const key = `${item.movieId}_${item.timing}_${seat}`;
      bookedSeats[key] = true;
    });
  }
  applyBookedVisuals();
}

/* Check if a seat is booked for current movie/time */
function isSeatBookedGlobal(seatId){
  const key = `${currentMovie}_${currentTiming}_${seatId}`;
  return !!bookedSeats[key];
}

function applyBookedVisuals(){
  for(const row of ROWS){
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const el = document.getElementById(`seat-${seatId}`);
      if(!el) continue;
      if(isSeatBookedGlobal(seatId)){
        el.classList.remove('available','selected');
        el.classList.add('booked');
      } else {
        el.classList.remove('booked');
        el.classList.add('available');
      }
    }
  }
}

/* Seat click handler */
function onSeatClick(seatId){
  // can't select if booked
  if(isSeatBookedGlobal(seatId)) {
    showNotification('This seat is already booked!', true);
    return;
  }
  const el = document.getElementById(`seat-${seatId}`);
  if(!el) return;
  if(el.classList.contains('selected')){
    el.classList.remove('selected');
    selectedSeats = selectedSeats.filter(s=>s!==seatId);
    showNotification(`Seat ${seatId} removed from selection`);
  } else {
    el.classList.add('selected');
    selectedSeats.push(seatId);
    showNotification(`Seat ${seatId} added to selection`);
  }
  updateSummary();
}

/* Clear selection */
function clearSelection(){
  selectedSeats = [];
  document.querySelectorAll('.seat.selected').forEach(s => s.classList.remove('selected'));
  updateSummary();
  showNotification('Selection cleared');
}

/* Update summary panel */
function updateSummary(){
  // movie and time
  document.getElementById('sum-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('sum-time').innerText = document.getElementById('show-timing').value;
  currentTiming = document.getElementById('show-timing').value;
  // price per seat depends on selected category
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  document.getElementById('sum-price').innerText = `₹${price}`;
  // seats
  document.getElementById('sum-seats').innerText = selectedSeats.length ? selectedSeats.join(', ') : 'None';
  // total
  const total = selectedSeats.length * price;
  document.getElementById('sum-total').innerText = `₹${total}`;
  // payment summary fields
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ') || '-';
  document.getElementById('pay-total').innerText = `₹${total}`;
}

/* Proceed to payment validation (email required) */
function proceedToPayment(){
  const email = document.getElementById('email-input').value.trim();
  if(selectedSeats.length===0){
    showNotification('Please select at least one seat.', true);
    return;
  }
  if(!validateEmail(email)){
    showNotification('Please enter a valid email address.', true);
    return;
  }
  // populate payment page summary
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ');
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  const total = selectedSeats.length * price;
  document.getElementById('pay-total').innerText = `₹${total}`;
  // store interim booking info in session (page-level)
  window.currentBooking = {
    movieId: currentMovie,
    movieTitle: MOVIES[currentMovie].title,
    timing: currentTiming,
    seats: selectedSeats.slice(),
    category: cat,
    pricePerSeat: price,
    total: total,
    email: email,
    user: 'Avaneesh Thakur'
  };
  showPage('payment');
}

/* Payment confirmation (simulated) */
function confirmPayment(){
  const name = document.getElementById('pay-name').value.trim();
  const number = document.getElementById('pay-number').value.trim();
  const expiry = document.getElementById('pay-expiry').value.trim();
  const cvv = document.getElementById('pay-cvv').value.trim();
  // simple validations
  if(!name){ showNotification('Please enter name for payment.', true); return; }
  if(!number || number.length < 6){ showNotification('Please enter a valid card/UPI number (simulated).', true); return; }
  if(!expiry || !/^\d{2}\/\d{2}$/.test(expiry)){ showNotification('Expiry must be in MM/YY format.', true); return; }
  if(!cvv || cvv.length<3){ showNotification('Enter CVV (3 digits).', true); return; }

  const b = window.currentBooking || {};
  b.payMethod = 'Simulated Card/UPI';
  b.paymentInfo = {name, numberMasked: maskNumber(number), date: new Date().toISOString()};
  b.confirmationId = 'BMS' + Math.floor(Math.random()*900000 + 100000);

  // save to localStorage bookings array
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  bookings.push(b);
  localStorage.setItem('bookings', JSON.stringify(bookings));

  // mark seats as booked
  for(const s of b.seats){
    const key = `${b.movieId}_${b.timing}_${s}`;
    bookedSeats[key] = true;
  }
  applyBookedVisuals();

  // populate confirmation page
  document.getElementById('conf-movie').innerText = b.movieTitle;
  document.getElementById('conf-time').innerText = b.timing;
  document.getElementById('conf-seats').innerText = b.seats.join(', ');
  document.getElementById('conf-email').innerText = b.email;
  document.getElementById('conf-total').innerText = `₹${b.total}`;

  // clear selected seats for the UI
  selectedSeats = [];
  updateSummary();

  // move to confirmation
  showPage('confirm');
}

/* Booking list rendering */
function renderBookings(){
  const container = document.getElementById('bookings-list');
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  container.innerHTML = '';
  if(!bookings.length){
    container.innerHTML = '<div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>';
    return;
  }
  for(const bk of bookings.slice().reverse()){
    const el = document.createElement('div');
    el.className = 'booking-item';
    el.innerHTML = `
      <div style="display:flex;justify-content:space-between;align-items:center">
        <div>
          <div style="font-weight:800; font-size: 18px;">${escapeHtml(bk.movieTitle)}</div>
          <div class="muted" style="margin-top: 8px;">${escapeHtml(bk.timing)} • Seats: ${escapeHtml(bk.seats.join(', '))}</div>
          <div class="muted" style="margin-top: 4px;">Email: ${escapeHtml(bk.email)}</div>
        </div>
        <div style="text-align:right">
          <div style="font-weight:800; font-size: 20px;">₹${bk.total}</div>
          <div class="muted" style="margin-top: 4px;">Conf: ${escapeHtml(bk.confirmationId || '')}</div>
        </div>
      </div>
    `;
    container.appendChild(el);
  }
}

/* Helpers */
function maskNumber(num){
  if(num.length<=4) return '****';
  const last = num.slice(-4);
  return '**** **** **** ' + last;
}
function validateEmail(email){
  if(!email) return false;
  // simple regex
  const re = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return re.test(email);
}
function escapeHtml(text){
  if(!text) return '';
  return text.replace(/&/g,'&amp;').replace(/</g,'&lt;').replace(/>/g,'&gt;');
}

/* Logout function clears temporary bookings and selections (and local storage if desired) */
function logout(){
  if(confirm('Logout will clear session-selected data (bookings persist). Proceed?')){
    selectedSeats = [];
    window.currentBooking = null;
    // Optionally clear localStorage bookings: commented out to preserve bookings
    // localStorage.removeItem('bookings');
    // just go to home
    showPage('home');
    showNotification('Logged out successfully');
  }
}

/* Utility to set initial booking page movie/time on load if needed */
document.addEventListener('DOMContentLoaded', init);

/* ensure quick UI functions for links that call from html attributes */
function goToBook(movieId){ currentMovie = movieId; populateDetails(movieId); clearSelection(); renderSeatMap(); showPage('book'); }
function goToDetails(movieId){ currentMovie = movieId; populateDetails(movieId); showPage('details'); }

// ensure show-page when clicking elements by id
document.getElementById('btn-see-showing').addEventListener('click', ()=> showPage('showing'));
document.getElementById('btn-see-upcoming').addEventListener('click', ()=> showPage('upcoming'));

// Keep seat visuals consistent when timing or movie changes
document.getElementById('show-timing').addEventListener('change', ()=> { currentTiming = document.getElementById('show-timing').value; applyBookedVisuals(); });

// In case someone navigates directly to payment without currentBooking, prevent errors:
window.addEventListener('beforeunload', ()=>{}); // placeholder

</script>
</body>
</html>
"""

# write html file
p = Path(HTML_FILENAME)
p.write_text(html_code, encoding='utf-8')
print(f"Written {HTML_FILENAME} ({p.stat().st_size} bytes)")

# Serve the file using a simple HTTP server in a background thread
class QuietHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, format, *args):
        # suppress console spam from the server; comment out if you want logs
        return

def start_server(port=PORT):
    Handler = QuietHTTPRequestHandler
    os.chdir(os.path.dirname(os.path.abspath(HTML_FILENAME)) or ".")
    with socketserver.TCPServer(("", port), Handler) as httpd:
        print(f"Serving HTTP on 0.0.0.0 port {port} (http://localhost:{port}/{HTML_FILENAME}) ...")
        httpd.serve_forever()

# start server in background thread to keep notebook interactive
server_thread = threading.Thread(target=start_server, args=(PORT,), daemon=True)
server_thread.start()

# open the page in default browser (give the server a moment to start)
time.sleep(0.6)
url = f"http://localhost:{PORT}/{HTML_FILENAME}"
print(f"Opening {url} in your web browser...")
webbrowser.open(url)

Exception in thread Thread-8 (start_server):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/opt/anaconda3/lib/python3.13/threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/gr/87c8xy3d21q8n0tlcwllwc800000gn/T/ipykernel_13874/2083870292.py", line 1577, in start_server
    with socketserver.TCPServer(("", port), Handler) as httpd:
         ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 457, in __init__
    self.server_bind()
    ~~~~~~~~~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 478, in server_bind
    self.socket.bind(self.server_address)


Written bookmyshow.html (58791 bytes)
Opening http://localhost:8000/bookmyshow.html in your web browser...


True

In [4]:
# bookmyshow_prototype.py
# Run this cell in Jupyter Notebook. It will write bookmyshow.html, start a local HTTP server
# and open the page in your browser.
# Uses only standard libraries.

import os
import http.server
import socketserver
import threading
import webbrowser
import time
from pathlib import Path

HTML_FILENAME = "bookmyshow.html"
PORT = 8000

html_code = r"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>BookMyShow Prototype</title>
<style>
  :root{
    --primary:#E50914; /* BookMyShow-like red */
    --dark:#222;
    --muted:#666;
    --success:#2ecc71;
    --danger:#e74c3c;
    --white:#ffffff;
    --light-bg:#f8f9fa;
    --card-shadow: 0 8px 24px rgba(0,0,0,0.12);
    --hover-shadow: 0 16px 40px rgba(0,0,0,0.18);
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  }
  * {
    box-sizing: border-box;
  }
  html,body{
    height:100%;
    margin:0;
    padding:0;
    background:var(--light-bg);
    color:var(--dark);
    overflow-x: hidden;
  }
  body {
    display: flex;
    flex-direction: column;
    min-height: 100vh;
  }
  .container{
    width: 100%;
    max-width: 100%;
    margin: 0;
    padding: 0;
    flex: 1;
  }
  header{
    display:flex;
    align-items:center;
    justify-content:space-between;
    padding: 16px 5%;
    background: white;
    box-shadow: 0 2px 10px rgba(0,0,0,0.08);
    position: sticky;
    top: 0;
    z-index: 100;
  }
  .brand{
    display:flex;
    gap:16px;
    align-items:center;
  }
  .logo{
    width:52px;height:52px;border-radius:8px;background:var(--primary);display:flex;align-items:center;justify-content:center;color:#fff;font-weight:bold;font-size:20px;
    box-shadow:var(--card-shadow);
  }
  nav a{
    margin-right:20px;color:var(--dark);text-decoration:none;font-weight:600;
    padding: 10px 16px;
    border-radius: 8px;
    transition: all 0.3s ease;
    font-size: 16px;
  }
  nav a:hover{
    background: #f5f5f5;
    transform: translateY(-2px);
  }
  nav a.active{color:var(--primary); background: #ffeaea; box-shadow: 0 4px 12px rgba(229, 9, 20, 0.15);}
  .hero{
    display:flex;
    gap:30px;
    align-items:flex-start;
    margin-top:0;
    padding: 30px 5%;
    background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%);
    min-height: 500px;
  }
  .card{
    background:#fff;border-radius:16px;padding:24px;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    border: 1px solid rgba(0,0,0,0.05);
  }
  .card:hover {
    box-shadow: var(--hover-shadow);
    transform: translateY(-5px);
  }
  /* LARGER MOVIE GRID STYLES */
  .movie-grid{
    display: grid;
    grid-template-columns: repeat(auto-fill, minmax(320px, 1fr));
    gap: 30px;
    padding: 30px 5%;
  }
  .movie{
    border-radius:16px;overflow:hidden;background:#fff;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    position: relative;
  }
  .movie:hover {
    transform: translateY(-8px);
    box-shadow: var(--hover-shadow);
  }
  .poster{
    width:100%;height:480px;background-size:cover;background-position:center;
    transition: transform 0.6s ease;
  }
  .movie:hover .poster {
    transform: scale(1.08);
  }
  .movie-body{padding:24px;}
  .btn{
    display:inline-block;
    padding:14px 24px;
    border-radius:10px;
    text-decoration:none;
    color:#fff;
    background:var(--primary);
    border:none;
    cursor:pointer;
    font-weight:700;
    font-size: 16px;
    transition: all 0.3s ease;
    text-align: center;
  }
  .btn:hover {
    background: #c40811;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(229, 9, 20, 0.35);
  }
  .btn.secondary{background:#6c757d;color:#fff}
  .btn.secondary:hover {
    background: #5a6268;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(0, 0, 0, 0.2);
  }
  .btn-large {
    padding: 16px 28px;
    font-size: 17px;
  }
  .muted{color:var(--muted);font-size:16px}
  .small{font-size:14px}
  footer{
    margin-top:40px;
    padding: 30px 5%;
    text-align:center;
    color:var(--muted);
    background: white;
    border-top: 1px solid #eee;
  }
  /* Seat map */
  .seat-map-container {
    padding: 20px 5%;
  }
  .seat-map{display:grid;gap:10px; margin: 0 auto; max-width: 800px;}
  .seat-row{display:flex;gap:10px;align-items:center; justify-content: center;}
  .seat{
    width:44px;height:44px;border-radius:8px;background:#e6e6e6;display:flex;align-items:center;justify-content:center;cursor:pointer;user-select:none;
    transition: all 0.3s ease;
    font-weight: 600;
    font-size: 14px;
  }
  .seat.available{background:#b6f0c8; color: #155724;} /* available green-ish */
  .seat.available:hover{background:#9ae6b2; transform: scale(1.15); box-shadow: 0 4px 8px rgba(0,0,0,0.15);}
  .seat.booked{background:#ccc; cursor:not-allowed; opacity:0.6; color: #666;}
  .seat.selected{background:#ff7070; color: white;} /* selected red-ish */
  .seat.selected:hover{background:#ff5252; transform: scale(1.15); box-shadow: 0 4px 8px rgba(255, 80, 80, 0.3);}
  .seat.plate{background:#ffd9d9}
  .legend{
    display:flex;
    gap:20px;
    align-items:center;
    margin-top:12px;
    justify-content: center;
    flex-wrap: wrap;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    max-width: 800px;
    margin: 20px auto;
  }
  .legend .box{
    width:22px;height:22px;border-radius:6px;display:inline-block;margin-right:8px;vertical-align:middle
  }
  .flex{display:flex;gap:16px;align-items:center}
  .spacer{height:16px}
  .summary{
    margin-top:16px;
    padding:20px;
    background:#fafafa;
    border-radius:12px;
    border: 1px solid #eee;
  }
  input[type="text"], input[type="email"], select {
    padding:12px 16px;
    border-radius:8px;
    border:1px solid #ddd;
    width:100%;
    transition: all 0.3s ease;
    font-size: 15px;
    background: white;
  }
  input[type="text"]:focus, input[type="email"]:focus, select:focus {
    border-color: var(--primary);
    outline: none;
    box-shadow: 0 0 0 3px rgba(229, 9, 20, 0.15);
    transform: translateY(-2px);
  }
  label{
    display:block;
    margin-bottom:8px;
    font-weight:600;
    font-size: 15px;
  }
  .row{
    display:flex;
    gap:20px;
    flex-wrap:wrap;
    padding: 20px 5%;
  }
  .page-section {
    padding: 40px 5%;
  }
  /* small details */
  .topline{font-size:15px;color:var(--muted)}
  .badge{
    background:#f1f1f1;
    padding:8px 16px;
    border-radius:10px;
    font-size:15px;
    font-weight: 600;
  }
  .center{text-align:center}
  .hidden{display:none}
  .booking-list{
    margin-top:16px;
    padding: 0 5%;
  }
  .booking-item{
    padding:20px;
    border-radius:12px;
    background:#fff;
    margin-bottom:16px;
    box-shadow:var(--card-shadow);
    transition: all 0.3s ease;
    border-left: 4px solid var(--primary);
  }
  .booking-item:hover {
    box-shadow: var(--hover-shadow);
    transform: translateX(5px);
  }
  .footer-small{font-size:14px;color:var(--muted)}
  .input-inline{display:flex;gap:12px;align-items:center}
  .screen {
    background: linear-gradient(135deg, #333 0%, #555 100%);
    color: white;
    text-align: center;
    padding: 16px;
    margin: 30px auto;
    border-radius: 8px;
    font-weight: bold;
    letter-spacing: 3px;
    font-size: 18px;
    max-width: 800px;
    box-shadow: 0 8px 16px rgba(0,0,0,0.2);
    text-transform: uppercase;
  }
  .rating {
    display: inline-flex;
    align-items: center;
    background: #f1c40f;
    color: #333;
    padding: 6px 12px;
    border-radius: 8px;
    font-weight: bold;
    font-size: 15px;
    margin-left: 12px;
  }
  .movie-meta {
    display: flex;
    align-items: center;
    gap: 12px;
    margin-top: 12px;
    flex-wrap: wrap;
  }
  .genre-tag {
    background: #f0f0f0;
    padding: 6px 12px;
    border-radius: 8px;
    font-size: 14px;
    font-weight: 600;
  }
  .price-tag {
    color: var(--primary);
    font-weight: bold;
    font-size: 18px;
  }
  .notification {
    position: fixed;
    top: 30px;
    right: 30px;
    padding: 16px 24px;
    background: var(--success);
    color: white;
    border-radius: 10px;
    box-shadow: var(--hover-shadow);
    z-index: 1000;
    transform: translateX(150%);
    transition: transform 0.4s ease;
    font-weight: 600;
    max-width: 400px;
  }
  .notification.show {
    transform: translateX(0);
  }
  .notification.error {
    background: var(--danger);
  }
  .section-title {
    font-size: 42px;
    margin-bottom: 16px;
    color: var(--dark);
    font-weight: 800;
    text-align: center;
  }
  .section-subtitle {
    font-size: 20px;
    color: var(--muted);
    margin-bottom: 40px;
    max-width: 700px;
    text-align: center;
    margin-left: auto;
    margin-right: auto;
    line-height: 1.6;
  }
  .welcome-message {
    font-size: 24px;
    margin-bottom: 10px;
    color: var(--dark);
  }
  .welcome-subtitle {
    font-size: 18px;
    color: var(--muted);
    margin-bottom: 30px;
  }
  .hero-left {
    flex: 1;
    min-width: 0;
  }
  .hero-right {
    width: 480px;
    min-width: 480px;
  }
  .quick-actions {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 16px;
    margin-top: 20px;
  }
  .stats {
    display: flex;
    gap: 20px;
    margin-top: 30px;
  }
  .stat-item {
    text-align: center;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    flex: 1;
  }
  .stat-number {
    font-size: 28px;
    font-weight: 800;
    color: var(--primary);
    display: block;
  }
  .stat-label {
    font-size: 14px;
    color: var(--muted);
    margin-top: 5px;
  }
  /* LARGER MOVIE TITLE STYLES */
  .movie-title {
    font-size: 24px;
    font-weight: 800;
    margin-bottom: 8px;
    line-height: 1.2;
  }
  .movie-description {
    font-size: 16px;
    line-height: 1.6;
    color: var(--muted);
    margin-top: 12px;
  }
  .movie-actions {
    display: flex;
    gap: 12px;
    margin-top: 20px;
  }
  .movie-info {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-top: 16px;
  }
  .movie-duration {
    font-size: 15px;
    color: var(--muted);
  }
  /* UPCOMING MOVIE STYLES */
  .upcoming-movie {
    position: relative;
    overflow: hidden;
  }
  .upcoming-overlay {
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: linear-gradient(180deg, rgba(0,0,0,0.1) 0%, rgba(0,0,0,0.7) 100%);
    display: flex;
    flex-direction: column;
    justify-content: flex-end;
    padding: 24px;
    color: white;
  }
  .coming-soon-badge {
    background: var(--primary);
    color: white;
    padding: 10px 20px;
    border-radius: 25px;
    font-weight: 700;
    font-size: 16px;
    display: inline-block;
    margin-bottom: 12px;
  }
  @media (max-width: 1200px) {
    .hero-right {
      width: 400px;
      min-width: 400px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(300px, 1fr));
    }
  }
  @media (max-width: 992px) {
    .hero {
      flex-direction: column;
    }
    .hero-right {
      width: 100%;
      min-width: auto;
    }
    .quick-actions {
      grid-template-columns: 1fr;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
      gap: 20px;
    }
    .poster {
      height: 420px;
    }
  }
  @media (max-width: 768px) {
    header {
      flex-direction: column;
      gap: 16px;
      padding: 16px 20px;
    }
    nav {
      width: 100%;
      display: flex;
      justify-content: space-between;
    }
    nav a {
      margin-right: 0;
      flex: 1;
      text-align: center;
      font-size: 14px;
      padding: 8px 12px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(260px, 1fr));
      padding: 20px;
      gap: 16px;
    }
    .row {
      padding: 20px;
    }
    .page-section {
      padding: 30px 20px;
    }
    .section-title {
      font-size: 36px;
    }
    .section-subtitle {
      font-size: 18px;
    }
    .poster {
      height: 380px;
    }
  }
  @media (max-width: 480px) {
    .movie-grid {
      grid-template-columns: 1fr;
    }
    .poster {
      height: 500px;
    }
  }
</style>
</head>
<body>
<div class="container">
  <header>
    <div class="brand">
      <div class="logo">BMS</div>
      <div>
        <div style="font-weight:800;font-size:22px">BookMyShow Prototype</div>
        <div class="topline">Welcome, <strong>Avaneesh Thakur</strong></div>
      </div>
    </div>
    <nav>
      <a href="#" id="nav-home" class="active">Home</a>
      <a href="#" id="nav-showing">Currently Showing</a>
      <a href="#" id="nav-upcoming">Upcoming</a>
      <a href="#" id="nav-bookings">My Bookings</a>
      <a href="#" id="nav-logout">Logout</a>
    </nav>
  </header>

  <main style="flex: 1;">
    <!-- HOME -->
    <section id="page-home">
      <div class="hero">
        <div class="hero-left">
          <div class="card">
            <div class="welcome-message">Hello, Avaneesh Thakur 👋</div>
            <div class="welcome-subtitle">Welcome to your BookMyShow prototype. Choose a movie to book tickets. Prices shown in ₹ (INR)</div>
            
            <div class="stats">
              <div class="stat-item">
                <span class="stat-number">12</span>
                <span class="stat-label">Movies Available</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">5</span>
                <span class="stat-label">Cities</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">24/7</span>
                <span class="stat-label">Support</span>
              </div>
            </div>
            
            <div class="spacer"></div>
            <div class="quick-actions">
              <a class="btn btn-large" id="btn-see-showing">See Currently Showing</a>
              <a class="btn btn-large secondary" id="btn-see-upcoming">See Upcoming</a>
            </div>
          </div>
          <div style="height:20px"></div>
          <div class="card">
            <h3 style="margin-top:0; font-size: 24px;">Quick Booking</h3>
            <p class="muted" style="font-size: 16px;">Choose a movie and go directly to booking</p>
            <div style="display:flex;gap:12px;align-items:center;margin-top:16px;">
              <select id="quick-movie" style="flex: 1;">
                <option value="jawan" selected>Jawan (Currently Showing)</option>
                <option value="animal">Animal (Currently Showing)</option>
                <option value="salaar">Salaar (Currently Showing)</option>
                <option value="kalki">Kalki 2898 AD (Upcoming)</option>
                <option value="pushpa">Pushpa 2 (Upcoming)</option>
              </select>
              <a class="btn btn-large" id="quick-book">Book Now</a>
            </div>
          </div>
        </div>
        <div class="hero-right">
          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Currently Showing</h4>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://ibb.co/qF9NzK5s');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Jawan</div>
                <div class="muted">Action • 2h 30m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="jawan" id="home-book-jawan">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.imgur.com/9Qb8Y8S.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Animal</div>
                <div class="muted">Action • 3h 21m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="animal" id="home-book-animal">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center">
              <div style="width:100px;height:140px;background-image:url('https://i.imgur.com/8Qb8Y8T.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Salaar</div>
                <div class="muted">Action • 2h 55m • Telugu/Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="salaar" id="home-book-salaar">Book Tickets</a>
                </div>
              </div>
            </div>
          </div>

          <div style="height:20px"></div>

          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Upcoming</h4>
            <div style="display:flex;gap:16px;flex-direction:column">
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.imgur.com/2QY8kPz.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Kalki 2898 AD</div>
                  <div class="muted small">Sci-Fi • December 2025</div>
                </div>
              </div>
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.imgur.com/8sQmN7R.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Pushpa 2</div>
                  <div class="muted small">Action • December 2025</div>
                </div>
              </div>
            </div>
          </div>

        </div>
      </div>
    </section>

    <!-- SHOWING - LARGER LAYOUT -->
    <section id="page-showing" class="hidden">
      <div class="page-section">
        <div class="section-title">Currently Showing</div>
        <div class="section-subtitle">Book tickets for the latest blockbusters in theaters now. Experience the biggest movies on the largest screens.</div>
        
        <div class="movie-grid" id="showing-grid">
          <!-- Movie: Jawan - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://ibb.co/qF9NzK5s')"></div>
            <div class="movie-body">
              <div class="movie-title">Jawan <span class="rating">8.2/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 30m • Hindi • UA</div>
                <div class="price-tag">₹180 - ₹350</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="jawan" onclick="goToDetails('jawan')">View Details</a>
                <a class="btn secondary" onclick="goToBook('jawan')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Animal - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/9Qb8Y8S.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Animal <span class="rating">7.8/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Crime</span>
              </div>
              <div class="movie-description">
                A father, who is often away due to work, is unable to comprehend the intensity of his son's love and admiration.
              </div>
              <div class="movie-info">
                <div class="movie-duration">3h 21m • Hindi • A</div>
                <div class="price-tag">₹200 - ₹400</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="animal" onclick="goToDetails('animal')">View Details</a>
                <a class="btn secondary" onclick="goToBook('animal')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Salaar - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/8Qb8Y8T.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Salaar <span class="rating">8.5/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs in an epic showdown.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 55m • Telugu/Hindi • UA</div>
                <div class="price-tag">₹190 - ₹380</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="salaar" onclick="goToDetails('salaar')">View Details</a>
                <a class="btn secondary" onclick="goToBook('salaar')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Dunki -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/6Qb8Y8U.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Dunki <span class="rating">7.9/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Comedy</span>
                <span class="genre-tag">Adventure</span>
              </div>
              <div class="movie-description">
                A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called 'Donkey Flight'.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 41m • Hindi • UA</div>
                <div class="price-tag">₹170 - ₹320</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="dunki" onclick="goToDetails('dunki')">View Details</a>
                <a class="btn secondary" onclick="goToBook('dunki')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Fighter -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/7Qb8Y8V.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Fighter <span class="rating">8.1/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Patriotic</span>
              </div>
              <div class="movie-description">
                An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 26m • Hindi • UA</div>
                <div class="price-tag">₹190 - ₹360</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="fighter" onclick="goToDetails('fighter')">View Details</a>
                <a class="btn secondary" onclick="goToBook('fighter')">Book Now</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- UPCOMING - LARGER LAYOUT -->
    <section id="page-upcoming" class="hidden">
      <div class="page-section">
        <div class="section-title">Upcoming Movies</div>
        <div class="section-subtitle">Get ready for these exciting releases coming soon to theaters. Mark your calendars for the most anticipated films of 2025.</div>
        
        <div class="movie-grid">
          <!-- Kalki 2898 AD - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/2QY8kPz.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Kalki 2898 AD</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Sci-Fi • Epic • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience.
              </div>
            </div>
          </div>

          <!-- Pushpa 2 - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/8sQmN7R.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Pushpa 2: The Rule</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Drama • Crime</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Devara -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/9Qb8Y8W.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Devara</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Period • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Singham Again -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/1Qb8Y8X.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Singham Again</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Cop • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The third installment in the popular Singham franchise, featuring Rohit Shetty's cop universe with high-octane action and drama.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: War 2 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/3Qb8Y8Y.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">War 2</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Spy • Thriller</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Baahubali 3 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/4Qb8Y8Z.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Baahubali 3</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Epic • Fantasy • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- MOVIE DETAILS -->
    <section id="page-details" class="hidden">
      <div class="page-section">
        <div class="card" id="details-card">
          <div style="display:flex;gap:24px;align-items:flex-start">
            <div style="width:240px;height:340px;background-size:cover;background-position:center;border-radius:12px; box-shadow: var(--card-shadow);" id="details-poster"></div>
            <div style="flex:1">
              <h2 id="details-title" style="margin:0; font-size: 32px;">Movie Title</h2>
              <div class="muted" id="details-meta" style="font-size: 18px; margin: 10px 0;">Meta</div>
              <p id="details-desc" style="font-size: 16px; line-height: 1.6;">Description</p>
              <div style="margin-top:20px">
                <a class="btn btn-large" id="details-book">Book Tickets</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- BOOK TICKETS -->
    <section id="page-book" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;" id="book-title">Book Tickets</h2>
          <div class="row">
            <div style="flex:2; min-width: 500px;">
              <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
                <div>
                  <label>Show Timing</label>
                  <select id="show-timing">
                    <option>10:00 AM</option>
                    <option>01:30 PM</option>
                    <option>05:00 PM</option>
                    <option>08:30 PM</option>
                  </select>
                </div>

                <div>
                  <label>Seat Category</label>
                  <select id="seat-category">
                    <option value="platinum">Platinum (Rows A-B) — ₹350</option>
                    <option value="gold">Gold (Row C) — ₹250</option>
                    <option value="silver">Silver (Rows D-E) — ₹180</option>
                  </select>
                </div>
              </div>

              <div class="spacer"></div>

              <div style="margin-top:16px">
                <div style="font-weight:700; font-size: 20px;">Select Seats</div>
                <div class="muted" style="font-size: 16px;">Click seats to select/deselect</div>
                
                <div class="legend">
                  <div><span class="box" style="background:#b6f0c8"></span>Available</div>
                  <div><span class="box" style="background:#ff7070"></span>Selected</div>
                  <div><span class="box" style="background:#ccc"></span>Booked/Unavailable</div>
                </div>

                <div style="height:16px"></div>
                
                <div class="screen">SCREEN</div>

                <div class="seat-map card" id="seat-map" style="padding:20px"></div>

              </div>

            </div>

            <div style="flex:1; min-width: 350px;">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Booking Summary</h3>
                <div class="summary">
                  <div style="margin-bottom: 12px;">Movie: <strong id="sum-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Timing: <strong id="sum-time" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="sum-seats" style="float: right;">None</strong></div>
                  <div style="margin-bottom: 12px;">Price per seat: <strong id="sum-price" style="float: right;">₹0</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <span id="sum-total" style="float: right;">₹0</span></div>
                  <div style="height:20px"></div>
                  <label for="email-input">Email for Confirmation</label>
                  <input type="email" id="email-input" placeholder="you@example.com" />
                  <div style="height:16px"></div>
                  <div style="display:flex;gap:12px">
                    <a class="btn" id="proceed-payment" style="flex: 2;">Proceed to Payment</a>
                    <a class="btn secondary" id="clear-selection" style="flex: 1;">Clear</a>
                  </div>
                </div>
              </div>
            </div>
          </div>
        </div> 
      </div>
    </section>

    <!-- PAYMENT -->
    <section id="page-payment" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">Payment</h2>
          <div class="row">
            <div style="flex:1;min-width:500px">
              <div style="margin-bottom:20px">
                <label>Card / UPI Name</label>
                <input type="text" id="pay-name" placeholder="Avaneesh Thakur" value="Avaneesh Thakur" />
              </div>
              <div style="margin-bottom:20px">
                <label>Card / UPI Number</label>
                <input type="text" id="pay-number" placeholder="Enter 12-16 digit card or UPI id" />
              </div>
              <div style="display:grid; grid-template-columns: 1fr 1fr; gap: 16px;">
                <div>
                  <label>Expiry</label>
                  <input type="text" id="pay-expiry" placeholder="MM/YY" />
                </div>
                <div>
                  <label>CVV</label>
                  <input type="text" id="pay-cvv" placeholder="123" />
                </div>
              </div>
              <div style="height:20px"></div>
              <div class="muted">This is a simulated payment — no real transaction will occur.</div>
              <div style="height:20px"></div>
              <div style="display:flex;gap:12px">
                <a class="btn btn-large" id="confirm-pay">Pay & Confirm</a>
                <a class="btn secondary btn-large" id="back-to-book">Back</a>
              </div>
            </div>
            <div style="width:400px">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Payment Summary</h3>
                <div style="margin-top:16px">
                  <div style="margin-bottom: 12px;">Movie: <strong id="pay-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="pay-seats" style="float: right;">-</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <strong id="pay-total" style="float: right;">₹0</strong></div>
                  <div style="height:16px"></div>
                  <div class="muted">A confirmation email will be "sent" to the address you provided (simulated).</div>
                </div>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- CONFIRMATION -->
    <section id="page-confirm" class="hidden">
      <div class="page-section">
        <div class="card" style="max-width: 800px; margin: 0 auto;">
          <h2 style="font-size: 32px; text-align: center;">Booking Confirmed! 🎉</h2>
          <p style="font-size: 18px; text-align: center;">Thank you, <strong>Avaneesh Thakur</strong>. Your booking is confirmed.</p>
          <div class="card" style="background: #f8f9fa;">
            <div style="margin-bottom: 16px; font-size: 16px;">Movie: <strong id="conf-movie" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Timing: <strong id="conf-time" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Seats: <strong id="conf-seats" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Email: <strong id="conf-email" style="float: right;"></strong></div>
            <div style="height:12px; border-bottom: 1px solid #ddd; margin-bottom: 16px;"></div>
            <div style="font-weight:800;font-size:24px">Total Paid: <strong id="conf-total" style="float: right;"></strong></div>
          </div>
          <div style="height:20px"></div>
          <div style="display:flex;gap:12px; justify-content: center;">
            <a class="btn btn-large" id="conf-home">Return to Homepage</a>
            <a class="btn secondary btn-large" id="conf-bookings">My Bookings</a>
          </div>
        </div>
      </div>
    </section>

    <!-- MY BOOKINGS -->
    <section id="page-bookings" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">My Bookings</h2>
          <div id="bookings-list">
            <div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>
          </div>
        </div>
      </div>
    </section>

  </main>

  <footer>
    <div class="footer-small">Made by Avaneesh Thakur • BookMyShow Prototype • 2024</div>
  </footer>
</div>

<div id="notification" class="notification"></div>

<script>
// Update MOVIES object with December 2025 release dates and Jawan image URL
const MOVIES = {
  jawan: {
    id:'jawan',
    title:'Jawan',
    poster:'https://ibb.co/qF9NzK5s',
    meta:'Action • 2h 30m • Hindi',
    desc:'A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society, while keeping a promise made years ago. He comes up against a monstrous outlaw with no fear, who has caused extreme suffering to many.',
  },
  animal: {
    id:'animal',
    title:'Animal',
    poster:'https://i.imgur.com/9Qb8Y8S.jpg',
    meta:'Action • 3h 21m • Hindi',
    desc:'A father, who is often away due to work, is unable to comprehend the intensity of his son\'s love. Ironically, this fervent love and admiration for his father and family creates conflict between the father and son, which leads the son to become a threat to many.',
  },
  salaar: {
    id:'salaar',
    title:'Salaar',
    poster:'https://i.imgur.com/8Qb8Y8T.jpg',
    meta:'Action • 2h 55m • Telugu/Hindi',
    desc:'A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs. He is forced to become the saviour of the people when the city becomes a battleground for powerful and ruthless criminal forces.',
  },
  dunki: {
    id:'dunki',
    title:'Dunki',
    poster:'https://i.imgur.com/6Qb8Y8U.jpg',
    meta:'Drama • 2h 41m • Hindi',
    desc:'A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called Donkey Flight. The film explores themes of friendship, dreams, and the price of ambition.',
  },
  fighter: {
    id:'fighter',
    title:'Fighter',
    poster:'https://i.imgur.com/7Qb8Y8V.jpg',
    meta:'Action • 2h 26m • Hindi',
    desc:'An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet. The film showcases breathtaking aerial combat sequences and the personal sacrifices of our armed forces.',
  },
  kalki: {
    id:'kalki',
    title:'Kalki 2898 AD',
    poster:'https://i.imgur.com/2QY8kPz.jpg',
    meta:'Sci-Fi • December 2025',
    desc:'A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience. The film promises to redefine Indian cinema with its scale and visual effects.',
  },
  pushpa: {
    id:'pushpa',
    title:'Pushpa 2: The Rule',
    poster:'https://i.imgur.com/8sQmN7R.jpg',
    meta:'Action • December 2025',
    desc:'The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world. The film promises even more action and drama than its predecessor.',
  },
  devara: {
    id:'devara',
    title:'Devara',
    poster:'https://i.imgur.com/9Qb8Y8W.jpg',
    meta:'Action • December 2025',
    desc:'A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.',
  },
  singham: {
    id:'singham',
    title:'Singham Again',
    poster:'https://i.imgur.com/1Qb8Y8X.jpg',
    meta:'Action • December 2025',
    desc:'The third installment in the popular Singham franchise, featuring Rohit Shetty\'s cop universe with high-octane action and drama.',
  },
  war2: {
    id:'war2',
    title:'War 2',
    poster:'https://i.imgur.com/3Qb8Y8Y.jpg',
    meta:'Action • December 2025',
    desc:'The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.',
  },
  baahubali3: {
    id:'baahubali3',
    title:'Baahubali 3',
    poster:'https://i.imgur.com/4Qb8Y8Z.jpg',
    meta:'Epic • December 2025',
    desc:'The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.',
  }
};

// Rest of the JavaScript code remains exactly the same as previous version
// Only the MOVIES object has been updated with December 2025 dates and Jawan image URL

/* Single-page navigation */
const pages = {
  home: document.getElementById('page-home'),
  showing: document.getElementById('page-showing'),
  upcoming: document.getElementById('page-upcoming'),
  details: document.getElementById('page-details'),
  book: document.getElementById('page-book'),
  payment: document.getElementById('page-payment'),
  confirm: document.getElementById('page-confirm'),
  bookings: document.getElementById('page-bookings'),
};

function showPage(name){
  for(const p of Object.values(pages)) p.classList.add('hidden');
  pages[name].classList.remove('hidden');
  // navbar active states
  document.querySelectorAll('nav a').forEach(a=>a.classList.remove('active'));
  if(name==='home') document.getElementById('nav-home').classList.add('active');
  if(name==='showing') document.getElementById('nav-showing').classList.add('active');
  if(name==='upcoming') document.getElementById('nav-upcoming').classList.add('active');
  if(name==='bookings') document.getElementById('nav-bookings').classList.add('active');
}

/* Seat map state */
const ROWS = ['A','B','C','D','E'];
const SEATS_PER_ROW = 10;
let bookedSeats = {}; // { 'jawan_10:00_A3': true } to mark booked for simulation
let selectedSeats = []; // list of seat ids e.g., 'A3'
let currentMovie = 'jawan';
let currentTiming = '10:00 AM';

const PRICES = {
  platinum:350,
  gold:250,
  silver:180
};

function showNotification(message, isError = false) {
  const notification = document.getElementById('notification');
  notification.textContent = message;
  notification.className = 'notification';
  if (isError) {
    notification.classList.add('error');
  }
  notification.classList.add('show');
  
  setTimeout(() => {
    notification.classList.remove('show');
  }, 3000);
}

function init(){
  // nav handlers
  document.getElementById('nav-home').addEventListener('click', e=>{ e.preventDefault(); showPage('home')});
  document.getElementById('nav-showing').addEventListener('click', e=>{ e.preventDefault(); showPage('showing')});
  document.getElementById('nav-upcoming').addEventListener('click', e=>{ e.preventDefault(); showPage('upcoming')});
  document.getElementById('nav-bookings').addEventListener('click', e=>{ e.preventDefault(); renderBookings(); showPage('bookings')});
  document.getElementById('nav-logout').addEventListener('click', e=>{ e.preventDefault(); logout(); });

  document.getElementById('btn-see-showing').addEventListener('click', ()=>{ showPage('showing')});
  document.getElementById('btn-see-upcoming').addEventListener('click', ()=>{ showPage('upcoming')});
  document.getElementById('home-book-jawan').addEventListener('click', ()=> goToBook('jawan'));
  document.getElementById('home-book-animal').addEventListener('click', ()=> goToBook('animal'));
  document.getElementById('home-book-salaar').addEventListener('click', ()=> goToBook('salaar'));
  document.getElementById('quick-book').addEventListener('click', ()=> {
    const m = document.getElementById('quick-movie').value;
    goToBook(m);
  });

  // details/book buttons
  document.getElementById('details-book').addEventListener('click', ()=> goToBook(currentMovie));

  // book page controls
  document.getElementById('seat-category').addEventListener('change', updateSummary);
  document.getElementById('show-timing').addEventListener('change', ()=> { 
    currentTiming = document.getElementById('show-timing').value; 
    updateSummary();
    applyBookedVisuals();
  });
  document.getElementById('clear-selection').addEventListener('click', clearSelection);
  document.getElementById('proceed-payment').addEventListener('click', proceedToPayment);

  // payment page controls
  document.getElementById('confirm-pay').addEventListener('click', confirmPayment);
  document.getElementById('back-to-book').addEventListener('click', ()=> showPage('book'));

  // confirmation page buttons
  document.getElementById('conf-home').addEventListener('click', ()=> showPage('home'));
  document.getElementById('conf-bookings').addEventListener('click', ()=> { renderBookings(); showPage('bookings'); });

  // initial render
  renderSeatMap();
  populateDetails('jawan');
  updateSummary();
  restoreBookedFromLocal();
}

function goToDetails(movieId){
  currentMovie = movieId;
  populateDetails(movieId);
  showPage('details');
}
function goToBook(movieId){
  currentMovie = movieId;
  // reset selection
  clearSelection();
  populateDetails(movieId);
  showPage('book');
}

/* Populate details card */
function populateDetails(movieId){
  const m = MOVIES[movieId];
  document.getElementById('details-poster').style.backgroundImage = `url('${m.poster}')`;
  document.getElementById('details-title').innerText = m.title;
  document.getElementById('details-meta').innerText = m.meta;
  document.getElementById('details-desc').innerText = m.desc;
  document.getElementById('book-title').innerText = `Book Tickets - ${m.title}`;
  document.getElementById('sum-movie').innerText = m.title;
  document.getElementById('pay-movie').innerText = m.title;
}

/* Seat map rendering */
function renderSeatMap(){
  const container = document.getElementById('seat-map');
  container.innerHTML = '';
  for(const row of ROWS){
    const rowEl = document.createElement('div');
    rowEl.className='seat-row';
    const label = document.createElement('div');
    label.style.width='28px';
    label.style.fontWeight='700';
    label.innerText = row;
    rowEl.appendChild(label);
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const seatEl = document.createElement('div');
      seatEl.className='seat available';
      seatEl.id = `seat-${seatId}`;
      seatEl.innerText = i;
      seatEl.dataset.seat = seatId;
      // assign category styles visually but selection price uses mapping
      if(row==='A' || row==='B') seatEl.title='Platinum';
      else if(row==='C') seatEl.title='Gold';
      else seatEl.title='Silver';

      seatEl.addEventListener('click', ()=> onSeatClick(seatId));
      rowEl.appendChild(seatEl);
    }
    container.appendChild(rowEl);
  }
  // mark already booked seats (simulate)
  applyBookedVisuals();
}

/* simulate restoring booked seats (from localStorage bookings) */
function restoreBookedFromLocal(){
  const b = JSON.parse(localStorage.getItem('bookings') || '[]');
  // mark as booked for current movie/timeslot by id
  bookedSeats = {}; // reset
  for(const item of b){
    if(!item.movieId || !item.timing || !item.seats) continue;
    // for simplicity mark seats as booked for that movie across times
    item.seats.forEach(seat=>{
      const key = `${item.movieId}_${item.timing}_${seat}`;
      bookedSeats[key] = true;
    });
  }
  applyBookedVisuals();
}

/* Check if a seat is booked for current movie/time */
function isSeatBookedGlobal(seatId){
  const key = `${currentMovie}_${currentTiming}_${seatId}`;
  return !!bookedSeats[key];
}

function applyBookedVisuals(){
  for(const row of ROWS){
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const el = document.getElementById(`seat-${seatId}`);
      if(!el) continue;
      if(isSeatBookedGlobal(seatId)){
        el.classList.remove('available','selected');
        el.classList.add('booked');
      } else {
        el.classList.remove('booked');
        el.classList.add('available');
      }
    }
  }
}

/* Seat click handler */
function onSeatClick(seatId){
  // can't select if booked
  if(isSeatBookedGlobal(seatId)) {
    showNotification('This seat is already booked!', true);
    return;
  }
  const el = document.getElementById(`seat-${seatId}`);
  if(!el) return;
  if(el.classList.contains('selected')){
    el.classList.remove('selected');
    selectedSeats = selectedSeats.filter(s=>s!==seatId);
    showNotification(`Seat ${seatId} removed from selection`);
  } else {
    el.classList.add('selected');
    selectedSeats.push(seatId);
    showNotification(`Seat ${seatId} added to selection`);
  }
  updateSummary();
}

/* Clear selection */
function clearSelection(){
  selectedSeats = [];
  document.querySelectorAll('.seat.selected').forEach(s => s.classList.remove('selected'));
  updateSummary();
  showNotification('Selection cleared');
}

/* Update summary panel */
function updateSummary(){
  // movie and time
  document.getElementById('sum-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('sum-time').innerText = document.getElementById('show-timing').value;
  currentTiming = document.getElementById('show-timing').value;
  // price per seat depends on selected category
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  document.getElementById('sum-price').innerText = `₹${price}`;
  // seats
  document.getElementById('sum-seats').innerText = selectedSeats.length ? selectedSeats.join(', ') : 'None';
  // total
  const total = selectedSeats.length * price;
  document.getElementById('sum-total').innerText = `₹${total}`;
  // payment summary fields
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ') || '-';
  document.getElementById('pay-total').innerText = `₹${total}`;
}

/* Proceed to payment validation (email required) */
function proceedToPayment(){
  const email = document.getElementById('email-input').value.trim();
  if(selectedSeats.length===0){
    showNotification('Please select at least one seat.', true);
    return;
  }
  if(!validateEmail(email)){
    showNotification('Please enter a valid email address.', true);
    return;
  }
  // populate payment page summary
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ');
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  const total = selectedSeats.length * price;
  document.getElementById('pay-total').innerText = `₹${total}`;
  // store interim booking info in session (page-level)
  window.currentBooking = {
    movieId: currentMovie,
    movieTitle: MOVIES[currentMovie].title,
    timing: currentTiming,
    seats: selectedSeats.slice(),
    category: cat,
    pricePerSeat: price,
    total: total,
    email: email,
    user: 'Avaneesh Thakur'
  };
  showPage('payment');
}

/* Payment confirmation (simulated) */
function confirmPayment(){
  const name = document.getElementById('pay-name').value.trim();
  const number = document.getElementById('pay-number').value.trim();
  const expiry = document.getElementById('pay-expiry').value.trim();
  const cvv = document.getElementById('pay-cvv').value.trim();
  // simple validations
  if(!name){ showNotification('Please enter name for payment.', true); return; }
  if(!number || number.length < 6){ showNotification('Please enter a valid card/UPI number (simulated).', true); return; }
  if(!expiry || !/^\d{2}\/\d{2}$/.test(expiry)){ showNotification('Expiry must be in MM/YY format.', true); return; }
  if(!cvv || cvv.length<3){ showNotification('Enter CVV (3 digits).', true); return; }

  const b = window.currentBooking || {};
  b.payMethod = 'Simulated Card/UPI';
  b.paymentInfo = {name, numberMasked: maskNumber(number), date: new Date().toISOString()};
  b.confirmationId = 'BMS' + Math.floor(Math.random()*900000 + 100000);

  // save to localStorage bookings array
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  bookings.push(b);
  localStorage.setItem('bookings', JSON.stringify(bookings));

  // mark seats as booked
  for(const s of b.seats){
    const key = `${b.movieId}_${b.timing}_${s}`;
    bookedSeats[key] = true;
  }
  applyBookedVisuals();

  // populate confirmation page
  document.getElementById('conf-movie').innerText = b.movieTitle;
  document.getElementById('conf-time').innerText = b.timing;
  document.getElementById('conf-seats').innerText = b.seats.join(', ');
  document.getElementById('conf-email').innerText = b.email;
  document.getElementById('conf-total').innerText = `₹${b.total}`;

  // clear selected seats for the UI
  selectedSeats = [];
  updateSummary();

  // move to confirmation
  showPage('confirm');
}

/* Booking list rendering */
function renderBookings(){
  const container = document.getElementById('bookings-list');
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  container.innerHTML = '';
  if(!bookings.length){
    container.innerHTML = '<div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>';
    return;
  }
  for(const bk of bookings.slice().reverse()){
    const el = document.createElement('div');
    el.className = 'booking-item';
    el.innerHTML = `
      <div style="display:flex;justify-content:space-between;align-items:center">
        <div>
          <div style="font-weight:800; font-size: 18px;">${escapeHtml(bk.movieTitle)}</div>
          <div class="muted" style="margin-top: 8px;">${escapeHtml(bk.timing)} • Seats: ${escapeHtml(bk.seats.join(', '))}</div>
          <div class="muted" style="margin-top: 4px;">Email: ${escapeHtml(bk.email)}</div>
        </div>
        <div style="text-align:right">
          <div style="font-weight:800; font-size: 20px;">₹${bk.total}</div>
          <div class="muted" style="margin-top: 4px;">Conf: ${escapeHtml(bk.confirmationId || '')}</div>
        </div>
      </div>
    `;
    container.appendChild(el);
  }
}

/* Helpers */
function maskNumber(num){
  if(num.length<=4) return '****';
  const last = num.slice(-4);
  return '**** **** **** ' + last;
}
function validateEmail(email){
  if(!email) return false;
  // simple regex
  const re = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return re.test(email);
}
function escapeHtml(text){
  if(!text) return '';
  return text.replace(/&/g,'&amp;').replace(/</g,'&lt;').replace(/>/g,'&gt;');
}

/* Logout function clears temporary bookings and selections (and local storage if desired) */
function logout(){
  if(confirm('Logout will clear session-selected data (bookings persist). Proceed?')){
    selectedSeats = [];
    window.currentBooking = null;
    // Optionally clear localStorage bookings: commented out to preserve bookings
    // localStorage.removeItem('bookings');
    // just go to home
    showPage('home');
    showNotification('Logged out successfully');
  }
}

/* Utility to set initial booking page movie/time on load if needed */
document.addEventListener('DOMContentLoaded', init);

/* ensure quick UI functions for links that call from html attributes */
function goToBook(movieId){ currentMovie = movieId; populateDetails(movieId); clearSelection(); renderSeatMap(); showPage('book'); }
function goToDetails(movieId){ currentMovie = movieId; populateDetails(movieId); showPage('details'); }

// ensure show-page when clicking elements by id
document.getElementById('btn-see-showing').addEventListener('click', ()=> showPage('showing'));
document.getElementById('btn-see-upcoming').addEventListener('click', ()=> showPage('upcoming'));

// Keep seat visuals consistent when timing or movie changes
document.getElementById('show-timing').addEventListener('change', ()=> { currentTiming = document.getElementById('show-timing').value; applyBookedVisuals(); });

// In case someone navigates directly to payment without currentBooking, prevent errors:
window.addEventListener('beforeunload', ()=>{}); // placeholder

</script>
</body>
</html>
"""

# write html file
p = Path(HTML_FILENAME)
p.write_text(html_code, encoding='utf-8')
print(f"Written {HTML_FILENAME} ({p.stat().st_size} bytes)")

# Serve the file using a simple HTTP server in a background thread
class QuietHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, format, *args):
        # suppress console spam from the server; comment out if you want logs
        return

def start_server(port=PORT):
    Handler = QuietHTTPRequestHandler
    os.chdir(os.path.dirname(os.path.abspath(HTML_FILENAME)) or ".")
    with socketserver.TCPServer(("", port), Handler) as httpd:
        print(f"Serving HTTP on 0.0.0.0 port {port} (http://localhost:{port}/{HTML_FILENAME}) ...")
        httpd.serve_forever()

# start server in background thread to keep notebook interactive
server_thread = threading.Thread(target=start_server, args=(PORT,), daemon=True)
server_thread.start()

# open the page in default browser (give the server a moment to start)
time.sleep(0.6)
url = f"http://localhost:{PORT}/{HTML_FILENAME}"
print(f"Opening {url} in your web browser...")
webbrowser.open(url)

Exception in thread Thread-9 (start_server):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/opt/anaconda3/lib/python3.13/threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/gr/87c8xy3d21q8n0tlcwllwc800000gn/T/ipykernel_13874/382048741.py", line 1577, in start_server
    with socketserver.TCPServer(("", port), Handler) as httpd:
         ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 457, in __init__
    self.server_bind()
    ~~~~~~~~~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 478, in server_bind
    self.socket.bind(self.server_address)
 

Written bookmyshow.html (58609 bytes)
Opening http://localhost:8000/bookmyshow.html in your web browser...


True

In [5]:
# bookmyshow_prototype.py
# Run this cell in Jupyter Notebook. It will write bookmyshow.html, start a local HTTP server
# and open the page in your browser.
# Uses only standard libraries.

import os
import http.server
import socketserver
import threading
import webbrowser
import time
from pathlib import Path

HTML_FILENAME = "bookmyshow.html"
PORT = 8000

html_code = r"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>BookMyShow Prototype</title>
<style>
  :root{
    --primary:#E50914; /* BookMyShow-like red */
    --dark:#222;
    --muted:#666;
    --success:#2ecc71;
    --danger:#e74c3c;
    --white:#ffffff;
    --light-bg:#f8f9fa;
    --card-shadow: 0 8px 24px rgba(0,0,0,0.12);
    --hover-shadow: 0 16px 40px rgba(0,0,0,0.18);
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  }
  * {
    box-sizing: border-box;
  }
  html,body{
    height:100%;
    margin:0;
    padding:0;
    background:var(--light-bg);
    color:var(--dark);
    overflow-x: hidden;
  }
  body {
    display: flex;
    flex-direction: column;
    min-height: 100vh;
  }
  .container{
    width: 100%;
    max-width: 100%;
    margin: 0;
    padding: 0;
    flex: 1;
  }
  header{
    display:flex;
    align-items:center;
    justify-content:space-between;
    padding: 16px 5%;
    background: white;
    box-shadow: 0 2px 10px rgba(0,0,0,0.08);
    position: sticky;
    top: 0;
    z-index: 100;
  }
  .brand{
    display:flex;
    gap:16px;
    align-items:center;
  }
  .logo{
    width:52px;height:52px;border-radius:8px;background:var(--primary);display:flex;align-items:center;justify-content:center;color:#fff;font-weight:bold;font-size:20px;
    box-shadow:var(--card-shadow);
  }
  nav a{
    margin-right:20px;color:var(--dark);text-decoration:none;font-weight:600;
    padding: 10px 16px;
    border-radius: 8px;
    transition: all 0.3s ease;
    font-size: 16px;
  }
  nav a:hover{
    background: #f5f5f5;
    transform: translateY(-2px);
  }
  nav a.active{color:var(--primary); background: #ffeaea; box-shadow: 0 4px 12px rgba(229, 9, 20, 0.15);}
  .hero{
    display:flex;
    gap:30px;
    align-items:flex-start;
    margin-top:0;
    padding: 30px 5%;
    background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%);
    min-height: 500px;
  }
  .card{
    background:#fff;border-radius:16px;padding:24px;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    border: 1px solid rgba(0,0,0,0.05);
  }
  .card:hover {
    box-shadow: var(--hover-shadow);
    transform: translateY(-5px);
  }
  /* LARGER MOVIE GRID STYLES */
  .movie-grid{
    display: grid;
    grid-template-columns: repeat(auto-fill, minmax(320px, 1fr));
    gap: 30px;
    padding: 30px 5%;
  }
  .movie{
    border-radius:16px;overflow:hidden;background:#fff;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    position: relative;
  }
  .movie:hover {
    transform: translateY(-8px);
    box-shadow: var(--hover-shadow);
  }
  .poster{
    width:100%;height:480px;background-size:cover;background-position:center;
    transition: transform 0.6s ease;
  }
  .movie:hover .poster {
    transform: scale(1.08);
  }
  .movie-body{padding:24px;}
  .btn{
    display:inline-block;
    padding:14px 24px;
    border-radius:10px;
    text-decoration:none;
    color:#fff;
    background:var(--primary);
    border:none;
    cursor:pointer;
    font-weight:700;
    font-size: 16px;
    transition: all 0.3s ease;
    text-align: center;
  }
  .btn:hover {
    background: #c40811;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(229, 9, 20, 0.35);
  }
  .btn.secondary{background:#6c757d;color:#fff}
  .btn.secondary:hover {
    background: #5a6268;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(0, 0, 0, 0.2);
  }
  .btn-large {
    padding: 16px 28px;
    font-size: 17px;
  }
  .muted{color:var(--muted);font-size:16px}
  .small{font-size:14px}
  footer{
    margin-top:40px;
    padding: 30px 5%;
    text-align:center;
    color:var(--muted);
    background: white;
    border-top: 1px solid #eee;
  }
  /* Seat map */
  .seat-map-container {
    padding: 20px 5%;
  }
  .seat-map{display:grid;gap:10px; margin: 0 auto; max-width: 800px;}
  .seat-row{display:flex;gap:10px;align-items:center; justify-content: center;}
  .seat{
    width:44px;height:44px;border-radius:8px;background:#e6e6e6;display:flex;align-items:center;justify-content:center;cursor:pointer;user-select:none;
    transition: all 0.3s ease;
    font-weight: 600;
    font-size: 14px;
  }
  .seat.available{background:#b6f0c8; color: #155724;} /* available green-ish */
  .seat.available:hover{background:#9ae6b2; transform: scale(1.15); box-shadow: 0 4px 8px rgba(0,0,0,0.15);}
  .seat.booked{background:#ccc; cursor:not-allowed; opacity:0.6; color: #666;}
  .seat.selected{background:#ff7070; color: white;} /* selected red-ish */
  .seat.selected:hover{background:#ff5252; transform: scale(1.15); box-shadow: 0 4px 8px rgba(255, 80, 80, 0.3);}
  .seat.plate{background:#ffd9d9}
  .legend{
    display:flex;
    gap:20px;
    align-items:center;
    margin-top:12px;
    justify-content: center;
    flex-wrap: wrap;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    max-width: 800px;
    margin: 20px auto;
  }
  .legend .box{
    width:22px;height:22px;border-radius:6px;display:inline-block;margin-right:8px;vertical-align:middle
  }
  .flex{display:flex;gap:16px;align-items:center}
  .spacer{height:16px}
  .summary{
    margin-top:16px;
    padding:20px;
    background:#fafafa;
    border-radius:12px;
    border: 1px solid #eee;
  }
  input[type="text"], input[type="email"], select {
    padding:12px 16px;
    border-radius:8px;
    border:1px solid #ddd;
    width:100%;
    transition: all 0.3s ease;
    font-size: 15px;
    background: white;
  }
  input[type="text"]:focus, input[type="email"]:focus, select:focus {
    border-color: var(--primary);
    outline: none;
    box-shadow: 0 0 0 3px rgba(229, 9, 20, 0.15);
    transform: translateY(-2px);
  }
  label{
    display:block;
    margin-bottom:8px;
    font-weight:600;
    font-size: 15px;
  }
  .row{
    display:flex;
    gap:20px;
    flex-wrap:wrap;
    padding: 20px 5%;
  }
  .page-section {
    padding: 40px 5%;
  }
  /* small details */
  .topline{font-size:15px;color:var(--muted)}
  .badge{
    background:#f1f1f1;
    padding:8px 16px;
    border-radius:10px;
    font-size:15px;
    font-weight: 600;
  }
  .center{text-align:center}
  .hidden{display:none}
  .booking-list{
    margin-top:16px;
    padding: 0 5%;
  }
  .booking-item{
    padding:20px;
    border-radius:12px;
    background:#fff;
    margin-bottom:16px;
    box-shadow:var(--card-shadow);
    transition: all 0.3s ease;
    border-left: 4px solid var(--primary);
  }
  .booking-item:hover {
    box-shadow: var(--hover-shadow);
    transform: translateX(5px);
  }
  .footer-small{font-size:14px;color:var(--muted)}
  .input-inline{display:flex;gap:12px;align-items:center}
  .screen {
    background: linear-gradient(135deg, #333 0%, #555 100%);
    color: white;
    text-align: center;
    padding: 16px;
    margin: 30px auto;
    border-radius: 8px;
    font-weight: bold;
    letter-spacing: 3px;
    font-size: 18px;
    max-width: 800px;
    box-shadow: 0 8px 16px rgba(0,0,0,0.2);
    text-transform: uppercase;
  }
  .rating {
    display: inline-flex;
    align-items: center;
    background: #f1c40f;
    color: #333;
    padding: 6px 12px;
    border-radius: 8px;
    font-weight: bold;
    font-size: 15px;
    margin-left: 12px;
  }
  .movie-meta {
    display: flex;
    align-items: center;
    gap: 12px;
    margin-top: 12px;
    flex-wrap: wrap;
  }
  .genre-tag {
    background: #f0f0f0;
    padding: 6px 12px;
    border-radius: 8px;
    font-size: 14px;
    font-weight: 600;
  }
  .price-tag {
    color: var(--primary);
    font-weight: bold;
    font-size: 18px;
  }
  .notification {
    position: fixed;
    top: 30px;
    right: 30px;
    padding: 16px 24px;
    background: var(--success);
    color: white;
    border-radius: 10px;
    box-shadow: var(--hover-shadow);
    z-index: 1000;
    transform: translateX(150%);
    transition: transform 0.4s ease;
    font-weight: 600;
    max-width: 400px;
  }
  .notification.show {
    transform: translateX(0);
  }
  .notification.error {
    background: var(--danger);
  }
  .section-title {
    font-size: 42px;
    margin-bottom: 16px;
    color: var(--dark);
    font-weight: 800;
    text-align: center;
  }
  .section-subtitle {
    font-size: 20px;
    color: var(--muted);
    margin-bottom: 40px;
    max-width: 700px;
    text-align: center;
    margin-left: auto;
    margin-right: auto;
    line-height: 1.6;
  }
  .welcome-message {
    font-size: 24px;
    margin-bottom: 10px;
    color: var(--dark);
  }
  .welcome-subtitle {
    font-size: 18px;
    color: var(--muted);
    margin-bottom: 30px;
  }
  .hero-left {
    flex: 1;
    min-width: 0;
  }
  .hero-right {
    width: 480px;
    min-width: 480px;
  }
  .quick-actions {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 16px;
    margin-top: 20px;
  }
  .stats {
    display: flex;
    gap: 20px;
    margin-top: 30px;
  }
  .stat-item {
    text-align: center;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    flex: 1;
  }
  .stat-number {
    font-size: 28px;
    font-weight: 800;
    color: var(--primary);
    display: block;
  }
  .stat-label {
    font-size: 14px;
    color: var(--muted);
    margin-top: 5px;
  }
  /* LARGER MOVIE TITLE STYLES */
  .movie-title {
    font-size: 24px;
    font-weight: 800;
    margin-bottom: 8px;
    line-height: 1.2;
  }
  .movie-description {
    font-size: 16px;
    line-height: 1.6;
    color: var(--muted);
    margin-top: 12px;
  }
  .movie-actions {
    display: flex;
    gap: 12px;
    margin-top: 20px;
  }
  .movie-info {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-top: 16px;
  }
  .movie-duration {
    font-size: 15px;
    color: var(--muted);
  }
  /* UPCOMING MOVIE STYLES */
  .upcoming-movie {
    position: relative;
    overflow: hidden;
  }
  .upcoming-overlay {
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: linear-gradient(180deg, rgba(0,0,0,0.1) 0%, rgba(0,0,0,0.7) 100%);
    display: flex;
    flex-direction: column;
    justify-content: flex-end;
    padding: 24px;
    color: white;
  }
  .coming-soon-badge {
    background: var(--primary);
    color: white;
    padding: 10px 20px;
    border-radius: 25px;
    font-weight: 700;
    font-size: 16px;
    display: inline-block;
    margin-bottom: 12px;
  }
  @media (max-width: 1200px) {
    .hero-right {
      width: 400px;
      min-width: 400px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(300px, 1fr));
    }
  }
  @media (max-width: 992px) {
    .hero {
      flex-direction: column;
    }
    .hero-right {
      width: 100%;
      min-width: auto;
    }
    .quick-actions {
      grid-template-columns: 1fr;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
      gap: 20px;
    }
    .poster {
      height: 420px;
    }
  }
  @media (max-width: 768px) {
    header {
      flex-direction: column;
      gap: 16px;
      padding: 16px 20px;
    }
    nav {
      width: 100%;
      display: flex;
      justify-content: space-between;
    }
    nav a {
      margin-right: 0;
      flex: 1;
      text-align: center;
      font-size: 14px;
      padding: 8px 12px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(260px, 1fr));
      padding: 20px;
      gap: 16px;
    }
    .row {
      padding: 20px;
    }
    .page-section {
      padding: 30px 20px;
    }
    .section-title {
      font-size: 36px;
    }
    .section-subtitle {
      font-size: 18px;
    }
    .poster {
      height: 380px;
    }
  }
  @media (max-width: 480px) {
    .movie-grid {
      grid-template-columns: 1fr;
    }
    .poster {
      height: 500px;
    }
  }
</style>
</head>
<body>
<div class="container">
  <header>
    <div class="brand">
      <div class="logo">BMS</div>
      <div>
        <div style="font-weight:800;font-size:22px">BookMyShow Prototype</div>
        <div class="topline">Welcome, <strong>Avaneesh Thakur</strong></div>
      </div>
    </div>
    <nav>
      <a href="#" id="nav-home" class="active">Home</a>
      <a href="#" id="nav-showing">Currently Showing</a>
      <a href="#" id="nav-upcoming">Upcoming</a>
      <a href="#" id="nav-bookings">My Bookings</a>
      <a href="#" id="nav-logout">Logout</a>
    </nav>
  </header>

  <main style="flex: 1;">
    <!-- HOME -->
    <section id="page-home">
      <div class="hero">
        <div class="hero-left">
          <div class="card">
            <div class="welcome-message">Hello, Avaneesh Thakur 👋</div>
            <div class="welcome-subtitle">Welcome to your BookMyShow prototype. Choose a movie to book tickets. Prices shown in ₹ (INR)</div>
            
            <div class="stats">
              <div class="stat-item">
                <span class="stat-number">12</span>
                <span class="stat-label">Movies Available</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">5</span>
                <span class="stat-label">Cities</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">24/7</span>
                <span class="stat-label">Support</span>
              </div>
            </div>
            
            <div class="spacer"></div>
            <div class="quick-actions">
              <a class="btn btn-large" id="btn-see-showing">See Currently Showing</a>
              <a class="btn btn-large secondary" id="btn-see-upcoming">See Upcoming</a>
            </div>
          </div>
          <div style="height:20px"></div>
          <div class="card">
            <h3 style="margin-top:0; font-size: 24px;">Quick Booking</h3>
            <p class="muted" style="font-size: 16px;">Choose a movie and go directly to booking</p>
            <div style="display:flex;gap:12px;align-items:center;margin-top:16px;">
              <select id="quick-movie" style="flex: 1;">
                <option value="jawan" selected>Jawan (Currently Showing)</option>
                <option value="animal">Animal (Currently Showing)</option>
                <option value="salaar">Salaar (Currently Showing)</option>
                <option value="kalki">Kalki 2898 AD (Upcoming)</option>
                <option value="pushpa">Pushpa 2 (Upcoming)</option>
              </select>
              <a class="btn btn-large" id="quick-book">Book Now</a>
            </div>
          </div>
        </div>
        <div class="hero-right">
          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Currently Showing</h4>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.ibb.co/q-F9-Nz-K5s/jawan.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Jawan</div>
                <div class="muted">Action • 2h 30m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="jawan" id="home-book-jawan">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.imgur.com/9Qb8Y8S.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Animal</div>
                <div class="muted">Action • 3h 21m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="animal" id="home-book-animal">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center">
              <div style="width:100px;height:140px;background-image:url('https://i.imgur.com/8Qb8Y8T.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Salaar</div>
                <div class="muted">Action • 2h 55m • Telugu/Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="salaar" id="home-book-salaar">Book Tickets</a>
                </div>
              </div>
            </div>
          </div>

          <div style="height:20px"></div>

          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Upcoming</h4>
            <div style="display:flex;gap:16px;flex-direction:column">
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.imgur.com/2QY8kPz.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Kalki 2898 AD</div>
                  <div class="muted small">Sci-Fi • December 2025</div>
                </div>
              </div>
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.imgur.com/8sQmN7R.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Pushpa 2</div>
                  <div class="muted small">Action • December 2025</div>
                </div>
              </div>
            </div>
          </div>

        </div>
      </div>
    </section>

    <!-- SHOWING - LARGER LAYOUT -->
    <section id="page-showing" class="hidden">
      <div class="page-section">
        <div class="section-title">Currently Showing</div>
        <div class="section-subtitle">Book tickets for the latest blockbusters in theaters now. Experience the biggest movies on the largest screens.</div>
        
        <div class="movie-grid" id="showing-grid">
          <!-- Movie: Jawan - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/q-F9-Nz-K5s/jawan.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Jawan <span class="rating">8.2/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 30m • Hindi • UA</div>
                <div class="price-tag">₹180 - ₹350</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="jawan" onclick="goToDetails('jawan')">View Details</a>
                <a class="btn secondary" onclick="goToBook('jawan')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Animal - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/9Qb8Y8S.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Animal <span class="rating">7.8/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Crime</span>
              </div>
              <div class="movie-description">
                A father, who is often away due to work, is unable to comprehend the intensity of his son's love and admiration.
              </div>
              <div class="movie-info">
                <div class="movie-duration">3h 21m • Hindi • A</div>
                <div class="price-tag">₹200 - ₹400</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="animal" onclick="goToDetails('animal')">View Details</a>
                <a class="btn secondary" onclick="goToBook('animal')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Salaar - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/8Qb8Y8T.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Salaar <span class="rating">8.5/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs in an epic showdown.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 55m • Telugu/Hindi • UA</div>
                <div class="price-tag">₹190 - ₹380</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="salaar" onclick="goToDetails('salaar')">View Details</a>
                <a class="btn secondary" onclick="goToBook('salaar')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Dunki -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/6Qb8Y8U.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Dunki <span class="rating">7.9/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Comedy</span>
                <span class="genre-tag">Adventure</span>
              </div>
              <div class="movie-description">
                A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called 'Donkey Flight'.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 41m • Hindi • UA</div>
                <div class="price-tag">₹170 - ₹320</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="dunki" onclick="goToDetails('dunki')">View Details</a>
                <a class="btn secondary" onclick="goToBook('dunki')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Fighter -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/7Qb8Y8V.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Fighter <span class="rating">8.1/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Patriotic</span>
              </div>
              <div class="movie-description">
                An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 26m • Hindi • UA</div>
                <div class="price-tag">₹190 - ₹360</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="fighter" onclick="goToDetails('fighter')">View Details</a>
                <a class="btn secondary" onclick="goToBook('fighter')">Book Now</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- UPCOMING - LARGER LAYOUT -->
    <section id="page-upcoming" class="hidden">
      <div class="page-section">
        <div class="section-title">Upcoming Movies</div>
        <div class="section-subtitle">Get ready for these exciting releases coming soon to theaters. Mark your calendars for the most anticipated films of 2025.</div>
        
        <div class="movie-grid">
          <!-- Kalki 2898 AD - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/2QY8kPz.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Kalki 2898 AD</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Sci-Fi • Epic • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience.
              </div>
            </div>
          </div>

          <!-- Pushpa 2 - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/8sQmN7R.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Pushpa 2: The Rule</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Drama • Crime</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Devara -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/9Qb8Y8W.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Devara</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Period • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Singham Again -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/1Qb8Y8X.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Singham Again</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Cop • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The third installment in the popular Singham franchise, featuring Rohit Shetty's cop universe with high-octane action and drama.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: War 2 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/3Qb8Y8Y.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">War 2</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Spy • Thriller</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Baahubali 3 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/4Qb8Y8Z.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Baahubali 3</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Epic • Fantasy • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- MOVIE DETAILS -->
    <section id="page-details" class="hidden">
      <div class="page-section">
        <div class="card" id="details-card">
          <div style="display:flex;gap:24px;align-items:flex-start">
            <div style="width:240px;height:340px;background-size:cover;background-position:center;border-radius:12px; box-shadow: var(--card-shadow);" id="details-poster"></div>
            <div style="flex:1">
              <h2 id="details-title" style="margin:0; font-size: 32px;">Movie Title</h2>
              <div class="muted" id="details-meta" style="font-size: 18px; margin: 10px 0;">Meta</div>
              <p id="details-desc" style="font-size: 16px; line-height: 1.6;">Description</p>
              <div style="margin-top:20px">
                <a class="btn btn-large" id="details-book">Book Tickets</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- BOOK TICKETS -->
    <section id="page-book" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;" id="book-title">Book Tickets</h2>
          <div class="row">
            <div style="flex:2; min-width: 500px;">
              <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
                <div>
                  <label>Show Timing</label>
                  <select id="show-timing">
                    <option>10:00 AM</option>
                    <option>01:30 PM</option>
                    <option>05:00 PM</option>
                    <option>08:30 PM</option>
                  </select>
                </div>

                <div>
                  <label>Seat Category</label>
                  <select id="seat-category">
                    <option value="platinum">Platinum (Rows A-B) — ₹350</option>
                    <option value="gold">Gold (Row C) — ₹250</option>
                    <option value="silver">Silver (Rows D-E) — ₹180</option>
                  </select>
                </div>
              </div>

              <div class="spacer"></div>

              <div style="margin-top:16px">
                <div style="font-weight:700; font-size: 20px;">Select Seats</div>
                <div class="muted" style="font-size: 16px;">Click seats to select/deselect</div>
                
                <div class="legend">
                  <div><span class="box" style="background:#b6f0c8"></span>Available</div>
                  <div><span class="box" style="background:#ff7070"></span>Selected</div>
                  <div><span class="box" style="background:#ccc"></span>Booked/Unavailable</div>
                </div>

                <div style="height:16px"></div>
                
                <div class="screen">SCREEN</div>

                <div class="seat-map card" id="seat-map" style="padding:20px"></div>

              </div>

            </div>

            <div style="flex:1; min-width: 350px;">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Booking Summary</h3>
                <div class="summary">
                  <div style="margin-bottom: 12px;">Movie: <strong id="sum-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Timing: <strong id="sum-time" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="sum-seats" style="float: right;">None</strong></div>
                  <div style="margin-bottom: 12px;">Price per seat: <strong id="sum-price" style="float: right;">₹0</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <span id="sum-total" style="float: right;">₹0</span></div>
                  <div style="height:20px"></div>
                  <label for="email-input">Email for Confirmation</label>
                  <input type="email" id="email-input" placeholder="you@example.com" />
                  <div style="height:16px"></div>
                  <div style="display:flex;gap:12px">
                    <a class="btn" id="proceed-payment" style="flex: 2;">Proceed to Payment</a>
                    <a class="btn secondary" id="clear-selection" style="flex: 1;">Clear</a>
                  </div>
                </div>
              </div>
            </div>
          </div>
        </div> 
      </div>
    </section>

    <!-- PAYMENT -->
    <section id="page-payment" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">Payment</h2>
          <div class="row">
            <div style="flex:1;min-width:500px">
              <div style="margin-bottom:20px">
                <label>Card / UPI Name</label>
                <input type="text" id="pay-name" placeholder="Avaneesh Thakur" value="Avaneesh Thakur" />
              </div>
              <div style="margin-bottom:20px">
                <label>Card / UPI Number</label>
                <input type="text" id="pay-number" placeholder="Enter 12-16 digit card or UPI id" />
              </div>
              <div style="display:grid; grid-template-columns: 1fr 1fr; gap: 16px;">
                <div>
                  <label>Expiry</label>
                  <input type="text" id="pay-expiry" placeholder="MM/YY" />
                </div>
                <div>
                  <label>CVV</label>
                  <input type="text" id="pay-cvv" placeholder="123" />
                </div>
              </div>
              <div style="height:20px"></div>
              <div class="muted">This is a simulated payment — no real transaction will occur.</div>
              <div style="height:20px"></div>
              <div style="display:flex;gap:12px">
                <a class="btn btn-large" id="confirm-pay">Pay & Confirm</a>
                <a class="btn secondary btn-large" id="back-to-book">Back</a>
              </div>
            </div>
            <div style="width:400px">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Payment Summary</h3>
                <div style="margin-top:16px">
                  <div style="margin-bottom: 12px;">Movie: <strong id="pay-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="pay-seats" style="float: right;">-</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <strong id="pay-total" style="float: right;">₹0</strong></div>
                  <div style="height:16px"></div>
                  <div class="muted">A confirmation email will be "sent" to the address you provided (simulated).</div>
                </div>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- CONFIRMATION -->
    <section id="page-confirm" class="hidden">
      <div class="page-section">
        <div class="card" style="max-width: 800px; margin: 0 auto;">
          <h2 style="font-size: 32px; text-align: center;">Booking Confirmed! 🎉</h2>
          <p style="font-size: 18px; text-align: center;">Thank you, <strong>Avaneesh Thakur</strong>. Your booking is confirmed.</p>
          <div class="card" style="background: #f8f9fa;">
            <div style="margin-bottom: 16px; font-size: 16px;">Movie: <strong id="conf-movie" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Timing: <strong id="conf-time" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Seats: <strong id="conf-seats" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Email: <strong id="conf-email" style="float: right;"></strong></div>
            <div style="height:12px; border-bottom: 1px solid #ddd; margin-bottom: 16px;"></div>
            <div style="font-weight:800;font-size:24px">Total Paid: <strong id="conf-total" style="float: right;"></strong></div>
          </div>
          <div style="height:20px"></div>
          <div style="display:flex;gap:12px; justify-content: center;">
            <a class="btn btn-large" id="conf-home">Return to Homepage</a>
            <a class="btn secondary btn-large" id="conf-bookings">My Bookings</a>
          </div>
        </div>
      </div>
    </section>

    <!-- MY BOOKINGS -->
    <section id="page-bookings" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">My Bookings</h2>
          <div id="bookings-list">
            <div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>
          </div>
        </div>
      </div>
    </section>

  </main>

  <footer>
    <div class="footer-small">Made by Avaneesh Thakur • BookMyShow Prototype • 2024</div>
  </footer>
</div>

<div id="notification" class="notification"></div>

<script>
// Update MOVIES object with December 2025 release dates and Jawan image URL
const MOVIES = {
  jawan: {
    id:'jawan',
    title:'Jawan',
    poster:'https://i.ibb.co/q-F9-Nz-K5s/jawan.jpg',
    meta:'Action • 2h 30m • Hindi',
    desc:'A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society, while keeping a promise made years ago. He comes up against a monstrous outlaw with no fear, who has caused extreme suffering to many.',
  },
  animal: {
    id:'animal',
    title:'Animal',
    poster:'https://i.imgur.com/9Qb8Y8S.jpg',
    meta:'Action • 3h 21m • Hindi',
    desc:'A father, who is often away due to work, is unable to comprehend the intensity of his son\'s love. Ironically, this fervent love and admiration for his father and family creates conflict between the father and son, which leads the son to become a threat to many.',
  },
  salaar: {
    id:'salaar',
    title:'Salaar',
    poster:'https://i.imgur.com/8Qb8Y8T.jpg',
    meta:'Action • 2h 55m • Telugu/Hindi',
    desc:'A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs. He is forced to become the saviour of the people when the city becomes a battleground for powerful and ruthless criminal forces.',
  },
  dunki: {
    id:'dunki',
    title:'Dunki',
    poster:'https://i.imgur.com/6Qb8Y8U.jpg',
    meta:'Drama • 2h 41m • Hindi',
    desc:'A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called Donkey Flight. The film explores themes of friendship, dreams, and the price of ambition.',
  },
  fighter: {
    id:'fighter',
    title:'Fighter',
    poster:'https://i.imgur.com/7Qb8Y8V.jpg',
    meta:'Action • 2h 26m • Hindi',
    desc:'An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet. The film showcases breathtaking aerial combat sequences and the personal sacrifices of our armed forces.',
  },
  kalki: {
    id:'kalki',
    title:'Kalki 2898 AD',
    poster:'https://i.imgur.com/2QY8kPz.jpg',
    meta:'Sci-Fi • December 2025',
    desc:'A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience. The film promises to redefine Indian cinema with its scale and visual effects.',
  },
  pushpa: {
    id:'pushpa',
    title:'Pushpa 2: The Rule',
    poster:'https://i.imgur.com/8sQmN7R.jpg',
    meta:'Action • December 2025',
    desc:'The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world. The film promises even more action and drama than its predecessor.',
  },
  devara: {
    id:'devara',
    title:'Devara',
    poster:'https://i.imgur.com/9Qb8Y8W.jpg',
    meta:'Action • December 2025',
    desc:'A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.',
  },
  singham: {
    id:'singham',
    title:'Singham Again',
    poster:'https://i.imgur.com/1Qb8Y8X.jpg',
    meta:'Action • December 2025',
    desc:'The third installment in the popular Singham franchise, featuring Rohit Shetty\'s cop universe with high-octane action and drama.',
  },
  war2: {
    id:'war2',
    title:'War 2',
    poster:'https://i.imgur.com/3Qb8Y8Y.jpg',
    meta:'Action • December 2025',
    desc:'The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.',
  },
  baahubali3: {
    id:'baahubali3',
    title:'Baahubali 3',
    poster:'https://i.imgur.com/4Qb8Y8Z.jpg',
    meta:'Epic • December 2025',
    desc:'The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.',
  }
};

// Rest of the JavaScript code remains exactly the same as previous version
// Only the MOVIES object has been updated with December 2025 dates and Jawan image URL

/* Single-page navigation */
const pages = {
  home: document.getElementById('page-home'),
  showing: document.getElementById('page-showing'),
  upcoming: document.getElementById('page-upcoming'),
  details: document.getElementById('page-details'),
  book: document.getElementById('page-book'),
  payment: document.getElementById('page-payment'),
  confirm: document.getElementById('page-confirm'),
  bookings: document.getElementById('page-bookings'),
};

function showPage(name){
  for(const p of Object.values(pages)) p.classList.add('hidden');
  pages[name].classList.remove('hidden');
  // navbar active states
  document.querySelectorAll('nav a').forEach(a=>a.classList.remove('active'));
  if(name==='home') document.getElementById('nav-home').classList.add('active');
  if(name==='showing') document.getElementById('nav-showing').classList.add('active');
  if(name==='upcoming') document.getElementById('nav-upcoming').classList.add('active');
  if(name==='bookings') document.getElementById('nav-bookings').classList.add('active');
}

/* Seat map state */
const ROWS = ['A','B','C','D','E'];
const SEATS_PER_ROW = 10;
let bookedSeats = {}; // { 'jawan_10:00_A3': true } to mark booked for simulation
let selectedSeats = []; // list of seat ids e.g., 'A3'
let currentMovie = 'jawan';
let currentTiming = '10:00 AM';

const PRICES = {
  platinum:350,
  gold:250,
  silver:180
};

function showNotification(message, isError = false) {
  const notification = document.getElementById('notification');
  notification.textContent = message;
  notification.className = 'notification';
  if (isError) {
    notification.classList.add('error');
  }
  notification.classList.add('show');
  
  setTimeout(() => {
    notification.classList.remove('show');
  }, 3000);
}

function init(){
  // nav handlers
  document.getElementById('nav-home').addEventListener('click', e=>{ e.preventDefault(); showPage('home')});
  document.getElementById('nav-showing').addEventListener('click', e=>{ e.preventDefault(); showPage('showing')});
  document.getElementById('nav-upcoming').addEventListener('click', e=>{ e.preventDefault(); showPage('upcoming')});
  document.getElementById('nav-bookings').addEventListener('click', e=>{ e.preventDefault(); renderBookings(); showPage('bookings')});
  document.getElementById('nav-logout').addEventListener('click', e=>{ e.preventDefault(); logout(); });

  document.getElementById('btn-see-showing').addEventListener('click', ()=>{ showPage('showing')});
  document.getElementById('btn-see-upcoming').addEventListener('click', ()=>{ showPage('upcoming')});
  document.getElementById('home-book-jawan').addEventListener('click', ()=> goToBook('jawan'));
  document.getElementById('home-book-animal').addEventListener('click', ()=> goToBook('animal'));
  document.getElementById('home-book-salaar').addEventListener('click', ()=> goToBook('salaar'));
  document.getElementById('quick-book').addEventListener('click', ()=> {
    const m = document.getElementById('quick-movie').value;
    goToBook(m);
  });

  // details/book buttons
  document.getElementById('details-book').addEventListener('click', ()=> goToBook(currentMovie));

  // book page controls
  document.getElementById('seat-category').addEventListener('change', updateSummary);
  document.getElementById('show-timing').addEventListener('change', ()=> { 
    currentTiming = document.getElementById('show-timing').value; 
    updateSummary();
    applyBookedVisuals();
  });
  document.getElementById('clear-selection').addEventListener('click', clearSelection);
  document.getElementById('proceed-payment').addEventListener('click', proceedToPayment);

  // payment page controls
  document.getElementById('confirm-pay').addEventListener('click', confirmPayment);
  document.getElementById('back-to-book').addEventListener('click', ()=> showPage('book'));

  // confirmation page buttons
  document.getElementById('conf-home').addEventListener('click', ()=> showPage('home'));
  document.getElementById('conf-bookings').addEventListener('click', ()=> { renderBookings(); showPage('bookings'); });

  // initial render
  renderSeatMap();
  populateDetails('jawan');
  updateSummary();
  restoreBookedFromLocal();
}

function goToDetails(movieId){
  currentMovie = movieId;
  populateDetails(movieId);
  showPage('details');
}
function goToBook(movieId){
  currentMovie = movieId;
  // reset selection
  clearSelection();
  populateDetails(movieId);
  showPage('book');
}

/* Populate details card */
function populateDetails(movieId){
  const m = MOVIES[movieId];
  document.getElementById('details-poster').style.backgroundImage = `url('${m.poster}')`;
  document.getElementById('details-title').innerText = m.title;
  document.getElementById('details-meta').innerText = m.meta;
  document.getElementById('details-desc').innerText = m.desc;
  document.getElementById('book-title').innerText = `Book Tickets - ${m.title}`;
  document.getElementById('sum-movie').innerText = m.title;
  document.getElementById('pay-movie').innerText = m.title;
}

/* Seat map rendering */
function renderSeatMap(){
  const container = document.getElementById('seat-map');
  container.innerHTML = '';
  for(const row of ROWS){
    const rowEl = document.createElement('div');
    rowEl.className='seat-row';
    const label = document.createElement('div');
    label.style.width='28px';
    label.style.fontWeight='700';
    label.innerText = row;
    rowEl.appendChild(label);
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const seatEl = document.createElement('div');
      seatEl.className='seat available';
      seatEl.id = `seat-${seatId}`;
      seatEl.innerText = i;
      seatEl.dataset.seat = seatId;
      // assign category styles visually but selection price uses mapping
      if(row==='A' || row==='B') seatEl.title='Platinum';
      else if(row==='C') seatEl.title='Gold';
      else seatEl.title='Silver';

      seatEl.addEventListener('click', ()=> onSeatClick(seatId));
      rowEl.appendChild(seatEl);
    }
    container.appendChild(rowEl);
  }
  // mark already booked seats (simulate)
  applyBookedVisuals();
}

/* simulate restoring booked seats (from localStorage bookings) */
function restoreBookedFromLocal(){
  const b = JSON.parse(localStorage.getItem('bookings') || '[]');
  // mark as booked for current movie/timeslot by id
  bookedSeats = {}; // reset
  for(const item of b){
    if(!item.movieId || !item.timing || !item.seats) continue;
    // for simplicity mark seats as booked for that movie across times
    item.seats.forEach(seat=>{
      const key = `${item.movieId}_${item.timing}_${seat}`;
      bookedSeats[key] = true;
    });
  }
  applyBookedVisuals();
}

/* Check if a seat is booked for current movie/time */
function isSeatBookedGlobal(seatId){
  const key = `${currentMovie}_${currentTiming}_${seatId}`;
  return !!bookedSeats[key];
}

function applyBookedVisuals(){
  for(const row of ROWS){
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const el = document.getElementById(`seat-${seatId}`);
      if(!el) continue;
      if(isSeatBookedGlobal(seatId)){
        el.classList.remove('available','selected');
        el.classList.add('booked');
      } else {
        el.classList.remove('booked');
        el.classList.add('available');
      }
    }
  }
}

/* Seat click handler */
function onSeatClick(seatId){
  // can't select if booked
  if(isSeatBookedGlobal(seatId)) {
    showNotification('This seat is already booked!', true);
    return;
  }
  const el = document.getElementById(`seat-${seatId}`);
  if(!el) return;
  if(el.classList.contains('selected')){
    el.classList.remove('selected');
    selectedSeats = selectedSeats.filter(s=>s!==seatId);
    showNotification(`Seat ${seatId} removed from selection`);
  } else {
    el.classList.add('selected');
    selectedSeats.push(seatId);
    showNotification(`Seat ${seatId} added to selection`);
  }
  updateSummary();
}

/* Clear selection */
function clearSelection(){
  selectedSeats = [];
  document.querySelectorAll('.seat.selected').forEach(s => s.classList.remove('selected'));
  updateSummary();
  showNotification('Selection cleared');
}

/* Update summary panel */
function updateSummary(){
  // movie and time
  document.getElementById('sum-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('sum-time').innerText = document.getElementById('show-timing').value;
  currentTiming = document.getElementById('show-timing').value;
  // price per seat depends on selected category
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  document.getElementById('sum-price').innerText = `₹${price}`;
  // seats
  document.getElementById('sum-seats').innerText = selectedSeats.length ? selectedSeats.join(', ') : 'None';
  // total
  const total = selectedSeats.length * price;
  document.getElementById('sum-total').innerText = `₹${total}`;
  // payment summary fields
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ') || '-';
  document.getElementById('pay-total').innerText = `₹${total}`;
}

/* Proceed to payment validation (email required) */
function proceedToPayment(){
  const email = document.getElementById('email-input').value.trim();
  if(selectedSeats.length===0){
    showNotification('Please select at least one seat.', true);
    return;
  }
  if(!validateEmail(email)){
    showNotification('Please enter a valid email address.', true);
    return;
  }
  // populate payment page summary
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ');
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  const total = selectedSeats.length * price;
  document.getElementById('pay-total').innerText = `₹${total}`;
  // store interim booking info in session (page-level)
  window.currentBooking = {
    movieId: currentMovie,
    movieTitle: MOVIES[currentMovie].title,
    timing: currentTiming,
    seats: selectedSeats.slice(),
    category: cat,
    pricePerSeat: price,
    total: total,
    email: email,
    user: 'Avaneesh Thakur'
  };
  showPage('payment');
}

/* Payment confirmation (simulated) */
function confirmPayment(){
  const name = document.getElementById('pay-name').value.trim();
  const number = document.getElementById('pay-number').value.trim();
  const expiry = document.getElementById('pay-expiry').value.trim();
  const cvv = document.getElementById('pay-cvv').value.trim();
  // simple validations
  if(!name){ showNotification('Please enter name for payment.', true); return; }
  if(!number || number.length < 6){ showNotification('Please enter a valid card/UPI number (simulated).', true); return; }
  if(!expiry || !/^\d{2}\/\d{2}$/.test(expiry)){ showNotification('Expiry must be in MM/YY format.', true); return; }
  if(!cvv || cvv.length<3){ showNotification('Enter CVV (3 digits).', true); return; }

  const b = window.currentBooking || {};
  b.payMethod = 'Simulated Card/UPI';
  b.paymentInfo = {name, numberMasked: maskNumber(number), date: new Date().toISOString()};
  b.confirmationId = 'BMS' + Math.floor(Math.random()*900000 + 100000);

  // save to localStorage bookings array
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  bookings.push(b);
  localStorage.setItem('bookings', JSON.stringify(bookings));

  // mark seats as booked
  for(const s of b.seats){
    const key = `${b.movieId}_${b.timing}_${s}`;
    bookedSeats[key] = true;
  }
  applyBookedVisuals();

  // populate confirmation page
  document.getElementById('conf-movie').innerText = b.movieTitle;
  document.getElementById('conf-time').innerText = b.timing;
  document.getElementById('conf-seats').innerText = b.seats.join(', ');
  document.getElementById('conf-email').innerText = b.email;
  document.getElementById('conf-total').innerText = `₹${b.total}`;

  // clear selected seats for the UI
  selectedSeats = [];
  updateSummary();

  // move to confirmation
  showPage('confirm');
}

/* Booking list rendering */
function renderBookings(){
  const container = document.getElementById('bookings-list');
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  container.innerHTML = '';
  if(!bookings.length){
    container.innerHTML = '<div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>';
    return;
  }
  for(const bk of bookings.slice().reverse()){
    const el = document.createElement('div');
    el.className = 'booking-item';
    el.innerHTML = `
      <div style="display:flex;justify-content:space-between;align-items:center">
        <div>
          <div style="font-weight:800; font-size: 18px;">${escapeHtml(bk.movieTitle)}</div>
          <div class="muted" style="margin-top: 8px;">${escapeHtml(bk.timing)} • Seats: ${escapeHtml(bk.seats.join(', '))}</div>
          <div class="muted" style="margin-top: 4px;">Email: ${escapeHtml(bk.email)}</div>
        </div>
        <div style="text-align:right">
          <div style="font-weight:800; font-size: 20px;">₹${bk.total}</div>
          <div class="muted" style="margin-top: 4px;">Conf: ${escapeHtml(bk.confirmationId || '')}</div>
        </div>
      </div>
    `;
    container.appendChild(el);
  }
}

/* Helpers */
function maskNumber(num){
  if(num.length<=4) return '****';
  const last = num.slice(-4);
  return '**** **** **** ' + last;
}
function validateEmail(email){
  if(!email) return false;
  // simple regex
  const re = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return re.test(email);
}
function escapeHtml(text){
  if(!text) return '';
  return text.replace(/&/g,'&amp;').replace(/</g,'&lt;').replace(/>/g,'&gt;');
}

/* Logout function clears temporary bookings and selections (and local storage if desired) */
function logout(){
  if(confirm('Logout will clear session-selected data (bookings persist). Proceed?')){
    selectedSeats = [];
    window.currentBooking = null;
    // Optionally clear localStorage bookings: commented out to preserve bookings
    // localStorage.removeItem('bookings');
    // just go to home
    showPage('home');
    showNotification('Logged out successfully');
  }
}

/* Utility to set initial booking page movie/time on load if needed */
document.addEventListener('DOMContentLoaded', init);

/* ensure quick UI functions for links that call from html attributes */
function goToBook(movieId){ currentMovie = movieId; populateDetails(movieId); clearSelection(); renderSeatMap(); showPage('book'); }
function goToDetails(movieId){ currentMovie = movieId; populateDetails(movieId); showPage('details'); }

// ensure show-page when clicking elements by id
document.getElementById('btn-see-showing').addEventListener('click', ()=> showPage('showing'));
document.getElementById('btn-see-upcoming').addEventListener('click', ()=> showPage('upcoming'));

// Keep seat visuals consistent when timing or movie changes
document.getElementById('show-timing').addEventListener('change', ()=> { currentTiming = document.getElementById('show-timing').value; applyBookedVisuals(); });

// In case someone navigates directly to payment without currentBooking, prevent errors:
window.addEventListener('beforeunload', ()=>{}); // placeholder

</script>
</body>
</html>
"""

# write html file
p = Path(HTML_FILENAME)
p.write_text(html_code, encoding='utf-8')
print(f"Written {HTML_FILENAME} ({p.stat().st_size} bytes)")

# Serve the file using a simple HTTP server in a background thread
class QuietHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, format, *args):
        # suppress console spam from the server; comment out if you want logs
        return

def start_server(port=PORT):
    Handler = QuietHTTPRequestHandler
    os.chdir(os.path.dirname(os.path.abspath(HTML_FILENAME)) or ".")
    with socketserver.TCPServer(("", port), Handler) as httpd:
        print(f"Serving HTTP on 0.0.0.0 port {port} (http://localhost:{port}/{HTML_FILENAME}) ...")
        httpd.serve_forever()

# start server in background thread to keep notebook interactive
server_thread = threading.Thread(target=start_server, args=(PORT,), daemon=True)
server_thread.start()

# open the page in default browser (give the server a moment to start)
time.sleep(0.6)
url = f"http://localhost:{PORT}/{HTML_FILENAME}"
print(f"Opening {url} in your web browser...")
webbrowser.open(url)

Exception in thread Thread-10 (start_server):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/opt/anaconda3/lib/python3.13/threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/gr/87c8xy3d21q8n0tlcwllwc800000gn/T/ipykernel_13874/293789743.py", line 1577, in start_server
    with socketserver.TCPServer(("", port), Handler) as httpd:
         ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 457, in __init__
    self.server_bind()
    ~~~~~~~~~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 478, in server_bind
    self.socket.bind(self.server_address)


Written bookmyshow.html (58654 bytes)
Opening http://localhost:8000/bookmyshow.html in your web browser...


True

In [7]:
# bookmyshow_prototype.py
# Run this cell in Jupyter Notebook. It will write bookmyshow.html, start a local HTTP server
# and open the page in your browser.
# Uses only standard libraries.

import os
import http.server
import socketserver
import threading
import webbrowser
import time
from pathlib import Path

HTML_FILENAME = "bookmyshow.html"
PORT = 8000

html_code = r"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>BookMyShow Prototype</title>
<style>
  :root{
    --primary:#E50914; /* BookMyShow-like red */
    --dark:#222;
    --muted:#666;
    --success:#2ecc71;
    --danger:#e74c3c;
    --white:#ffffff;
    --light-bg:#f8f9fa;
    --card-shadow: 0 8px 24px rgba(0,0,0,0.12);
    --hover-shadow: 0 16px 40px rgba(0,0,0,0.18);
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  }
  * {
    box-sizing: border-box;
  }
  html,body{
    height:100%;
    margin:0;
    padding:0;
    background:var(--light-bg);
    color:var(--dark);
    overflow-x: hidden;
  }
  body {
    display: flex;
    flex-direction: column;
    min-height: 100vh;
  }
  .container{
    width: 100%;
    max-width: 100%;
    margin: 0;
    padding: 0;
    flex: 1;
  }
  header{
    display:flex;
    align-items:center;
    justify-content:space-between;
    padding: 16px 5%;
    background: white;
    box-shadow: 0 2px 10px rgba(0,0,0,0.08);
    position: sticky;
    top: 0;
    z-index: 100;
  }
  .brand{
    display:flex;
    gap:16px;
    align-items:center;
  }
  .logo{
    width:52px;height:52px;border-radius:8px;background:var(--primary);display:flex;align-items:center;justify-content:center;color:#fff;font-weight:bold;font-size:20px;
    box-shadow:var(--card-shadow);
  }
  nav a{
    margin-right:20px;color:var(--dark);text-decoration:none;font-weight:600;
    padding: 10px 16px;
    border-radius: 8px;
    transition: all 0.3s ease;
    font-size: 16px;
  }
  nav a:hover{
    background: #f5f5f5;
    transform: translateY(-2px);
  }
  nav a.active{color:var(--primary); background: #ffeaea; box-shadow: 0 4px 12px rgba(229, 9, 20, 0.15);}
  .hero{
    display:flex;
    gap:30px;
    align-items:flex-start;
    margin-top:0;
    padding: 30px 5%;
    background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%);
    min-height: 500px;
  }
  .card{
    background:#fff;border-radius:16px;padding:24px;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    border: 1px solid rgba(0,0,0,0.05);
  }
  .card:hover {
    box-shadow: var(--hover-shadow);
    transform: translateY(-5px);
  }
  /* LARGER MOVIE GRID STYLES */
  .movie-grid{
    display: grid;
    grid-template-columns: repeat(auto-fill, minmax(320px, 1fr));
    gap: 30px;
    padding: 30px 5%;
  }
  .movie{
    border-radius:16px;overflow:hidden;background:#fff;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    position: relative;
  }
  .movie:hover {
    transform: translateY(-8px);
    box-shadow: var(--hover-shadow);
  }
  .poster{
    width:100%;height:480px;background-size:cover;background-position:center;
    transition: transform 0.6s ease;
  }
  .movie:hover .poster {
    transform: scale(1.08);
  }
  .movie-body{padding:24px;}
  .btn{
    display:inline-block;
    padding:14px 24px;
    border-radius:10px;
    text-decoration:none;
    color:#fff;
    background:var(--primary);
    border:none;
    cursor:pointer;
    font-weight:700;
    font-size: 16px;
    transition: all 0.3s ease;
    text-align: center;
  }
  .btn:hover {
    background: #c40811;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(229, 9, 20, 0.35);
  }
  .btn.secondary{background:#6c757d;color:#fff}
  .btn.secondary:hover {
    background: #5a6268;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(0, 0, 0, 0.2);
  }
  .btn-large {
    padding: 16px 28px;
    font-size: 17px;
  }
  .muted{color:var(--muted);font-size:16px}
  .small{font-size:14px}
  footer{
    margin-top:40px;
    padding: 30px 5%;
    text-align:center;
    color:var(--muted);
    background: white;
    border-top: 1px solid #eee;
  }
  /* Seat map */
  .seat-map-container {
    padding: 20px 5%;
  }
  .seat-map{display:grid;gap:10px; margin: 0 auto; max-width: 800px;}
  .seat-row{display:flex;gap:10px;align-items:center; justify-content: center;}
  .seat{
    width:44px;height:44px;border-radius:8px;background:#e6e6e6;display:flex;align-items:center;justify-content:center;cursor:pointer;user-select:none;
    transition: all 0.3s ease;
    font-weight: 600;
    font-size: 14px;
  }
  .seat.available{background:#b6f0c8; color: #155724;} /* available green-ish */
  .seat.available:hover{background:#9ae6b2; transform: scale(1.15); box-shadow: 0 4px 8px rgba(0,0,0,0.15);}
  .seat.booked{background:#ccc; cursor:not-allowed; opacity:0.6; color: #666;}
  .seat.selected{background:#ff7070; color: white;} /* selected red-ish */
  .seat.selected:hover{background:#ff5252; transform: scale(1.15); box-shadow: 0 4px 8px rgba(255, 80, 80, 0.3);}
  .seat.plate{background:#ffd9d9}
  .legend{
    display:flex;
    gap:20px;
    align-items:center;
    margin-top:12px;
    justify-content: center;
    flex-wrap: wrap;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    max-width: 800px;
    margin: 20px auto;
  }
  .legend .box{
    width:22px;height:22px;border-radius:6px;display:inline-block;margin-right:8px;vertical-align:middle
  }
  .flex{display:flex;gap:16px;align-items:center}
  .spacer{height:16px}
  .summary{
    margin-top:16px;
    padding:20px;
    background:#fafafa;
    border-radius:12px;
    border: 1px solid #eee;
  }
  input[type="text"], input[type="email"], select {
    padding:12px 16px;
    border-radius:8px;
    border:1px solid #ddd;
    width:100%;
    transition: all 0.3s ease;
    font-size: 15px;
    background: white;
  }
  input[type="text"]:focus, input[type="email"]:focus, select:focus {
    border-color: var(--primary);
    outline: none;
    box-shadow: 0 0 0 3px rgba(229, 9, 20, 0.15);
    transform: translateY(-2px);
  }
  label{
    display:block;
    margin-bottom:8px;
    font-weight:600;
    font-size: 15px;
  }
  .row{
    display:flex;
    gap:20px;
    flex-wrap:wrap;
    padding: 20px 5%;
  }
  .page-section {
    padding: 40px 5%;
  }
  /* small details */
  .topline{font-size:15px;color:var(--muted)}
  .badge{
    background:#f1f1f1;
    padding:8px 16px;
    border-radius:10px;
    font-size:15px;
    font-weight: 600;
  }
  .center{text-align:center}
  .hidden{display:none}
  .booking-list{
    margin-top:16px;
    padding: 0 5%;
  }
  .booking-item{
    padding:20px;
    border-radius:12px;
    background:#fff;
    margin-bottom:16px;
    box-shadow:var(--card-shadow);
    transition: all 0.3s ease;
    border-left: 4px solid var(--primary);
  }
  .booking-item:hover {
    box-shadow: var(--hover-shadow);
    transform: translateX(5px);
  }
  .footer-small{font-size:14px;color:var(--muted)}
  .input-inline{display:flex;gap:12px;align-items:center}
  .screen {
    background: linear-gradient(135deg, #333 0%, #555 100%);
    color: white;
    text-align: center;
    padding: 16px;
    margin: 30px auto;
    border-radius: 8px;
    font-weight: bold;
    letter-spacing: 3px;
    font-size: 18px;
    max-width: 800px;
    box-shadow: 0 8px 16px rgba(0,0,0,0.2);
    text-transform: uppercase;
  }
  .rating {
    display: inline-flex;
    align-items: center;
    background: #f1c40f;
    color: #333;
    padding: 6px 12px;
    border-radius: 8px;
    font-weight: bold;
    font-size: 15px;
    margin-left: 12px;
  }
  .movie-meta {
    display: flex;
    align-items: center;
    gap: 12px;
    margin-top: 12px;
    flex-wrap: wrap;
  }
  .genre-tag {
    background: #f0f0f0;
    padding: 6px 12px;
    border-radius: 8px;
    font-size: 14px;
    font-weight: 600;
  }
  .price-tag {
    color: var(--primary);
    font-weight: bold;
    font-size: 18px;
  }
  .notification {
    position: fixed;
    top: 30px;
    right: 30px;
    padding: 16px 24px;
    background: var(--success);
    color: white;
    border-radius: 10px;
    box-shadow: var(--hover-shadow);
    z-index: 1000;
    transform: translateX(150%);
    transition: transform 0.4s ease;
    font-weight: 600;
    max-width: 400px;
  }
  .notification.show {
    transform: translateX(0);
  }
  .notification.error {
    background: var(--danger);
  }
  .section-title {
    font-size: 42px;
    margin-bottom: 16px;
    color: var(--dark);
    font-weight: 800;
    text-align: center;
  }
  .section-subtitle {
    font-size: 20px;
    color: var(--muted);
    margin-bottom: 40px;
    max-width: 700px;
    text-align: center;
    margin-left: auto;
    margin-right: auto;
    line-height: 1.6;
  }
  .welcome-message {
    font-size: 24px;
    margin-bottom: 10px;
    color: var(--dark);
  }
  .welcome-subtitle {
    font-size: 18px;
    color: var(--muted);
    margin-bottom: 30px;
  }
  .hero-left {
    flex: 1;
    min-width: 0;
  }
  .hero-right {
    width: 480px;
    min-width: 480px;
  }
  .quick-actions {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 16px;
    margin-top: 20px;
  }
  .stats {
    display: flex;
    gap: 20px;
    margin-top: 30px;
  }
  .stat-item {
    text-align: center;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    flex: 1;
  }
  .stat-number {
    font-size: 28px;
    font-weight: 800;
    color: var(--primary);
    display: block;
  }
  .stat-label {
    font-size: 14px;
    color: var(--muted);
    margin-top: 5px;
  }
  /* LARGER MOVIE TITLE STYLES */
  .movie-title {
    font-size: 24px;
    font-weight: 800;
    margin-bottom: 8px;
    line-height: 1.2;
  }
  .movie-description {
    font-size: 16px;
    line-height: 1.6;
    color: var(--muted);
    margin-top: 12px;
  }
  .movie-actions {
    display: flex;
    gap: 12px;
    margin-top: 20px;
  }
  .movie-info {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-top: 16px;
  }
  .movie-duration {
    font-size: 15px;
    color: var(--muted);
  }
  /* UPCOMING MOVIE STYLES */
  .upcoming-movie {
    position: relative;
    overflow: hidden;
  }
  .upcoming-overlay {
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: linear-gradient(180deg, rgba(0,0,0,0.1) 0%, rgba(0,0,0,0.7) 100%);
    display: flex;
    flex-direction: column;
    justify-content: flex-end;
    padding: 24px;
    color: white;
  }
  .coming-soon-badge {
    background: var(--primary);
    color: white;
    padding: 10px 20px;
    border-radius: 25px;
    font-weight: 700;
    font-size: 16px;
    display: inline-block;
    margin-bottom: 12px;
  }
  @media (max-width: 1200px) {
    .hero-right {
      width: 400px;
      min-width: 400px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(300px, 1fr));
    }
  }
  @media (max-width: 992px) {
    .hero {
      flex-direction: column;
    }
    .hero-right {
      width: 100%;
      min-width: auto;
    }
    .quick-actions {
      grid-template-columns: 1fr;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
      gap: 20px;
    }
    .poster {
      height: 420px;
    }
  }
  @media (max-width: 768px) {
    header {
      flex-direction: column;
      gap: 16px;
      padding: 16px 20px;
    }
    nav {
      width: 100%;
      display: flex;
      justify-content: space-between;
    }
    nav a {
      margin-right: 0;
      flex: 1;
      text-align: center;
      font-size: 14px;
      padding: 8px 12px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(260px, 1fr));
      padding: 20px;
      gap: 16px;
    }
    .row {
      padding: 20px;
    }
    .page-section {
      padding: 30px 20px;
    }
    .section-title {
      font-size: 36px;
    }
    .section-subtitle {
      font-size: 18px;
    }
    .poster {
      height: 380px;
    }
  }
  @media (max-width: 480px) {
    .movie-grid {
      grid-template-columns: 1fr;
    }
    .poster {
      height: 500px;
    }
  }
</style>
</head>
<body>
<div class="container">
  <header>
    <div class="brand">
      <div class="logo">BMS</div>
      <div>
        <div style="font-weight:800;font-size:22px">BookMyShow Prototype</div>
        <div class="topline">Welcome, <strong>Avaneesh Thakur</strong></div>
      </div>
    </div>
    <nav>
      <a href="#" id="nav-home" class="active">Home</a>
      <a href="#" id="nav-showing">Currently Showing</a>
      <a href="#" id="nav-upcoming">Upcoming</a>
      <a href="#" id="nav-bookings">My Bookings</a>
      <a href="#" id="nav-logout">Logout</a>
    </nav>
  </header>

  <main style="flex: 1;">
    <!-- HOME -->
    <section id="page-home">
      <div class="hero">
        <div class="hero-left">
          <div class="card">
            <div class="welcome-message">Hello, Avaneesh Thakur 👋</div>
            <div class="welcome-subtitle">Welcome to your BookMyShow prototype. Choose a movie to book tickets. Prices shown in ₹ (INR)</div>
            
            <div class="stats">
              <div class="stat-item">
                <span class="stat-number">12</span>
                <span class="stat-label">Movies Available</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">5</span>
                <span class="stat-label">Cities</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">24/7</span>
                <span class="stat-label">Support</span>
              </div>
            </div>
            
            <div class="spacer"></div>
            <div class="quick-actions">
              <a class="btn btn-large" id="btn-see-showing">See Currently Showing</a>
              <a class="btn btn-large secondary" id="btn-see-upcoming">See Upcoming</a>
            </div>
          </div>
          <div style="height:20px"></div>
          <div class="card">
            <h3 style="margin-top:0; font-size: 24px;">Quick Booking</h3>
            <p class="muted" style="font-size: 16px;">Choose a movie and go directly to booking</p>
            <div style="display:flex;gap:12px;align-items:center;margin-top:16px;">
              <select id="quick-movie" style="flex: 1;">
                <option value="jawan" selected>Jawan (Currently Showing)</option>
                <option value="animal">Animal (Currently Showing)</option>
                <option value="salaar">Salaar (Currently Showing)</option>
                <option value="kalki">Kalki 2898 AD (Upcoming)</option>
                <option value="pushpa">Pushpa 2 (Upcoming)</option>
              </select>
              <a class="btn btn-large" id="quick-book">Book Now</a>
            </div>
          </div>
        </div>
        <div class="hero-right">
          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Currently Showing</h4>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.ibb.co/q-F9-Nz-K5s/jawan.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Jawan</div>
                <div class="muted">Action • 2h 30m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="jawan" id="home-book-jawan">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.ibb.co/S-Xhwpsm-S/animal.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Animal</div>
                <div class="muted">Action • 3h 21m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="animal" id="home-book-animal">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center">
              <div style="width:100px;height:140px;background-image:url('https://i.ibb.co/V-Y593n-Hz/salaar.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Salaar</div>
                <div class="muted">Action • 2h 55m • Telugu/Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="salaar" id="home-book-salaar">Book Tickets</a>
                </div>
              </div>
            </div>
          </div>

          <div style="height:20px"></div>

          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Upcoming</h4>
            <div style="display:flex;gap:16px;flex-direction:column">
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.ibb.co/C-Kjfmq-Tg/kalki.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Kalki 2898 AD</div>
                  <div class="muted small">Sci-Fi • December 2025</div>
                </div>
              </div>
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.ibb.co/G-v81f15k/pushpa2.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Pushpa 2</div>
                  <div class="muted small">Action • December 2025</div>
                </div>
              </div>
            </div>
          </div>

        </div>
      </div>
    </section>

    <!-- SHOWING - LARGER LAYOUT -->
    <section id="page-showing" class="hidden">
      <div class="page-section">
        <div class="section-title">Currently Showing</div>
        <div class="section-subtitle">Book tickets for the latest blockbusters in theaters now. Experience the biggest movies on the largest screens.</div>
        
        <div class="movie-grid" id="showing-grid">
          <!-- Movie: Jawan - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/q-F9-Nz-K5s/jawan.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Jawan <span class="rating">8.2/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 30m • Hindi • UA</div>
                <div class="price-tag">₹180 - ₹350</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="jawan" onclick="goToDetails('jawan')">View Details</a>
                <a class="btn secondary" onclick="goToBook('jawan')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Animal - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/S-Xhwpsm-S/animal.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Animal <span class="rating">7.8/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Crime</span>
              </div>
              <div class="movie-description">
                A father, who is often away due to work, is unable to comprehend the intensity of his son's love and admiration.
              </div>
              <div class="movie-info">
                <div class="movie-duration">3h 21m • Hindi • A</div>
                <div class="price-tag">₹200 - ₹400</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="animal" onclick="goToDetails('animal')">View Details</a>
                <a class="btn secondary" onclick="goToBook('animal')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Salaar - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/V-Y593n-Hz/salaar.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Salaar <span class="rating">8.5/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs in an epic showdown.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 55m • Telugu/Hindi • UA</div>
                <div class="price-tag">₹190 - ₹380</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="salaar" onclick="goToDetails('salaar')">View Details</a>
                <a class="btn secondary" onclick="goToBook('salaar')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Dunki -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/Y-72-KD1jn/dunki.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Dunki <span class="rating">7.9/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Comedy</span>
                <span class="genre-tag">Adventure</span>
              </div>
              <div class="movie-description">
                A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called 'Donkey Flight'.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 41m • Hindi • UA</div>
                <div class="price-tag">₹170 - ₹320</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="dunki" onclick="goToDetails('dunki')">View Details</a>
                <a class="btn secondary" onclick="goToBook('dunki')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Fighter -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/h-tpk-P74/fighter.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Fighter <span class="rating">8.1/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Patriotic</span>
              </div>
              <div class="movie-description">
                An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 26m • Hindi • UA</div>
                <div class="price-tag">₹190 - ₹360</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="fighter" onclick="goToDetails('fighter')">View Details</a>
                <a class="btn secondary" onclick="goToBook('fighter')">Book Now</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- UPCOMING - LARGER LAYOUT -->
    <section id="page-upcoming" class="hidden">
      <div class="page-section">
        <div class="section-title">Upcoming Movies</div>
        <div class="section-subtitle">Get ready for these exciting releases coming soon to theaters. Mark your calendars for the most anticipated films of 2025.</div>
        
        <div class="movie-grid">
          <!-- Kalki 2898 AD - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/C-Kjfmq-Tg/kalki.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Kalki 2898 AD</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Sci-Fi • Epic • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience.
              </div>
            </div>
          </div>

          <!-- Pushpa 2 - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/G-v81f15k/pushpa2.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Pushpa 2: The Rule</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Drama • Crime</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Devara -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/9Qb8Y8W.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Devara</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Period • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Singham Again -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/1Qb8Y8X.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Singham Again</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Cop • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The third installment in the popular Singham franchise, featuring Rohit Shetty's cop universe with high-octane action and drama.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: War 2 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.imgur.com/3Qb8Y8Y.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">War 2</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Spy • Thriller</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Baahubali 3 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/c-Xs-DK2h-G/baahubali3.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Baahubali 3</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Epic • Fantasy • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- MOVIE DETAILS -->
    <section id="page-details" class="hidden">
      <div class="page-section">
        <div class="card" id="details-card">
          <div style="display:flex;gap:24px;align-items:flex-start">
            <div style="width:240px;height:340px;background-size:cover;background-position:center;border-radius:12px; box-shadow: var(--card-shadow);" id="details-poster"></div>
            <div style="flex:1">
              <h2 id="details-title" style="margin:0; font-size: 32px;">Movie Title</h2>
              <div class="muted" id="details-meta" style="font-size: 18px; margin: 10px 0;">Meta</div>
              <p id="details-desc" style="font-size: 16px; line-height: 1.6;">Description</p>
              <div style="margin-top:20px">
                <a class="btn btn-large" id="details-book">Book Tickets</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- BOOK TICKETS -->
    <section id="page-book" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;" id="book-title">Book Tickets</h2>
          <div class="row">
            <div style="flex:2; min-width: 500px;">
              <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
                <div>
                  <label>Show Timing</label>
                  <select id="show-timing">
                    <option>10:00 AM</option>
                    <option>01:30 PM</option>
                    <option>05:00 PM</option>
                    <option>08:30 PM</option>
                  </select>
                </div>

                <div>
                  <label>Seat Category</label>
                  <select id="seat-category">
                    <option value="platinum">Platinum (Rows A-B) — ₹350</option>
                    <option value="gold">Gold (Row C) — ₹250</option>
                    <option value="silver">Silver (Rows D-E) — ₹180</option>
                  </select>
                </div>
              </div>

              <div class="spacer"></div>

              <div style="margin-top:16px">
                <div style="font-weight:700; font-size: 20px;">Select Seats</div>
                <div class="muted" style="font-size: 16px;">Click seats to select/deselect</div>
                
                <div class="legend">
                  <div><span class="box" style="background:#b6f0c8"></span>Available</div>
                  <div><span class="box" style="background:#ff7070"></span>Selected</div>
                  <div><span class="box" style="background:#ccc"></span>Booked/Unavailable</div>
                </div>

                <div style="height:16px"></div>
                
                <div class="screen">SCREEN</div>

                <div class="seat-map card" id="seat-map" style="padding:20px"></div>

              </div>

            </div>

            <div style="flex:1; min-width: 350px;">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Booking Summary</h3>
                <div class="summary">
                  <div style="margin-bottom: 12px;">Movie: <strong id="sum-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Timing: <strong id="sum-time" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="sum-seats" style="float: right;">None</strong></div>
                  <div style="margin-bottom: 12px;">Price per seat: <strong id="sum-price" style="float: right;">₹0</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <span id="sum-total" style="float: right;">₹0</span></div>
                  <div style="height:20px"></div>
                  <label for="email-input">Email for Confirmation</label>
                  <input type="email" id="email-input" placeholder="you@example.com" />
                  <div style="height:16px"></div>
                  <div style="display:flex;gap:12px">
                    <a class="btn" id="proceed-payment" style="flex: 2;">Proceed to Payment</a>
                    <a class="btn secondary" id="clear-selection" style="flex: 1;">Clear</a>
                  </div>
                </div>
              </div>
            </div>
          </div>
        </div> 
      </div>
    </section>

    <!-- PAYMENT -->
    <section id="page-payment" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">Payment</h2>
          <div class="row">
            <div style="flex:1;min-width:500px">
              <div style="margin-bottom:20px">
                <label>Card / UPI Name</label>
                <input type="text" id="pay-name" placeholder="Avaneesh Thakur" value="Avaneesh Thakur" />
              </div>
              <div style="margin-bottom:20px">
                <label>Card / UPI Number</label>
                <input type="text" id="pay-number" placeholder="Enter 12-16 digit card or UPI id" />
              </div>
              <div style="display:grid; grid-template-columns: 1fr 1fr; gap: 16px;">
                <div>
                  <label>Expiry</label>
                  <input type="text" id="pay-expiry" placeholder="MM/YY" />
                </div>
                <div>
                  <label>CVV</label>
                  <input type="text" id="pay-cvv" placeholder="123" />
                </div>
              </div>
              <div style="height:20px"></div>
              <div class="muted">This is a simulated payment — no real transaction will occur.</div>
              <div style="height:20px"></div>
              <div style="display:flex;gap:12px">
                <a class="btn btn-large" id="confirm-pay">Pay & Confirm</a>
                <a class="btn secondary btn-large" id="back-to-book">Back</a>
              </div>
            </div>
            <div style="width:400px">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Payment Summary</h3>
                <div style="margin-top:16px">
                  <div style="margin-bottom: 12px;">Movie: <strong id="pay-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="pay-seats" style="float: right;">-</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <strong id="pay-total" style="float: right;">₹0</strong></div>
                  <div style="height:16px"></div>
                  <div class="muted">A confirmation email will be "sent" to the address you provided (simulated).</div>
                </div>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- CONFIRMATION -->
    <section id="page-confirm" class="hidden">
      <div class="page-section">
        <div class="card" style="max-width: 800px; margin: 0 auto;">
          <h2 style="font-size: 32px; text-align: center;">Booking Confirmed! 🎉</h2>
          <p style="font-size: 18px; text-align: center;">Thank you, <strong>Avaneesh Thakur</strong>. Your booking is confirmed.</p>
          <div class="card" style="background: #f8f9fa;">
            <div style="margin-bottom: 16px; font-size: 16px;">Movie: <strong id="conf-movie" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Timing: <strong id="conf-time" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Seats: <strong id="conf-seats" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Email: <strong id="conf-email" style="float: right;"></strong></div>
            <div style="height:12px; border-bottom: 1px solid #ddd; margin-bottom: 16px;"></div>
            <div style="font-weight:800;font-size:24px">Total Paid: <strong id="conf-total" style="float: right;"></strong></div>
          </div>
          <div style="height:20px"></div>
          <div style="display:flex;gap:12px; justify-content: center;">
            <a class="btn btn-large" id="conf-home">Return to Homepage</a>
            <a class="btn secondary btn-large" id="conf-bookings">My Bookings</a>
          </div>
        </div>
      </div>
    </section>

    <!-- MY BOOKINGS -->
    <section id="page-bookings" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">My Bookings</h2>
          <div id="bookings-list">
            <div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>
          </div>
        </div>
      </div>
    </section>

  </main>

  <footer>
    <div class="footer-small">Made by Avaneesh Thakur • BookMyShow Prototype • 2024</div>
  </footer>
</div>

<div id="notification" class="notification"></div>

<script>
// Update MOVIES object with all converted image URLs
const MOVIES = {
  jawan: {
    id:'jawan',
    title:'Jawan',
    poster:'https://i.ibb.co/q-F9-Nz-K5s/jawan.jpg',
    meta:'Action • 2h 30m • Hindi',
    desc:'A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society, while keeping a promise made years ago. He comes up against a monstrous outlaw with no fear, who has caused extreme suffering to many.',
  },
  animal: {
    id:'animal',
    title:'Animal',
    poster:'https://i.ibb.co/S-Xhwpsm-S/animal.jpg',
    meta:'Action • 3h 21m • Hindi',
    desc:'A father, who is often away due to work, is unable to comprehend the intensity of his son\'s love. Ironically, this fervent love and admiration for his father and family creates conflict between the father and son, which leads the son to become a threat to many.',
  },
  salaar: {
    id:'salaar',
    title:'Salaar',
    poster:'https://i.ibb.co/V-Y593n-Hz/salaar.jpg',
    meta:'Action • 2h 55m • Telugu/Hindi',
    desc:'A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs. He is forced to become the saviour of the people when the city becomes a battleground for powerful and ruthless criminal forces.',
  },
  dunki: {
    id:'dunki',
    title:'Dunki',
    poster:'https://i.ibb.co/Y-72-KD1jn/dunki.jpg',
    meta:'Drama • 2h 41m • Hindi',
    desc:'A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called Donkey Flight. The film explores themes of friendship, dreams, and the price of ambition.',
  },
  fighter: {
    id:'fighter',
    title:'Fighter',
    poster:'https://i.ibb.co/h-tpk-P74/fighter.jpg',
    meta:'Action • 2h 26m • Hindi',
    desc:'An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet. The film showcases breathtaking aerial combat sequences and the personal sacrifices of our armed forces.',
  },
  kalki: {
    id:'kalki',
    title:'Kalki 2898 AD',
    poster:'https://i.ibb.co/C-Kjfmq-Tg/kalki.jpg',
    meta:'Sci-Fi • December 2025',
    desc:'A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience. The film promises to redefine Indian cinema with its scale and visual effects.',
  },
  pushpa: {
    id:'pushpa',
    title:'Pushpa 2: The Rule',
    poster:'https://i.ibb.co/G-v81f15k/pushpa2.jpg',
    meta:'Action • December 2025',
    desc:'The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world. The film promises even more action and drama than its predecessor.',
  },
  devara: {
    id:'devara',
    title:'Devara',
    poster:'https://i.imgur.com/9Qb8Y8W.jpg',
    meta:'Action • December 2025',
    desc:'A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.',
  },
  singham: {
    id:'singham',
    title:'Singham Again',
    poster:'https://i.imgur.com/1Qb8Y8X.jpg',
    meta:'Action • December 2025',
    desc:'The third installment in the popular Singham franchise, featuring Rohit Shetty\'s cop universe with high-octane action and drama.',
  },
  war2: {
    id:'war2',
    title:'War 2',
    poster:'https://i.imgur.com/3Qb8Y8Y.jpg',
    meta:'Action • December 2025',
    desc:'The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.',
  },
  baahubali3: {
    id:'baahubali3',
    title:'Baahubali 3',
    poster:'https://i.ibb.co/c-Xs-DK2h-G/baahubali3.jpg',
    meta:'Epic • December 2025',
    desc:'The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.',
  }
};

// Rest of the JavaScript code remains exactly the same as previous version

/* Single-page navigation */
const pages = {
  home: document.getElementById('page-home'),
  showing: document.getElementById('page-showing'),
  upcoming: document.getElementById('page-upcoming'),
  details: document.getElementById('page-details'),
  book: document.getElementById('page-book'),
  payment: document.getElementById('page-payment'),
  confirm: document.getElementById('page-confirm'),
  bookings: document.getElementById('page-bookings'),
};

function showPage(name){
  for(const p of Object.values(pages)) p.classList.add('hidden');
  pages[name].classList.remove('hidden');
  // navbar active states
  document.querySelectorAll('nav a').forEach(a=>a.classList.remove('active'));
  if(name==='home') document.getElementById('nav-home').classList.add('active');
  if(name==='showing') document.getElementById('nav-showing').classList.add('active');
  if(name==='upcoming') document.getElementById('nav-upcoming').classList.add('active');
  if(name==='bookings') document.getElementById('nav-bookings').classList.add('active');
}

/* Seat map state */
const ROWS = ['A','B','C','D','E'];
const SEATS_PER_ROW = 10;
let bookedSeats = {}; // { 'jawan_10:00_A3': true } to mark booked for simulation
let selectedSeats = []; // list of seat ids e.g., 'A3'
let currentMovie = 'jawan';
let currentTiming = '10:00 AM';

const PRICES = {
  platinum:350,
  gold:250,
  silver:180
};

function showNotification(message, isError = false) {
  const notification = document.getElementById('notification');
  notification.textContent = message;
  notification.className = 'notification';
  if (isError) {
    notification.classList.add('error');
  }
  notification.classList.add('show');
  
  setTimeout(() => {
    notification.classList.remove('show');
  }, 3000);
}

function init(){
  // nav handlers
  document.getElementById('nav-home').addEventListener('click', e=>{ e.preventDefault(); showPage('home')});
  document.getElementById('nav-showing').addEventListener('click', e=>{ e.preventDefault(); showPage('showing')});
  document.getElementById('nav-upcoming').addEventListener('click', e=>{ e.preventDefault(); showPage('upcoming')});
  document.getElementById('nav-bookings').addEventListener('click', e=>{ e.preventDefault(); renderBookings(); showPage('bookings')});
  document.getElementById('nav-logout').addEventListener('click', e=>{ e.preventDefault(); logout(); });

  document.getElementById('btn-see-showing').addEventListener('click', ()=>{ showPage('showing')});
  document.getElementById('btn-see-upcoming').addEventListener('click', ()=>{ showPage('upcoming')});
  document.getElementById('home-book-jawan').addEventListener('click', ()=> goToBook('jawan'));
  document.getElementById('home-book-animal').addEventListener('click', ()=> goToBook('animal'));
  document.getElementById('home-book-salaar').addEventListener('click', ()=> goToBook('salaar'));
  document.getElementById('quick-book').addEventListener('click', ()=> {
    const m = document.getElementById('quick-movie').value;
    goToBook(m);
  });

  // details/book buttons
  document.getElementById('details-book').addEventListener('click', ()=> goToBook(currentMovie));

  // book page controls
  document.getElementById('seat-category').addEventListener('change', updateSummary);
  document.getElementById('show-timing').addEventListener('change', ()=> { 
    currentTiming = document.getElementById('show-timing').value; 
    updateSummary();
    applyBookedVisuals();
  });
  document.getElementById('clear-selection').addEventListener('click', clearSelection);
  document.getElementById('proceed-payment').addEventListener('click', proceedToPayment);

  // payment page controls
  document.getElementById('confirm-pay').addEventListener('click', confirmPayment);
  document.getElementById('back-to-book').addEventListener('click', ()=> showPage('book'));

  // confirmation page buttons
  document.getElementById('conf-home').addEventListener('click', ()=> showPage('home'));
  document.getElementById('conf-bookings').addEventListener('click', ()=> { renderBookings(); showPage('bookings'); });

  // initial render
  renderSeatMap();
  populateDetails('jawan');
  updateSummary();
  restoreBookedFromLocal();
}

function goToDetails(movieId){
  currentMovie = movieId;
  populateDetails(movieId);
  showPage('details');
}
function goToBook(movieId){
  currentMovie = movieId;
  // reset selection
  clearSelection();
  populateDetails(movieId);
  showPage('book');
}

/* Populate details card */
function populateDetails(movieId){
  const m = MOVIES[movieId];
  document.getElementById('details-poster').style.backgroundImage = `url('${m.poster}')`;
  document.getElementById('details-title').innerText = m.title;
  document.getElementById('details-meta').innerText = m.meta;
  document.getElementById('details-desc').innerText = m.desc;
  document.getElementById('book-title').innerText = `Book Tickets - ${m.title}`;
  document.getElementById('sum-movie').innerText = m.title;
  document.getElementById('pay-movie').innerText = m.title;
}

/* Seat map rendering */
function renderSeatMap(){
  const container = document.getElementById('seat-map');
  container.innerHTML = '';
  for(const row of ROWS){
    const rowEl = document.createElement('div');
    rowEl.className='seat-row';
    const label = document.createElement('div');
    label.style.width='28px';
    label.style.fontWeight='700';
    label.innerText = row;
    rowEl.appendChild(label);
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const seatEl = document.createElement('div');
      seatEl.className='seat available';
      seatEl.id = `seat-${seatId}`;
      seatEl.innerText = i;
      seatEl.dataset.seat = seatId;
      // assign category styles visually but selection price uses mapping
      if(row==='A' || row==='B') seatEl.title='Platinum';
      else if(row==='C') seatEl.title='Gold';
      else seatEl.title='Silver';

      seatEl.addEventListener('click', ()=> onSeatClick(seatId));
      rowEl.appendChild(seatEl);
    }
    container.appendChild(rowEl);
  }
  // mark already booked seats (simulate)
  applyBookedVisuals();
}

/* simulate restoring booked seats (from localStorage bookings) */
function restoreBookedFromLocal(){
  const b = JSON.parse(localStorage.getItem('bookings') || '[]');
  // mark as booked for current movie/timeslot by id
  bookedSeats = {}; // reset
  for(const item of b){
    if(!item.movieId || !item.timing || !item.seats) continue;
    // for simplicity mark seats as booked for that movie across times
    item.seats.forEach(seat=>{
      const key = `${item.movieId}_${item.timing}_${seat}`;
      bookedSeats[key] = true;
    });
  }
  applyBookedVisuals();
}

/* Check if a seat is booked for current movie/time */
function isSeatBookedGlobal(seatId){
  const key = `${currentMovie}_${currentTiming}_${seatId}`;
  return !!bookedSeats[key];
}

function applyBookedVisuals(){
  for(const row of ROWS){
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const el = document.getElementById(`seat-${seatId}`);
      if(!el) continue;
      if(isSeatBookedGlobal(seatId)){
        el.classList.remove('available','selected');
        el.classList.add('booked');
      } else {
        el.classList.remove('booked');
        el.classList.add('available');
      }
    }
  }
}

/* Seat click handler */
function onSeatClick(seatId){
  // can't select if booked
  if(isSeatBookedGlobal(seatId)) {
    showNotification('This seat is already booked!', true);
    return;
  }
  const el = document.getElementById(`seat-${seatId}`);
  if(!el) return;
  if(el.classList.contains('selected')){
    el.classList.remove('selected');
    selectedSeats = selectedSeats.filter(s=>s!==seatId);
    showNotification(`Seat ${seatId} removed from selection`);
  } else {
    el.classList.add('selected');
    selectedSeats.push(seatId);
    showNotification(`Seat ${seatId} added to selection`);
  }
  updateSummary();
}

/* Clear selection */
function clearSelection(){
  selectedSeats = [];
  document.querySelectorAll('.seat.selected').forEach(s => s.classList.remove('selected'));
  updateSummary();
  showNotification('Selection cleared');
}

/* Update summary panel */
function updateSummary(){
  // movie and time
  document.getElementById('sum-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('sum-time').innerText = document.getElementById('show-timing').value;
  currentTiming = document.getElementById('show-timing').value;
  // price per seat depends on selected category
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  document.getElementById('sum-price').innerText = `₹${price}`;
  // seats
  document.getElementById('sum-seats').innerText = selectedSeats.length ? selectedSeats.join(', ') : 'None';
  // total
  const total = selectedSeats.length * price;
  document.getElementById('sum-total').innerText = `₹${total}`;
  // payment summary fields
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ') || '-';
  document.getElementById('pay-total').innerText = `₹${total}`;
}

/* Proceed to payment validation (email required) */
function proceedToPayment(){
  const email = document.getElementById('email-input').value.trim();
  if(selectedSeats.length===0){
    showNotification('Please select at least one seat.', true);
    return;
  }
  if(!validateEmail(email)){
    showNotification('Please enter a valid email address.', true);
    return;
  }
  // populate payment page summary
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ');
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  const total = selectedSeats.length * price;
  document.getElementById('pay-total').innerText = `₹${total}`;
  // store interim booking info in session (page-level)
  window.currentBooking = {
    movieId: currentMovie,
    movieTitle: MOVIES[currentMovie].title,
    timing: currentTiming,
    seats: selectedSeats.slice(),
    category: cat,
    pricePerSeat: price,
    total: total,
    email: email,
    user: 'Avaneesh Thakur'
  };
  showPage('payment');
}

/* Payment confirmation (simulated) */
function confirmPayment(){
  const name = document.getElementById('pay-name').value.trim();
  const number = document.getElementById('pay-number').value.trim();
  const expiry = document.getElementById('pay-expiry').value.trim();
  const cvv = document.getElementById('pay-cvv').value.trim();
  // simple validations
  if(!name){ showNotification('Please enter name for payment.', true); return; }
  if(!number || number.length < 6){ showNotification('Please enter a valid card/UPI number (simulated).', true); return; }
  if(!expiry || !/^\d{2}\/\d{2}$/.test(expiry)){ showNotification('Expiry must be in MM/YY format.', true); return; }
  if(!cvv || cvv.length<3){ showNotification('Enter CVV (3 digits).', true); return; }

  const b = window.currentBooking || {};
  b.payMethod = 'Simulated Card/UPI';
  b.paymentInfo = {name, numberMasked: maskNumber(number), date: new Date().toISOString()};
  b.confirmationId = 'BMS' + Math.floor(Math.random()*900000 + 100000);

  // save to localStorage bookings array
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  bookings.push(b);
  localStorage.setItem('bookings', JSON.stringify(bookings));

  // mark seats as booked
  for(const s of b.seats){
    const key = `${b.movieId}_${b.timing}_${s}`;
    bookedSeats[key] = true;
  }
  applyBookedVisuals();

  // populate confirmation page
  document.getElementById('conf-movie').innerText = b.movieTitle;
  document.getElementById('conf-time').innerText = b.timing;
  document.getElementById('conf-seats').innerText = b.seats.join(', ');
  document.getElementById('conf-email').innerText = b.email;
  document.getElementById('conf-total').innerText = `₹${b.total}`;

  // clear selected seats for the UI
  selectedSeats = [];
  updateSummary();

  // move to confirmation
  showPage('confirm');
}

/* Booking list rendering */
function renderBookings(){
  const container = document.getElementById('bookings-list');
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  container.innerHTML = '';
  if(!bookings.length){
    container.innerHTML = '<div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>';
    return;
  }
  for(const bk of bookings.slice().reverse()){
    const el = document.createElement('div');
    el.className = 'booking-item';
    el.innerHTML = `
      <div style="display:flex;justify-content:space-between;align-items:center">
        <div>
          <div style="font-weight:800; font-size: 18px;">${escapeHtml(bk.movieTitle)}</div>
          <div class="muted" style="margin-top: 8px;">${escapeHtml(bk.timing)} • Seats: ${escapeHtml(bk.seats.join(', '))}</div>
          <div class="muted" style="margin-top: 4px;">Email: ${escapeHtml(bk.email)}</div>
        </div>
        <div style="text-align:right">
          <div style="font-weight:800; font-size: 20px;">₹${bk.total}</div>
          <div class="muted" style="margin-top: 4px;">Conf: ${escapeHtml(bk.confirmationId || '')}</div>
        </div>
      </div>
    `;
    container.appendChild(el);
  }
}

/* Helpers */
function maskNumber(num){
  if(num.length<=4) return '****';
  const last = num.slice(-4);
  return '**** **** **** ' + last;
}
function validateEmail(email){
  if(!email) return false;
  // simple regex
  const re = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return re.test(email);
}
function escapeHtml(text){
  if(!text) return '';
  return text.replace(/&/g,'&amp;').replace(/</g,'&lt;').replace(/>/g,'&gt;');
}

/* Logout function clears temporary bookings and selections (and local storage if desired) */
function logout(){
  if(confirm('Logout will clear session-selected data (bookings persist). Proceed?')){
    selectedSeats = [];
    window.currentBooking = null;
    // Optionally clear localStorage bookings: commented out to preserve bookings
    // localStorage.removeItem('bookings');
    // just go to home
    showPage('home');
    showNotification('Logged out successfully');
  }
}

/* Utility to set initial booking page movie/time on load if needed */
document.addEventListener('DOMContentLoaded', init);

/* ensure quick UI functions for links that call from html attributes */
function goToBook(movieId){ currentMovie = movieId; populateDetails(movieId); clearSelection(); renderSeatMap(); showPage('book'); }
function goToDetails(movieId){ currentMovie = movieId; populateDetails(movieId); showPage('details'); }

// ensure show-page when clicking elements by id
document.getElementById('btn-see-showing').addEventListener('click', ()=> showPage('showing'));
document.getElementById('btn-see-upcoming').addEventListener('click', ()=> showPage('upcoming'));

// Keep seat visuals consistent when timing or movie changes
document.getElementById('show-timing').addEventListener('change', ()=> { currentTiming = document.getElementById('show-timing').value; applyBookedVisuals(); });

// In case someone navigates directly to payment without currentBooking, prevent errors:
window.addEventListener('beforeunload', ()=>{}); // placeholder

</script>
</body>
</html>
"""

# write html file
p = Path(HTML_FILENAME)
p.write_text(html_code, encoding='utf-8')
print(f"Written {HTML_FILENAME} ({p.stat().st_size} bytes)")

# Serve the file using a simple HTTP server in a background thread
class QuietHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, format, *args):
        # suppress console spam from the server; comment out if you want logs
        return

def start_server(port=PORT):
    Handler = QuietHTTPRequestHandler
    os.chdir(os.path.dirname(os.path.abspath(HTML_FILENAME)) or ".")
    with socketserver.TCPServer(("", port), Handler) as httpd:
        print(f"Serving HTTP on 0.0.0.0 port {port} (http://localhost:{port}/{HTML_FILENAME}) ...")
        httpd.serve_forever()

# start server in background thread to keep notebook interactive
server_thread = threading.Thread(target=start_server, args=(PORT,), daemon=True)
server_thread.start()

# open the page in default browser (give the server a moment to start)
time.sleep(0.6)
url = f"http://localhost:{PORT}/{HTML_FILENAME}"
print(f"Opening {url} in your web browser...")
webbrowser.open(url)

Exception in thread Thread-12 (start_server):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/opt/anaconda3/lib/python3.13/threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/gr/87c8xy3d21q8n0tlcwllwc800000gn/T/ipykernel_13874/3637544254.py", line 1576, in start_server
    with socketserver.TCPServer(("", port), Handler) as httpd:
         ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 457, in __init__
    self.server_bind()
    ~~~~~~~~~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 478, in server_bind
    self.socket.bind(self.server_address)

Written bookmyshow.html (58674 bytes)
Opening http://localhost:8000/bookmyshow.html in your web browser...


True

In [8]:
# bookmyshow_prototype.py
# Run this cell in Jupyter Notebook. It will write bookmyshow.html, start a local HTTP server
# and open the page in your browser.
# Uses only standard libraries.

import os
import http.server
import socketserver
import threading
import webbrowser
import time
from pathlib import Path

HTML_FILENAME = "bookmyshow.html"
PORT = 8000

html_code = r"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>BookMyShow Prototype</title>
<style>
  :root{
    --primary:#E50914; /* BookMyShow-like red */
    --dark:#222;
    --muted:#666;
    --success:#2ecc71;
    --danger:#e74c3c;
    --white:#ffffff;
    --light-bg:#f8f9fa;
    --card-shadow: 0 8px 24px rgba(0,0,0,0.12);
    --hover-shadow: 0 16px 40px rgba(0,0,0,0.18);
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  }
  * {
    box-sizing: border-box;
  }
  html,body{
    height:100%;
    margin:0;
    padding:0;
    background:var(--light-bg);
    color:var(--dark);
    overflow-x: hidden;
  }
  body {
    display: flex;
    flex-direction: column;
    min-height: 100vh;
  }
  .container{
    width: 100%;
    max-width: 100%;
    margin: 0;
    padding: 0;
    flex: 1;
  }
  header{
    display:flex;
    align-items:center;
    justify-content:space-between;
    padding: 16px 5%;
    background: white;
    box-shadow: 0 2px 10px rgba(0,0,0,0.08);
    position: sticky;
    top: 0;
    z-index: 100;
  }
  .brand{
    display:flex;
    gap:16px;
    align-items:center;
  }
  .logo{
    width:52px;height:52px;border-radius:8px;background:var(--primary);display:flex;align-items:center;justify-content:center;color:#fff;font-weight:bold;font-size:20px;
    box-shadow:var(--card-shadow);
  }
  nav a{
    margin-right:20px;color:var(--dark);text-decoration:none;font-weight:600;
    padding: 10px 16px;
    border-radius: 8px;
    transition: all 0.3s ease;
    font-size: 16px;
  }
  nav a:hover{
    background: #f5f5f5;
    transform: translateY(-2px);
  }
  nav a.active{color:var(--primary); background: #ffeaea; box-shadow: 0 4px 12px rgba(229, 9, 20, 0.15);}
  .hero{
    display:flex;
    gap:30px;
    align-items:flex-start;
    margin-top:0;
    padding: 30px 5%;
    background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%);
    min-height: 500px;
  }
  .card{
    background:#fff;border-radius:16px;padding:24px;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    border: 1px solid rgba(0,0,0,0.05);
  }
  .card:hover {
    box-shadow: var(--hover-shadow);
    transform: translateY(-5px);
  }
  /* LARGER MOVIE GRID STYLES */
  .movie-grid{
    display: grid;
    grid-template-columns: repeat(auto-fill, minmax(320px, 1fr));
    gap: 30px;
    padding: 30px 5%;
  }
  .movie{
    border-radius:16px;overflow:hidden;background:#fff;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    position: relative;
  }
  .movie:hover {
    transform: translateY(-8px);
    box-shadow: var(--hover-shadow);
  }
  .poster{
    width:100%;height:480px;background-size:cover;background-position:center;
    transition: transform 0.6s ease;
  }
  .movie:hover .poster {
    transform: scale(1.08);
  }
  .movie-body{padding:24px;}
  .btn{
    display:inline-block;
    padding:14px 24px;
    border-radius:10px;
    text-decoration:none;
    color:#fff;
    background:var(--primary);
    border:none;
    cursor:pointer;
    font-weight:700;
    font-size: 16px;
    transition: all 0.3s ease;
    text-align: center;
  }
  .btn:hover {
    background: #c40811;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(229, 9, 20, 0.35);
  }
  .btn.secondary{background:#6c757d;color:#fff}
  .btn.secondary:hover {
    background: #5a6268;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(0, 0, 0, 0.2);
  }
  .btn-large {
    padding: 16px 28px;
    font-size: 17px;
  }
  .muted{color:var(--muted);font-size:16px}
  .small{font-size:14px}
  footer{
    margin-top:40px;
    padding: 30px 5%;
    text-align:center;
    color:var(--muted);
    background: white;
    border-top: 1px solid #eee;
  }
  /* Seat map */
  .seat-map-container {
    padding: 20px 5%;
  }
  .seat-map{display:grid;gap:10px; margin: 0 auto; max-width: 800px;}
  .seat-row{display:flex;gap:10px;align-items:center; justify-content: center;}
  .seat{
    width:44px;height:44px;border-radius:8px;background:#e6e6e6;display:flex;align-items:center;justify-content:center;cursor:pointer;user-select:none;
    transition: all 0.3s ease;
    font-weight: 600;
    font-size: 14px;
  }
  .seat.available{background:#b6f0c8; color: #155724;} /* available green-ish */
  .seat.available:hover{background:#9ae6b2; transform: scale(1.15); box-shadow: 0 4px 8px rgba(0,0,0,0.15);}
  .seat.booked{background:#ccc; cursor:not-allowed; opacity:0.6; color: #666;}
  .seat.selected{background:#ff7070; color: white;} /* selected red-ish */
  .seat.selected:hover{background:#ff5252; transform: scale(1.15); box-shadow: 0 4px 8px rgba(255, 80, 80, 0.3);}
  .seat.plate{background:#ffd9d9}
  .legend{
    display:flex;
    gap:20px;
    align-items:center;
    margin-top:12px;
    justify-content: center;
    flex-wrap: wrap;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    max-width: 800px;
    margin: 20px auto;
  }
  .legend .box{
    width:22px;height:22px;border-radius:6px;display:inline-block;margin-right:8px;vertical-align:middle
  }
  .flex{display:flex;gap:16px;align-items:center}
  .spacer{height:16px}
  .summary{
    margin-top:16px;
    padding:20px;
    background:#fafafa;
    border-radius:12px;
    border: 1px solid #eee;
  }
  input[type="text"], input[type="email"], select {
    padding:12px 16px;
    border-radius:8px;
    border:1px solid #ddd;
    width:100%;
    transition: all 0.3s ease;
    font-size: 15px;
    background: white;
  }
  input[type="text"]:focus, input[type="email"]:focus, select:focus {
    border-color: var(--primary);
    outline: none;
    box-shadow: 0 0 0 3px rgba(229, 9, 20, 0.15);
    transform: translateY(-2px);
  }
  label{
    display:block;
    margin-bottom:8px;
    font-weight:600;
    font-size: 15px;
  }
  .row{
    display:flex;
    gap:20px;
    flex-wrap:wrap;
    padding: 20px 5%;
  }
  .page-section {
    padding: 40px 5%;
  }
  /* small details */
  .topline{font-size:15px;color:var(--muted)}
  .badge{
    background:#f1f1f1;
    padding:8px 16px;
    border-radius:10px;
    font-size:15px;
    font-weight: 600;
  }
  .center{text-align:center}
  .hidden{display:none}
  .booking-list{
    margin-top:16px;
    padding: 0 5%;
  }
  .booking-item{
    padding:20px;
    border-radius:12px;
    background:#fff;
    margin-bottom:16px;
    box-shadow:var(--card-shadow);
    transition: all 0.3s ease;
    border-left: 4px solid var(--primary);
  }
  .booking-item:hover {
    box-shadow: var(--hover-shadow);
    transform: translateX(5px);
  }
  .footer-small{font-size:14px;color:var(--muted)}
  .input-inline{display:flex;gap:12px;align-items:center}
  .screen {
    background: linear-gradient(135deg, #333 0%, #555 100%);
    color: white;
    text-align: center;
    padding: 16px;
    margin: 30px auto;
    border-radius: 8px;
    font-weight: bold;
    letter-spacing: 3px;
    font-size: 18px;
    max-width: 800px;
    box-shadow: 0 8px 16px rgba(0,0,0,0.2);
    text-transform: uppercase;
  }
  .rating {
    display: inline-flex;
    align-items: center;
    background: #f1c40f;
    color: #333;
    padding: 6px 12px;
    border-radius: 8px;
    font-weight: bold;
    font-size: 15px;
    margin-left: 12px;
  }
  .movie-meta {
    display: flex;
    align-items: center;
    gap: 12px;
    margin-top: 12px;
    flex-wrap: wrap;
  }
  .genre-tag {
    background: #f0f0f0;
    padding: 6px 12px;
    border-radius: 8px;
    font-size: 14px;
    font-weight: 600;
  }
  .price-tag {
    color: var(--primary);
    font-weight: bold;
    font-size: 18px;
  }
  .notification {
    position: fixed;
    top: 30px;
    right: 30px;
    padding: 16px 24px;
    background: var(--success);
    color: white;
    border-radius: 10px;
    box-shadow: var(--hover-shadow);
    z-index: 1000;
    transform: translateX(150%);
    transition: transform 0.4s ease;
    font-weight: 600;
    max-width: 400px;
  }
  .notification.show {
    transform: translateX(0);
  }
  .notification.error {
    background: var(--danger);
  }
  .section-title {
    font-size: 42px;
    margin-bottom: 16px;
    color: var(--dark);
    font-weight: 800;
    text-align: center;
  }
  .section-subtitle {
    font-size: 20px;
    color: var(--muted);
    margin-bottom: 40px;
    max-width: 700px;
    text-align: center;
    margin-left: auto;
    margin-right: auto;
    line-height: 1.6;
  }
  .welcome-message {
    font-size: 24px;
    margin-bottom: 10px;
    color: var(--dark);
  }
  .welcome-subtitle {
    font-size: 18px;
    color: var(--muted);
    margin-bottom: 30px;
  }
  .hero-left {
    flex: 1;
    min-width: 0;
  }
  .hero-right {
    width: 480px;
    min-width: 480px;
  }
  .quick-actions {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 16px;
    margin-top: 20px;
  }
  .stats {
    display: flex;
    gap: 20px;
    margin-top: 30px;
  }
  .stat-item {
    text-align: center;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    flex: 1;
  }
  .stat-number {
    font-size: 28px;
    font-weight: 800;
    color: var(--primary);
    display: block;
  }
  .stat-label {
    font-size: 14px;
    color: var(--muted);
    margin-top: 5px;
  }
  /* LARGER MOVIE TITLE STYLES */
  .movie-title {
    font-size: 24px;
    font-weight: 800;
    margin-bottom: 8px;
    line-height: 1.2;
  }
  .movie-description {
    font-size: 16px;
    line-height: 1.6;
    color: var(--muted);
    margin-top: 12px;
  }
  .movie-actions {
    display: flex;
    gap: 12px;
    margin-top: 20px;
  }
  .movie-info {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-top: 16px;
  }
  .movie-duration {
    font-size: 15px;
    color: var(--muted);
  }
  /* UPCOMING MOVIE STYLES */
  .upcoming-movie {
    position: relative;
    overflow: hidden;
  }
  .upcoming-overlay {
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: linear-gradient(180deg, rgba(0,0,0,0.1) 0%, rgba(0,0,0,0.7) 100%);
    display: flex;
    flex-direction: column;
    justify-content: flex-end;
    padding: 24px;
    color: white;
  }
  .coming-soon-badge {
    background: var(--primary);
    color: white;
    padding: 10px 20px;
    border-radius: 25px;
    font-weight: 700;
    font-size: 16px;
    display: inline-block;
    margin-bottom: 12px;
  }
  @media (max-width: 1200px) {
    .hero-right {
      width: 400px;
      min-width: 400px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(300px, 1fr));
    }
  }
  @media (max-width: 992px) {
    .hero {
      flex-direction: column;
    }
    .hero-right {
      width: 100%;
      min-width: auto;
    }
    .quick-actions {
      grid-template-columns: 1fr;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
      gap: 20px;
    }
    .poster {
      height: 420px;
    }
  }
  @media (max-width: 768px) {
    header {
      flex-direction: column;
      gap: 16px;
      padding: 16px 20px;
    }
    nav {
      width: 100%;
      display: flex;
      justify-content: space-between;
    }
    nav a {
      margin-right: 0;
      flex: 1;
      text-align: center;
      font-size: 14px;
      padding: 8px 12px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(260px, 1fr));
      padding: 20px;
      gap: 16px;
    }
    .row {
      padding: 20px;
    }
    .page-section {
      padding: 30px 20px;
    }
    .section-title {
      font-size: 36px;
    }
    .section-subtitle {
      font-size: 18px;
    }
    .poster {
      height: 380px;
    }
  }
  @media (max-width: 480px) {
    .movie-grid {
      grid-template-columns: 1fr;
    }
    .poster {
      height: 500px;
    }
  }
</style>
</head>
<body>
<div class="container">
  <header>
    <div class="brand">
      <div class="logo">BMS</div>
      <div>
        <div style="font-weight:800;font-size:22px">BookMyShow Prototype</div>
        <div class="topline">Welcome, <strong>Avaneesh Thakur</strong></div>
      </div>
    </div>
    <nav>
      <a href="#" id="nav-home" class="active">Home</a>
      <a href="#" id="nav-showing">Currently Showing</a>
      <a href="#" id="nav-upcoming">Upcoming</a>
      <a href="#" id="nav-bookings">My Bookings</a>
      <a href="#" id="nav-logout">Logout</a>
    </nav>
  </header>

  <main style="flex: 1;">
    <!-- HOME -->
    <section id="page-home">
      <div class="hero">
        <div class="hero-left">
          <div class="card">
            <div class="welcome-message">Hello, Avaneesh Thakur 👋</div>
            <div class="welcome-subtitle">Welcome to your BookMyShow prototype. Choose a movie to book tickets. Prices shown in ₹ (INR)</div>
            
            <div class="stats">
              <div class="stat-item">
                <span class="stat-number">12</span>
                <span class="stat-label">Movies Available</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">5</span>
                <span class="stat-label">Cities</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">24/7</span>
                <span class="stat-label">Support</span>
              </div>
            </div>
            
            <div class="spacer"></div>
            <div class="quick-actions">
              <a class="btn btn-large" id="btn-see-showing">See Currently Showing</a>
              <a class="btn btn-large secondary" id="btn-see-upcoming">See Upcoming</a>
            </div>
          </div>
          <div style="height:20px"></div>
          <div class="card">
            <h3 style="margin-top:0; font-size: 24px;">Quick Booking</h3>
            <p class="muted" style="font-size: 16px;">Choose a movie and go directly to booking</p>
            <div style="display:flex;gap:12px;align-items:center;margin-top:16px;">
              <select id="quick-movie" style="flex: 1;">
                <option value="jawan" selected>Jawan (Currently Showing)</option>
                <option value="animal">Animal (Currently Showing)</option>
                <option value="salaar">Salaar (Currently Showing)</option>
                <option value="kalki">Kalki 2898 AD (Upcoming)</option>
                <option value="pushpa">Pushpa 2 (Upcoming)</option>
              </select>
              <a class="btn btn-large" id="quick-book">Book Now</a>
            </div>
          </div>
        </div>
        <div class="hero-right">
          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Currently Showing</h4>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.ibb.co/qF9NzK5s/jawan.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Jawan</div>
                <div class="muted">Action • 2h 30m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="jawan" id="home-book-jawan">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.ibb.co/SXhwpsmS/animal.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Animal</div>
                <div class="muted">Action • 3h 21m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="animal" id="home-book-animal">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center">
              <div style="width:100px;height:140px;background-image:url('https://i.ibb.co/VY593nHz/salaar.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Salaar</div>
                <div class="muted">Action • 2h 55m • Telugu/Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="salaar" id="home-book-salaar">Book Tickets</a>
                </div>
              </div>
            </div>
          </div>

          <div style="height:20px"></div>

          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Upcoming</h4>
            <div style="display:flex;gap:16px;flex-direction:column">
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.ibb.co/CKjfmqTg/kalki.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Kalki 2898 AD</div>
                  <div class="muted small">Sci-Fi • December 2025</div>
                </div>
              </div>
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.ibb.co/Gv81f15k/pushpa2.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Pushpa 2</div>
                  <div class="muted small">Action • December 2025</div>
                </div>
              </div>
            </div>
          </div>

        </div>
      </div>
    </section>

    <!-- SHOWING - LARGER LAYOUT -->
    <section id="page-showing" class="hidden">
      <div class="page-section">
        <div class="section-title">Currently Showing</div>
        <div class="section-subtitle">Book tickets for the latest blockbusters in theaters now. Experience the biggest movies on the largest screens.</div>
        
        <div class="movie-grid" id="showing-grid">
          <!-- Movie: Jawan - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/qF9NzK5s/jawan.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Jawan <span class="rating">8.2/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 30m • Hindi • UA</div>
                <div class="price-tag">₹180 - ₹350</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="jawan" onclick="goToDetails('jawan')">View Details</a>
                <a class="btn secondary" onclick="goToBook('jawan')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Animal - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/SXhwpsmS/animal.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Animal <span class="rating">7.8/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Crime</span>
              </div>
              <div class="movie-description">
                A father, who is often away due to work, is unable to comprehend the intensity of his son's love and admiration.
              </div>
              <div class="movie-info">
                <div class="movie-duration">3h 21m • Hindi • A</div>
                <div class="price-tag">₹200 - ₹400</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="animal" onclick="goToDetails('animal')">View Details</a>
                <a class="btn secondary" onclick="goToBook('animal')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Salaar - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/VY593nHz/salaar.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Salaar <span class="rating">8.5/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs in an epic showdown.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 55m • Telugu/Hindi • UA</div>
                <div class="price-tag">₹190 - ₹380</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="salaar" onclick="goToDetails('salaar')">View Details</a>
                <a class="btn secondary" onclick="goToBook('salaar')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Dunki -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/Y72KD1jn/dunki.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Dunki <span class="rating">7.9/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Comedy</span>
                <span class="genre-tag">Adventure</span>
              </div>
              <div class="movie-description">
                A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called 'Donkey Flight'.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 41m • Hindi • UA</div>
                <div class="price-tag">₹170 - ₹320</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="dunki" onclick="goToDetails('dunki')">View Details</a>
                <a class="btn secondary" onclick="goToBook('dunki')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Fighter -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/htpkP74/fighter.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Fighter <span class="rating">8.1/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Patriotic</span>
              </div>
              <div class="movie-description">
                An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 26m • Hindi • UA</div>
                <div class="price-tag">₹190 - ₹360</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="fighter" onclick="goToDetails('fighter')">View Details</a>
                <a class="btn secondary" onclick="goToBook('fighter')">Book Now</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- UPCOMING - LARGER LAYOUT -->
    <section id="page-upcoming" class="hidden">
      <div class="page-section">
        <div class="section-title">Upcoming Movies</div>
        <div class="section-subtitle">Get ready for these exciting releases coming soon to theaters. Mark your calendars for the most anticipated films of 2025.</div>
        
        <div class="movie-grid">
          <!-- Kalki 2898 AD - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/CKjfmqTg/kalki.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Kalki 2898 AD</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Sci-Fi • Epic • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience.
              </div>
            </div>
          </div>

          <!-- Pushpa 2 - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/Gv81f15k/pushpa2.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Pushpa 2: The Rule</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Drama • Crime</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Devara -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/Y7Jz6L6v/devara.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Devara</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Period • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Singham Again -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/v4gcmmpk/singham-again.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Singham Again</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Cop • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The third installment in the popular Singham franchise, featuring Rohit Shetty's cop universe with high-octane action and drama.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: War 2 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/PZrwXZnG/war2.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">War 2</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Spy • Thriller</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Baahubali 3 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/cXsDK2hG/baahubali3.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Baahubali 3</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Epic • Fantasy • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- MOVIE DETAILS -->
    <section id="page-details" class="hidden">
      <div class="page-section">
        <div class="card" id="details-card">
          <div style="display:flex;gap:24px;align-items:flex-start">
            <div style="width:240px;height:340px;background-size:cover;background-position:center;border-radius:12px; box-shadow: var(--card-shadow);" id="details-poster"></div>
            <div style="flex:1">
              <h2 id="details-title" style="margin:0; font-size: 32px;">Movie Title</h2>
              <div class="muted" id="details-meta" style="font-size: 18px; margin: 10px 0;">Meta</div>
              <p id="details-desc" style="font-size: 16px; line-height: 1.6;">Description</p>
              <div style="margin-top:20px">
                <a class="btn btn-large" id="details-book">Book Tickets</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- BOOK TICKETS -->
    <section id="page-book" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;" id="book-title">Book Tickets</h2>
          <div class="row">
            <div style="flex:2; min-width: 500px;">
              <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
                <div>
                  <label>Show Timing</label>
                  <select id="show-timing">
                    <option>10:00 AM</option>
                    <option>01:30 PM</option>
                    <option>05:00 PM</option>
                    <option>08:30 PM</option>
                  </select>
                </div>

                <div>
                  <label>Seat Category</label>
                  <select id="seat-category">
                    <option value="platinum">Platinum (Rows A-B) — ₹350</option>
                    <option value="gold">Gold (Row C) — ₹250</option>
                    <option value="silver">Silver (Rows D-E) — ₹180</option>
                  </select>
                </div>
              </div>

              <div class="spacer"></div>

              <div style="margin-top:16px">
                <div style="font-weight:700; font-size: 20px;">Select Seats</div>
                <div class="muted" style="font-size: 16px;">Click seats to select/deselect</div>
                
                <div class="legend">
                  <div><span class="box" style="background:#b6f0c8"></span>Available</div>
                  <div><span class="box" style="background:#ff7070"></span>Selected</div>
                  <div><span class="box" style="background:#ccc"></span>Booked/Unavailable</div>
                </div>

                <div style="height:16px"></div>
                
                <div class="screen">SCREEN</div>

                <div class="seat-map card" id="seat-map" style="padding:20px"></div>

              </div>

            </div>

            <div style="flex:1; min-width: 350px;">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Booking Summary</h3>
                <div class="summary">
                  <div style="margin-bottom: 12px;">Movie: <strong id="sum-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Timing: <strong id="sum-time" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="sum-seats" style="float: right;">None</strong></div>
                  <div style="margin-bottom: 12px;">Price per seat: <strong id="sum-price" style="float: right;">₹0</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <span id="sum-total" style="float: right;">₹0</span></div>
                  <div style="height:20px"></div>
                  <label for="email-input">Email for Confirmation</label>
                  <input type="email" id="email-input" placeholder="you@example.com" />
                  <div style="height:16px"></div>
                  <div style="display:flex;gap:12px">
                    <a class="btn" id="proceed-payment" style="flex: 2;">Proceed to Payment</a>
                    <a class="btn secondary" id="clear-selection" style="flex: 1;">Clear</a>
                  </div>
                </div>
              </div>
            </div>
          </div>
        </div> 
      </div>
    </section>

    <!-- PAYMENT -->
    <section id="page-payment" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">Payment</h2>
          <div class="row">
            <div style="flex:1;min-width:500px">
              <div style="margin-bottom:20px">
                <label>Card / UPI Name</label>
                <input type="text" id="pay-name" placeholder="Avaneesh Thakur" value="Avaneesh Thakur" />
              </div>
              <div style="margin-bottom:20px">
                <label>Card / UPI Number</label>
                <input type="text" id="pay-number" placeholder="Enter 12-16 digit card or UPI id" />
              </div>
              <div style="display:grid; grid-template-columns: 1fr 1fr; gap: 16px;">
                <div>
                  <label>Expiry</label>
                  <input type="text" id="pay-expiry" placeholder="MM/YY" />
                </div>
                <div>
                  <label>CVV</label>
                  <input type="text" id="pay-cvv" placeholder="123" />
                </div>
              </div>
              <div style="height:20px"></div>
              <div class="muted">This is a simulated payment — no real transaction will occur.</div>
              <div style="height:20px"></div>
              <div style="display:flex;gap:12px">
                <a class="btn btn-large" id="confirm-pay">Pay & Confirm</a>
                <a class="btn secondary btn-large" id="back-to-book">Back</a>
              </div>
            </div>
            <div style="width:400px">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Payment Summary</h3>
                <div style="margin-top:16px">
                  <div style="margin-bottom: 12px;">Movie: <strong id="pay-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="pay-seats" style="float: right;">-</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <strong id="pay-total" style="float: right;">₹0</strong></div>
                  <div style="height:16px"></div>
                  <div class="muted">A confirmation email will be "sent" to the address you provided (simulated).</div>
                </div>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- CONFIRMATION -->
    <section id="page-confirm" class="hidden">
      <div class="page-section">
        <div class="card" style="max-width: 800px; margin: 0 auto;">
          <h2 style="font-size: 32px; text-align: center;">Booking Confirmed! 🎉</h2>
          <p style="font-size: 18px; text-align: center;">Thank you, <strong>Avaneesh Thakur</strong>. Your booking is confirmed.</p>
          <div class="card" style="background: #f8f9fa;">
            <div style="margin-bottom: 16px; font-size: 16px;">Movie: <strong id="conf-movie" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Timing: <strong id="conf-time" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Seats: <strong id="conf-seats" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Email: <strong id="conf-email" style="float: right;"></strong></div>
            <div style="height:12px; border-bottom: 1px solid #ddd; margin-bottom: 16px;"></div>
            <div style="font-weight:800;font-size:24px">Total Paid: <strong id="conf-total" style="float: right;"></strong></div>
          </div>
          <div style="height:20px"></div>
          <div style="display:flex;gap:12px; justify-content: center;">
            <a class="btn btn-large" id="conf-home">Return to Homepage</a>
            <a class="btn secondary btn-large" id="conf-bookings">My Bookings</a>
          </div>
        </div>
      </div>
    </section>

    <!-- MY BOOKINGS -->
    <section id="page-bookings" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">My Bookings</h2>
          <div id="bookings-list">
            <div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>
          </div>
        </div>
      </div>
    </section>

  </main>

  <footer>
    <div class="footer-small">Made by Avaneesh Thakur • BookMyShow Prototype • 2024</div>
  </footer>
</div>

<div id="notification" class="notification"></div>

<script>
// Update MOVIES object with all converted image URLs
const MOVIES = {
  jawan: {
    id:'jawan',
    title:'Jawan',
    poster:'https://i.ibb.co/qF9NzK5s/jawan.jpg',
    meta:'Action • 2h 30m • Hindi',
    desc:'A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society, while keeping a promise made years ago. He comes up against a monstrous outlaw with no fear, who has caused extreme suffering to many.',
  },
  animal: {
    id:'animal',
    title:'Animal',
    poster:'https://i.ibb.co/SXhwpsmS/animal.jpg',
    meta:'Action • 3h 21m • Hindi',
    desc:'A father, who is often away due to work, is unable to comprehend the intensity of his son\'s love. Ironically, this fervent love and admiration for his father and family creates conflict between the father and son, which leads the son to become a threat to many.',
  },
  salaar: {
    id:'salaar',
    title:'Salaar',
    poster:'https://i.ibb.co/VY593nHz/salaar.jpg',
    meta:'Action • 2h 55m • Telugu/Hindi',
    desc:'A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs. He is forced to become the saviour of the people when the city becomes a battleground for powerful and ruthless criminal forces.',
  },
  dunki: {
    id:'dunki',
    title:'Dunki',
    poster:'https://i.ibb.co/Y72KD1jn/dunki.jpg',
    meta:'Drama • 2h 41m • Hindi',
    desc:'A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called Donkey Flight. The film explores themes of friendship, dreams, and the price of ambition.',
  },
  fighter: {
    id:'fighter',
    title:'Fighter',
    poster:'https://i.ibb.co/htpkP74/fighter.jpg',
    meta:'Action • 2h 26m • Hindi',
    desc:'An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet. The film showcases breathtaking aerial combat sequences and the personal sacrifices of our armed forces.',
  },
  kalki: {
    id:'kalki',
    title:'Kalki 2898 AD',
    poster:'https://i.ibb.co/CKjfmqTg/kalki.jpg',
    meta:'Sci-Fi • December 2025',
    desc:'A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience. The film promises to redefine Indian cinema with its scale and visual effects.',
  },
  pushpa: {
    id:'pushpa',
    title:'Pushpa 2: The Rule',
    poster:'https://i.ibb.co/Gv81f15k/pushpa2.jpg',
    meta:'Action • December 2025',
    desc:'The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world. The film promises even more action and drama than its predecessor.',
  },
  devara: {
    id:'devara',
    title:'Devara',
    poster:'https://i.ibb.co/Y7Jz6L6v/devara.jpg',
    meta:'Action • December 2025',
    desc:'A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.',
  },
  singham: {
    id:'singham',
    title:'Singham Again',
    poster:'https://i.ibb.co/v4gcmmpk/singham-again.jpg',
    meta:'Action • December 2025',
    desc:'The third installment in the popular Singham franchise, featuring Rohit Shetty\'s cop universe with high-octane action and drama.',
  },
  war2: {
    id:'war2',
    title:'War 2',
    poster:'https://i.ibb.co/PZrwXZnG/war2.jpg',
    meta:'Action • December 2025',
    desc:'The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.',
  },
  baahubali3: {
    id:'baahubali3',
    title:'Baahubali 3',
    poster:'https://i.ibb.co/cXsDK2hG/baahubali3.jpg',
    meta:'Epic • December 2025',
    desc:'The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.',
  }
};

// Rest of the JavaScript code remains exactly the same as previous version

/* Single-page navigation */
const pages = {
  home: document.getElementById('page-home'),
  showing: document.getElementById('page-showing'),
  upcoming: document.getElementById('page-upcoming'),
  details: document.getElementById('page-details'),
  book: document.getElementById('page-book'),
  payment: document.getElementById('page-payment'),
  confirm: document.getElementById('page-confirm'),
  bookings: document.getElementById('page-bookings'),
};

function showPage(name){
  for(const p of Object.values(pages)) p.classList.add('hidden');
  pages[name].classList.remove('hidden');
  // navbar active states
  document.querySelectorAll('nav a').forEach(a=>a.classList.remove('active'));
  if(name==='home') document.getElementById('nav-home').classList.add('active');
  if(name==='showing') document.getElementById('nav-showing').classList.add('active');
  if(name==='upcoming') document.getElementById('nav-upcoming').classList.add('active');
  if(name==='bookings') document.getElementById('nav-bookings').classList.add('active');
}

/* Seat map state */
const ROWS = ['A','B','C','D','E'];
const SEATS_PER_ROW = 10;
let bookedSeats = {}; // { 'jawan_10:00_A3': true } to mark booked for simulation
let selectedSeats = []; // list of seat ids e.g., 'A3'
let currentMovie = 'jawan';
let currentTiming = '10:00 AM';

const PRICES = {
  platinum:350,
  gold:250,
  silver:180
};

function showNotification(message, isError = false) {
  const notification = document.getElementById('notification');
  notification.textContent = message;
  notification.className = 'notification';
  if (isError) {
    notification.classList.add('error');
  }
  notification.classList.add('show');
  
  setTimeout(() => {
    notification.classList.remove('show');
  }, 3000);
}

function init(){
  // nav handlers
  document.getElementById('nav-home').addEventListener('click', e=>{ e.preventDefault(); showPage('home')});
  document.getElementById('nav-showing').addEventListener('click', e=>{ e.preventDefault(); showPage('showing')});
  document.getElementById('nav-upcoming').addEventListener('click', e=>{ e.preventDefault(); showPage('upcoming')});
  document.getElementById('nav-bookings').addEventListener('click', e=>{ e.preventDefault(); renderBookings(); showPage('bookings')});
  document.getElementById('nav-logout').addEventListener('click', e=>{ e.preventDefault(); logout(); });

  document.getElementById('btn-see-showing').addEventListener('click', ()=>{ showPage('showing')});
  document.getElementById('btn-see-upcoming').addEventListener('click', ()=>{ showPage('upcoming')});
  document.getElementById('home-book-jawan').addEventListener('click', ()=> goToBook('jawan'));
  document.getElementById('home-book-animal').addEventListener('click', ()=> goToBook('animal'));
  document.getElementById('home-book-salaar').addEventListener('click', ()=> goToBook('salaar'));
  document.getElementById('quick-book').addEventListener('click', ()=> {
    const m = document.getElementById('quick-movie').value;
    goToBook(m);
  });

  // details/book buttons
  document.getElementById('details-book').addEventListener('click', ()=> goToBook(currentMovie));

  // book page controls
  document.getElementById('seat-category').addEventListener('change', updateSummary);
  document.getElementById('show-timing').addEventListener('change', ()=> { 
    currentTiming = document.getElementById('show-timing').value; 
    updateSummary();
    applyBookedVisuals();
  });
  document.getElementById('clear-selection').addEventListener('click', clearSelection);
  document.getElementById('proceed-payment').addEventListener('click', proceedToPayment);

  // payment page controls
  document.getElementById('confirm-pay').addEventListener('click', confirmPayment);
  document.getElementById('back-to-book').addEventListener('click', ()=> showPage('book'));

  // confirmation page buttons
  document.getElementById('conf-home').addEventListener('click', ()=> showPage('home'));
  document.getElementById('conf-bookings').addEventListener('click', ()=> { renderBookings(); showPage('bookings'); });

  // initial render
  renderSeatMap();
  populateDetails('jawan');
  updateSummary();
  restoreBookedFromLocal();
}

function goToDetails(movieId){
  currentMovie = movieId;
  populateDetails(movieId);
  showPage('details');
}
function goToBook(movieId){
  currentMovie = movieId;
  // reset selection
  clearSelection();
  populateDetails(movieId);
  showPage('book');
}

/* Populate details card */
function populateDetails(movieId){
  const m = MOVIES[movieId];
  document.getElementById('details-poster').style.backgroundImage = `url('${m.poster}')`;
  document.getElementById('details-title').innerText = m.title;
  document.getElementById('details-meta').innerText = m.meta;
  document.getElementById('details-desc').innerText = m.desc;
  document.getElementById('book-title').innerText = `Book Tickets - ${m.title}`;
  document.getElementById('sum-movie').innerText = m.title;
  document.getElementById('pay-movie').innerText = m.title;
}

/* Seat map rendering */
function renderSeatMap(){
  const container = document.getElementById('seat-map');
  container.innerHTML = '';
  for(const row of ROWS){
    const rowEl = document.createElement('div');
    rowEl.className='seat-row';
    const label = document.createElement('div');
    label.style.width='28px';
    label.style.fontWeight='700';
    label.innerText = row;
    rowEl.appendChild(label);
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const seatEl = document.createElement('div');
      seatEl.className='seat available';
      seatEl.id = `seat-${seatId}`;
      seatEl.innerText = i;
      seatEl.dataset.seat = seatId;
      // assign category styles visually but selection price uses mapping
      if(row==='A' || row==='B') seatEl.title='Platinum';
      else if(row==='C') seatEl.title='Gold';
      else seatEl.title='Silver';

      seatEl.addEventListener('click', ()=> onSeatClick(seatId));
      rowEl.appendChild(seatEl);
    }
    container.appendChild(rowEl);
  }
  // mark already booked seats (simulate)
  applyBookedVisuals();
}

/* simulate restoring booked seats (from localStorage bookings) */
function restoreBookedFromLocal(){
  const b = JSON.parse(localStorage.getItem('bookings') || '[]');
  // mark as booked for current movie/timeslot by id
  bookedSeats = {}; // reset
  for(const item of b){
    if(!item.movieId || !item.timing || !item.seats) continue;
    // for simplicity mark seats as booked for that movie across times
    item.seats.forEach(seat=>{
      const key = `${item.movieId}_${item.timing}_${seat}`;
      bookedSeats[key] = true;
    });
  }
  applyBookedVisuals();
}

/* Check if a seat is booked for current movie/time */
function isSeatBookedGlobal(seatId){
  const key = `${currentMovie}_${currentTiming}_${seatId}`;
  return !!bookedSeats[key];
}

function applyBookedVisuals(){
  for(const row of ROWS){
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const el = document.getElementById(`seat-${seatId}`);
      if(!el) continue;
      if(isSeatBookedGlobal(seatId)){
        el.classList.remove('available','selected');
        el.classList.add('booked');
      } else {
        el.classList.remove('booked');
        el.classList.add('available');
      }
    }
  }
}

/* Seat click handler */
function onSeatClick(seatId){
  // can't select if booked
  if(isSeatBookedGlobal(seatId)) {
    showNotification('This seat is already booked!', true);
    return;
  }
  const el = document.getElementById(`seat-${seatId}`);
  if(!el) return;
  if(el.classList.contains('selected')){
    el.classList.remove('selected');
    selectedSeats = selectedSeats.filter(s=>s!==seatId);
    showNotification(`Seat ${seatId} removed from selection`);
  } else {
    el.classList.add('selected');
    selectedSeats.push(seatId);
    showNotification(`Seat ${seatId} added to selection`);
  }
  updateSummary();
}

/* Clear selection */
function clearSelection(){
  selectedSeats = [];
  document.querySelectorAll('.seat.selected').forEach(s => s.classList.remove('selected'));
  updateSummary();
  showNotification('Selection cleared');
}

/* Update summary panel */
function updateSummary(){
  // movie and time
  document.getElementById('sum-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('sum-time').innerText = document.getElementById('show-timing').value;
  currentTiming = document.getElementById('show-timing').value;
  // price per seat depends on selected category
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  document.getElementById('sum-price').innerText = `₹${price}`;
  // seats
  document.getElementById('sum-seats').innerText = selectedSeats.length ? selectedSeats.join(', ') : 'None';
  // total
  const total = selectedSeats.length * price;
  document.getElementById('sum-total').innerText = `₹${total}`;
  // payment summary fields
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ') || '-';
  document.getElementById('pay-total').innerText = `₹${total}`;
}

/* Proceed to payment validation (email required) */
function proceedToPayment(){
  const email = document.getElementById('email-input').value.trim();
  if(selectedSeats.length===0){
    showNotification('Please select at least one seat.', true);
    return;
  }
  if(!validateEmail(email)){
    showNotification('Please enter a valid email address.', true);
    return;
  }
  // populate payment page summary
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ');
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  const total = selectedSeats.length * price;
  document.getElementById('pay-total').innerText = `₹${total}`;
  // store interim booking info in session (page-level)
  window.currentBooking = {
    movieId: currentMovie,
    movieTitle: MOVIES[currentMovie].title,
    timing: currentTiming,
    seats: selectedSeats.slice(),
    category: cat,
    pricePerSeat: price,
    total: total,
    email: email,
    user: 'Avaneesh Thakur'
  };
  showPage('payment');
}

/* Payment confirmation (simulated) */
function confirmPayment(){
  const name = document.getElementById('pay-name').value.trim();
  const number = document.getElementById('pay-number').value.trim();
  const expiry = document.getElementById('pay-expiry').value.trim();
  const cvv = document.getElementById('pay-cvv').value.trim();
  // simple validations
  if(!name){ showNotification('Please enter name for payment.', true); return; }
  if(!number || number.length < 6){ showNotification('Please enter a valid card/UPI number (simulated).', true); return; }
  if(!expiry || !/^\d{2}\/\d{2}$/.test(expiry)){ showNotification('Expiry must be in MM/YY format.', true); return; }
  if(!cvv || cvv.length<3){ showNotification('Enter CVV (3 digits).', true); return; }

  const b = window.currentBooking || {};
  b.payMethod = 'Simulated Card/UPI';
  b.paymentInfo = {name, numberMasked: maskNumber(number), date: new Date().toISOString()};
  b.confirmationId = 'BMS' + Math.floor(Math.random()*900000 + 100000);

  // save to localStorage bookings array
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  bookings.push(b);
  localStorage.setItem('bookings', JSON.stringify(bookings));

  // mark seats as booked
  for(const s of b.seats){
    const key = `${b.movieId}_${b.timing}_${s}`;
    bookedSeats[key] = true;
  }
  applyBookedVisuals();

  // populate confirmation page
  document.getElementById('conf-movie').innerText = b.movieTitle;
  document.getElementById('conf-time').innerText = b.timing;
  document.getElementById('conf-seats').innerText = b.seats.join(', ');
  document.getElementById('conf-email').innerText = b.email;
  document.getElementById('conf-total').innerText = `₹${b.total}`;

  // clear selected seats for the UI
  selectedSeats = [];
  updateSummary();

  // move to confirmation
  showPage('confirm');
}

/* Booking list rendering */
function renderBookings(){
  const container = document.getElementById('bookings-list');
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  container.innerHTML = '';
  if(!bookings.length){
    container.innerHTML = '<div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>';
    return;
  }
  for(const bk of bookings.slice().reverse()){
    const el = document.createElement('div');
    el.className = 'booking-item';
    el.innerHTML = `
      <div style="display:flex;justify-content:space-between;align-items:center">
        <div>
          <div style="font-weight:800; font-size: 18px;">${escapeHtml(bk.movieTitle)}</div>
          <div class="muted" style="margin-top: 8px;">${escapeHtml(bk.timing)} • Seats: ${escapeHtml(bk.seats.join(', '))}</div>
          <div class="muted" style="margin-top: 4px;">Email: ${escapeHtml(bk.email)}</div>
        </div>
        <div style="text-align:right">
          <div style="font-weight:800; font-size: 20px;">₹${bk.total}</div>
          <div class="muted" style="margin-top: 4px;">Conf: ${escapeHtml(bk.confirmationId || '')}</div>
        </div>
      </div>
    `;
    container.appendChild(el);
  }
}

/* Helpers */
function maskNumber(num){
  if(num.length<=4) return '****';
  const last = num.slice(-4);
  return '**** **** **** ' + last;
}
function validateEmail(email){
  if(!email) return false;
  // simple regex
  const re = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return re.test(email);
}
function escapeHtml(text){
  if(!text) return '';
  return text.replace(/&/g,'&amp;').replace(/</g,'&lt;').replace(/>/g,'&gt;');
}

/* Logout function clears temporary bookings and selections (and local storage if desired) */
function logout(){
  if(confirm('Logout will clear session-selected data (bookings persist). Proceed?')){
    selectedSeats = [];
    window.currentBooking = null;
    // Optionally clear localStorage bookings: commented out to preserve bookings
    // localStorage.removeItem('bookings');
    // just go to home
    showPage('home');
    showNotification('Logged out successfully');
  }
}

/* Utility to set initial booking page movie/time on load if needed */
document.addEventListener('DOMContentLoaded', init);

/* ensure quick UI functions for links that call from html attributes */
function goToBook(movieId){ currentMovie = movieId; populateDetails(movieId); clearSelection(); renderSeatMap(); showPage('book'); }
function goToDetails(movieId){ currentMovie = movieId; populateDetails(movieId); showPage('details'); }

// ensure show-page when clicking elements by id
document.getElementById('btn-see-showing').addEventListener('click', ()=> showPage('showing'));
document.getElementById('btn-see-upcoming').addEventListener('click', ()=> showPage('upcoming'));

// Keep seat visuals consistent when timing or movie changes
document.getElementById('show-timing').addEventListener('change', ()=> { currentTiming = document.getElementById('show-timing').value; applyBookedVisuals(); });

// In case someone navigates directly to payment without currentBooking, prevent errors:
window.addEventListener('beforeunload', ()=>{}); // placeholder

</script>
</body>
</html>
"""

# write html file
p = Path(HTML_FILENAME)
p.write_text(html_code, encoding='utf-8')
print(f"Written {HTML_FILENAME} ({p.stat().st_size} bytes)")

# Serve the file using a simple HTTP server in a background thread
class QuietHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, format, *args):
        # suppress console spam from the server; comment out if you want logs
        return

def start_server(port=PORT):
    Handler = QuietHTTPRequestHandler
    os.chdir(os.path.dirname(os.path.abspath(HTML_FILENAME)) or ".")
    with socketserver.TCPServer(("", port), Handler) as httpd:
        print(f"Serving HTTP on 0.0.0.0 port {port} (http://localhost:{port}/{HTML_FILENAME}) ...")
        httpd.serve_forever()

# start server in background thread to keep notebook interactive
server_thread = threading.Thread(target=start_server, args=(PORT,), daemon=True)
server_thread.start()

# open the page in default browser (give the server a moment to start)
time.sleep(0.6)
url = f"http://localhost:{PORT}/{HTML_FILENAME}"
print(f"Opening {url} in your web browser...")
webbrowser.open(url)

Exception in thread Thread-13 (start_server):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/opt/anaconda3/lib/python3.13/threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/gr/87c8xy3d21q8n0tlcwllwc800000gn/T/ipykernel_13874/3138275605.py", line 1576, in start_server
    with socketserver.TCPServer(("", port), Handler) as httpd:
         ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 457, in __init__
    self.server_bind()
    ~~~~~~~~~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 478, in server_bind
    self.socket.bind(self.server_address)

Written bookmyshow.html (58670 bytes)
Opening http://localhost:8000/bookmyshow.html in your web browser...


True

In [9]:
# bookmyshow_prototype.py
# Run this cell in Jupyter Notebook. It will write bookmyshow.html, start a local HTTP server
# and open the page in your browser.
# Uses only standard libraries.

import os
import http.server
import socketserver
import threading
import webbrowser
import time
from pathlib import Path

HTML_FILENAME = "bookmyshow.html"
PORT = 8000

html_code = r"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>BookMyShow Prototype</title>
<style>
  :root{
    --primary:#E50914; /* BookMyShow-like red */
    --dark:#222;
    --muted:#666;
    --success:#2ecc71;
    --danger:#e74c3c;
    --white:#ffffff;
    --light-bg:#f8f9fa;
    --card-shadow: 0 8px 24px rgba(0,0,0,0.12);
    --hover-shadow: 0 16px 40px rgba(0,0,0,0.18);
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  }
  * {
    box-sizing: border-box;
  }
  html,body{
    height:100%;
    margin:0;
    padding:0;
    background:var(--light-bg);
    color:var(--dark);
    overflow-x: hidden;
  }
  body {
    display: flex;
    flex-direction: column;
    min-height: 100vh;
  }
  .container{
    width: 100%;
    max-width: 100%;
    margin: 0;
    padding: 0;
    flex: 1;
  }
  header{
    display:flex;
    align-items:center;
    justify-content:space-between;
    padding: 16px 5%;
    background: white;
    box-shadow: 0 2px 10px rgba(0,0,0,0.08);
    position: sticky;
    top: 0;
    z-index: 100;
  }
  .brand{
    display:flex;
    gap:16px;
    align-items:center;
  }
  .logo{
    width:52px;height:52px;border-radius:8px;background:var(--primary);display:flex;align-items:center;justify-content:center;color:#fff;font-weight:bold;font-size:20px;
    box-shadow:var(--card-shadow);
  }
  nav a{
    margin-right:20px;color:var(--dark);text-decoration:none;font-weight:600;
    padding: 10px 16px;
    border-radius: 8px;
    transition: all 0.3s ease;
    font-size: 16px;
  }
  nav a:hover{
    background: #f5f5f5;
    transform: translateY(-2px);
  }
  nav a.active{color:var(--primary); background: #ffeaea; box-shadow: 0 4px 12px rgba(229, 9, 20, 0.15);}
  .hero{
    display:flex;
    gap:30px;
    align-items:flex-start;
    margin-top:0;
    padding: 30px 5%;
    background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%);
    min-height: 500px;
  }
  .card{
    background:#fff;border-radius:16px;padding:24px;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    border: 1px solid rgba(0,0,0,0.05);
  }
  .card:hover {
    box-shadow: var(--hover-shadow);
    transform: translateY(-5px);
  }
  /* LARGER MOVIE GRID STYLES */
  .movie-grid{
    display: grid;
    grid-template-columns: repeat(auto-fill, minmax(320px, 1fr));
    gap: 30px;
    padding: 30px 5%;
  }
  .movie{
    border-radius:16px;overflow:hidden;background:#fff;box-shadow:var(--card-shadow);
    transition: all 0.4s ease;
    position: relative;
  }
  .movie:hover {
    transform: translateY(-8px);
    box-shadow: var(--hover-shadow);
  }
  .poster{
    width:100%;height:480px;background-size:cover;background-position:center;
    transition: transform 0.6s ease;
  }
  .movie:hover .poster {
    transform: scale(1.08);
  }
  .movie-body{padding:24px;}
  .btn{
    display:inline-block;
    padding:14px 24px;
    border-radius:10px;
    text-decoration:none;
    color:#fff;
    background:var(--primary);
    border:none;
    cursor:pointer;
    font-weight:700;
    font-size: 16px;
    transition: all 0.3s ease;
    text-align: center;
  }
  .btn:hover {
    background: #c40811;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(229, 9, 20, 0.35);
  }
  .btn.secondary{background:#6c757d;color:#fff}
  .btn.secondary:hover {
    background: #5a6268;
    transform: translateY(-3px);
    box-shadow: 0 6px 16px rgba(0, 0, 0, 0.2);
  }
  .btn-large {
    padding: 16px 28px;
    font-size: 17px;
  }
  .muted{color:var(--muted);font-size:16px}
  .small{font-size:14px}
  footer{
    margin-top:40px;
    padding: 30px 5%;
    text-align:center;
    color:var(--muted);
    background: white;
    border-top: 1px solid #eee;
  }
  /* Seat map */
  .seat-map-container {
    padding: 20px 5%;
  }
  .seat-map{display:grid;gap:10px; margin: 0 auto; max-width: 800px;}
  .seat-row{display:flex;gap:10px;align-items:center; justify-content: center;}
  .seat{
    width:44px;height:44px;border-radius:8px;background:#e6e6e6;display:flex;align-items:center;justify-content:center;cursor:pointer;user-select:none;
    transition: all 0.3s ease;
    font-weight: 600;
    font-size: 14px;
  }
  .seat.available{background:#b6f0c8; color: #155724;} /* available green-ish */
  .seat.available:hover{background:#9ae6b2; transform: scale(1.15); box-shadow: 0 4px 8px rgba(0,0,0,0.15);}
  .seat.booked{background:#ccc; cursor:not-allowed; opacity:0.6; color: #666;}
  .seat.selected{background:#ff7070; color: white;} /* selected red-ish */
  .seat.selected:hover{background:#ff5252; transform: scale(1.15); box-shadow: 0 4px 8px rgba(255, 80, 80, 0.3);}
  .seat.plate{background:#ffd9d9}
  .legend{
    display:flex;
    gap:20px;
    align-items:center;
    margin-top:12px;
    justify-content: center;
    flex-wrap: wrap;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    max-width: 800px;
    margin: 20px auto;
  }
  .legend .box{
    width:22px;height:22px;border-radius:6px;display:inline-block;margin-right:8px;vertical-align:middle
  }
  .flex{display:flex;gap:16px;align-items:center}
  .spacer{height:16px}
  .summary{
    margin-top:16px;
    padding:20px;
    background:#fafafa;
    border-radius:12px;
    border: 1px solid #eee;
  }
  input[type="text"], input[type="email"], select {
    padding:12px 16px;
    border-radius:8px;
    border:1px solid #ddd;
    width:100%;
    transition: all 0.3s ease;
    font-size: 15px;
    background: white;
  }
  input[type="text"]:focus, input[type="email"]:focus, select:focus {
    border-color: var(--primary);
    outline: none;
    box-shadow: 0 0 0 3px rgba(229, 9, 20, 0.15);
    transform: translateY(-2px);
  }
  label{
    display:block;
    margin-bottom:8px;
    font-weight:600;
    font-size: 15px;
  }
  .row{
    display:flex;
    gap:20px;
    flex-wrap:wrap;
    padding: 20px 5%;
  }
  .page-section {
    padding: 40px 5%;
  }
  /* small details */
  .topline{font-size:15px;color:var(--muted)}
  .badge{
    background:#f1f1f1;
    padding:8px 16px;
    border-radius:10px;
    font-size:15px;
    font-weight: 600;
  }
  .center{text-align:center}
  .hidden{display:none}
  .booking-list{
    margin-top:16px;
    padding: 0 5%;
  }
  .booking-item{
    padding:20px;
    border-radius:12px;
    background:#fff;
    margin-bottom:16px;
    box-shadow:var(--card-shadow);
    transition: all 0.3s ease;
    border-left: 4px solid var(--primary);
  }
  .booking-item:hover {
    box-shadow: var(--hover-shadow);
    transform: translateX(5px);
  }
  .footer-small{font-size:14px;color:var(--muted)}
  .input-inline{display:flex;gap:12px;align-items:center}
  .screen {
    background: linear-gradient(135deg, #333 0%, #555 100%);
    color: white;
    text-align: center;
    padding: 16px;
    margin: 30px auto;
    border-radius: 8px;
    font-weight: bold;
    letter-spacing: 3px;
    font-size: 18px;
    max-width: 800px;
    box-shadow: 0 8px 16px rgba(0,0,0,0.2);
    text-transform: uppercase;
  }
  .rating {
    display: inline-flex;
    align-items: center;
    background: #f1c40f;
    color: #333;
    padding: 6px 12px;
    border-radius: 8px;
    font-weight: bold;
    font-size: 15px;
    margin-left: 12px;
  }
  .movie-meta {
    display: flex;
    align-items: center;
    gap: 12px;
    margin-top: 12px;
    flex-wrap: wrap;
  }
  .genre-tag {
    background: #f0f0f0;
    padding: 6px 12px;
    border-radius: 8px;
    font-size: 14px;
    font-weight: 600;
  }
  .price-tag {
    color: var(--primary);
    font-weight: bold;
    font-size: 18px;
  }
  .notification {
    position: fixed;
    top: 30px;
    right: 30px;
    padding: 16px 24px;
    background: var(--success);
    color: white;
    border-radius: 10px;
    box-shadow: var(--hover-shadow);
    z-index: 1000;
    transform: translateX(150%);
    transition: transform 0.4s ease;
    font-weight: 600;
    max-width: 400px;
  }
  .notification.show {
    transform: translateX(0);
  }
  .notification.error {
    background: var(--danger);
  }
  .section-title {
    font-size: 42px;
    margin-bottom: 16px;
    color: var(--dark);
    font-weight: 800;
    text-align: center;
  }
  .section-subtitle {
    font-size: 20px;
    color: var(--muted);
    margin-bottom: 40px;
    max-width: 700px;
    text-align: center;
    margin-left: auto;
    margin-right: auto;
    line-height: 1.6;
  }
  .welcome-message {
    font-size: 24px;
    margin-bottom: 10px;
    color: var(--dark);
  }
  .welcome-subtitle {
    font-size: 18px;
    color: var(--muted);
    margin-bottom: 30px;
  }
  .hero-left {
    flex: 1;
    min-width: 0;
  }
  .hero-right {
    width: 480px;
    min-width: 480px;
  }
  .quick-actions {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 16px;
    margin-top: 20px;
  }
  .stats {
    display: flex;
    gap: 20px;
    margin-top: 30px;
  }
  .stat-item {
    text-align: center;
    padding: 16px;
    background: white;
    border-radius: 12px;
    box-shadow: var(--card-shadow);
    flex: 1;
  }
  .stat-number {
    font-size: 28px;
    font-weight: 800;
    color: var(--primary);
    display: block;
  }
  .stat-label {
    font-size: 14px;
    color: var(--muted);
    margin-top: 5px;
  }
  /* LARGER MOVIE TITLE STYLES */
  .movie-title {
    font-size: 24px;
    font-weight: 800;
    margin-bottom: 8px;
    line-height: 1.2;
  }
  .movie-description {
    font-size: 16px;
    line-height: 1.6;
    color: var(--muted);
    margin-top: 12px;
  }
  .movie-actions {
    display: flex;
    gap: 12px;
    margin-top: 20px;
  }
  .movie-info {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-top: 16px;
  }
  .movie-duration {
    font-size: 15px;
    color: var(--muted);
  }
  /* UPCOMING MOVIE STYLES */
  .upcoming-movie {
    position: relative;
    overflow: hidden;
  }
  .upcoming-overlay {
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: linear-gradient(180deg, rgba(0,0,0,0.1) 0%, rgba(0,0,0,0.7) 100%);
    display: flex;
    flex-direction: column;
    justify-content: flex-end;
    padding: 24px;
    color: white;
  }
  .coming-soon-badge {
    background: var(--primary);
    color: white;
    padding: 10px 20px;
    border-radius: 25px;
    font-weight: 700;
    font-size: 16px;
    display: inline-block;
    margin-bottom: 12px;
  }
  @media (max-width: 1200px) {
    .hero-right {
      width: 400px;
      min-width: 400px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(300px, 1fr));
    }
  }
  @media (max-width: 992px) {
    .hero {
      flex-direction: column;
    }
    .hero-right {
      width: 100%;
      min-width: auto;
    }
    .quick-actions {
      grid-template-columns: 1fr;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
      gap: 20px;
    }
    .poster {
      height: 420px;
    }
  }
  @media (max-width: 768px) {
    header {
      flex-direction: column;
      gap: 16px;
      padding: 16px 20px;
    }
    nav {
      width: 100%;
      display: flex;
      justify-content: space-between;
    }
    nav a {
      margin-right: 0;
      flex: 1;
      text-align: center;
      font-size: 14px;
      padding: 8px 12px;
    }
    .movie-grid {
      grid-template-columns: repeat(auto-fill, minmax(260px, 1fr));
      padding: 20px;
      gap: 16px;
    }
    .row {
      padding: 20px;
    }
    .page-section {
      padding: 30px 20px;
    }
    .section-title {
      font-size: 36px;
    }
    .section-subtitle {
      font-size: 18px;
    }
    .poster {
      height: 380px;
    }
  }
  @media (max-width: 480px) {
    .movie-grid {
      grid-template-columns: 1fr;
    }
    .poster {
      height: 500px;
    }
  }
</style>
</head>
<body>
<div class="container">
  <header>
    <div class="brand">
      <div class="logo">BMS</div>
      <div>
        <div style="font-weight:800;font-size:22px">BookMyShow Prototype</div>
        <div class="topline">Welcome, <strong>Avaneesh Thakur</strong></div>
      </div>
    </div>
    <nav>
      <a href="#" id="nav-home" class="active">Home</a>
      <a href="#" id="nav-showing">Currently Showing</a>
      <a href="#" id="nav-upcoming">Upcoming</a>
      <a href="#" id="nav-bookings">My Bookings</a>
      <a href="#" id="nav-logout">Logout</a>
    </nav>
  </header>

  <main style="flex: 1;">
    <!-- HOME -->
    <section id="page-home">
      <div class="hero">
        <div class="hero-left">
          <div class="card">
            <div class="welcome-message">Hello, Avaneesh Thakur 👋</div>
            <div class="welcome-subtitle">Welcome to your BookMyShow prototype. Choose a movie to book tickets. Prices shown in ₹ (INR)</div>
            
            <div class="stats">
              <div class="stat-item">
                <span class="stat-number">12</span>
                <span class="stat-label">Movies Available</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">5</span>
                <span class="stat-label">Cities</span>
              </div>
              <div class="stat-item">
                <span class="stat-number">24/7</span>
                <span class="stat-label">Support</span>
              </div>
            </div>
            
            <div class="spacer"></div>
            <div class="quick-actions">
              <a class="btn btn-large" id="btn-see-showing">See Currently Showing</a>
              <a class="btn btn-large secondary" id="btn-see-upcoming">See Upcoming</a>
            </div>
          </div>
          <div style="height:20px"></div>
          <div class="card">
            <h3 style="margin-top:0; font-size: 24px;">Quick Booking</h3>
            <p class="muted" style="font-size: 16px;">Choose a movie and go directly to booking</p>
            <div style="display:flex;gap:12px;align-items:center;margin-top:16px;">
              <select id="quick-movie" style="flex: 1;">
                <option value="jawan" selected>Jawan (Currently Showing)</option>
                <option value="animal">Animal (Currently Showing)</option>
                <option value="salaar">Salaar (Currently Showing)</option>
                <option value="kalki">Kalki 2898 AD (Upcoming)</option>
                <option value="pushpa">Pushpa 2 (Upcoming)</option>
              </select>
              <a class="btn btn-large" id="quick-book">Book Now</a>
            </div>
          </div>
        </div>
        <div class="hero-right">
          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Currently Showing</h4>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.ibb.co/qF9NzK5s/jawan.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Jawan</div>
                <div class="muted">Action • 2h 30m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="jawan" id="home-book-jawan">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center; margin-bottom: 20px;">
              <div style="width:100px;height:140px;background-image:url('https://i.ibb.co/SXhwpsmS/animal.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Animal</div>
                <div class="muted">Action • 3h 21m • Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="animal" id="home-book-animal">Book Tickets</a>
                </div>
              </div>
            </div>
            
            <div style="display:flex;gap:16px;align-items:center">
              <div style="width:100px;height:140px;background-image:url('https://i.ibb.co/VY593nHz/salaar.jpg');background-size:cover;border-radius:8px"></div>
              <div style="flex: 1;">
                <div style="font-weight:800; font-size: 18px;">Salaar</div>
                <div class="muted">Action • 2h 55m • Telugu/Hindi</div>
                <div style="margin-top:12px">
                  <a class="btn" data-movie="salaar" id="home-book-salaar">Book Tickets</a>
                </div>
              </div>
            </div>
          </div>

          <div style="height:20px"></div>

          <div class="card">
            <h4 style="margin-top:0; font-size: 20px;">Upcoming</h4>
            <div style="display:flex;gap:16px;flex-direction:column">
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.ibb.co/CKjfmqTg/kalki.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Kalki 2898 AD</div>
                  <div class="muted small">Sci-Fi • December 2025</div>
                </div>
              </div>
              <div style="display:flex;gap:16px;align-items:center">
                <div style="width:80px;height:110px;background-image:url('https://i.ibb.co/Gv81f15k/pushpa2.jpg');background-size:cover;border-radius:8px"></div>
                <div>
                  <div style="font-weight:700; font-size: 16px;">Pushpa 2</div>
                  <div class="muted small">Action • December 2025</div>
                </div>
              </div>
            </div>
          </div>

        </div>
      </div>
    </section>

    <!-- SHOWING - LARGER LAYOUT -->
    <section id="page-showing" class="hidden">
      <div class="page-section">
        <div class="section-title">Currently Showing</div>
        <div class="section-subtitle">Book tickets for the latest blockbusters in theaters now. Experience the biggest movies on the largest screens.</div>
        
        <div class="movie-grid" id="showing-grid">
          <!-- Movie: Jawan - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/qF9NzK5s/jawan.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Jawan <span class="rating">8.2/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 30m • Hindi • UA</div>
                <div class="price-tag">₹180 - ₹350</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="jawan" onclick="goToDetails('jawan')">View Details</a>
                <a class="btn secondary" onclick="goToBook('jawan')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Animal - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/SXhwpsmS/animal.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Animal <span class="rating">7.8/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Crime</span>
              </div>
              <div class="movie-description">
                A father, who is often away due to work, is unable to comprehend the intensity of his son's love and admiration.
              </div>
              <div class="movie-info">
                <div class="movie-duration">3h 21m • Hindi • A</div>
                <div class="price-tag">₹200 - ₹400</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="animal" onclick="goToDetails('animal')">View Details</a>
                <a class="btn secondary" onclick="goToBook('animal')">Book Now</a>
              </div>
            </div>
          </div>
          
          <!-- Movie: Salaar - LARGER -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/VY593nHz/salaar.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Salaar <span class="rating">8.5/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Thriller</span>
                <span class="genre-tag">Drama</span>
              </div>
              <div class="movie-description">
                A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs in an epic showdown.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 55m • Telugu/Hindi • UA</div>
                <div class="price-tag">₹190 - ₹380</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="salaar" onclick="goToDetails('salaar')">View Details</a>
                <a class="btn secondary" onclick="goToBook('salaar')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Dunki -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/Y72KD1jn/dunki.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Dunki <span class="rating">7.9/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Comedy</span>
                <span class="genre-tag">Adventure</span>
              </div>
              <div class="movie-description">
                A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called 'Donkey Flight'.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 41m • Hindi • UA</div>
                <div class="price-tag">₹170 - ₹320</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="dunki" onclick="goToDetails('dunki')">View Details</a>
                <a class="btn secondary" onclick="goToBook('dunki')">Book Now</a>
              </div>
            </div>
          </div>

          <!-- Additional Movie: Fighter -->
          <div class="movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/htpkP74/fighter.jpg')"></div>
            <div class="movie-body">
              <div class="movie-title">Fighter <span class="rating">8.1/10</span></div>
              <div class="movie-meta">
                <span class="genre-tag">Action</span>
                <span class="genre-tag">Drama</span>
                <span class="genre-tag">Patriotic</span>
              </div>
              <div class="movie-description">
                An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet.
              </div>
              <div class="movie-info">
                <div class="movie-duration">2h 26m • Hindi • UA</div>
                <div class="price-tag">₹190 - ₹360</div>
              </div>
              <div class="movie-actions">
                <a class="btn" data-movie="fighter" onclick="goToDetails('fighter')">View Details</a>
                <a class="btn secondary" onclick="goToBook('fighter')">Book Now</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- UPCOMING - LARGER LAYOUT -->
    <section id="page-upcoming" class="hidden">
      <div class="page-section">
        <div class="section-title">Upcoming Movies</div>
        <div class="section-subtitle">Get ready for these exciting releases coming soon to theaters. Mark your calendars for the most anticipated films of 2025.</div>
        
        <div class="movie-grid">
          <!-- Kalki 2898 AD - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/CKjfmqTg/kalki.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Kalki 2898 AD</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Sci-Fi • Epic • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience.
              </div>
            </div>
          </div>

          <!-- Pushpa 2 - LARGER -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/Gv81f15k/pushpa2.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Pushpa 2: The Rule</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Drama • Crime</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Devara -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/Y7Jz6L6v/devara.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Devara</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Period • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Singham Again -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/v4gcmmpk/singham-again.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Singham Again</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Cop • Drama</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The third installment in the popular Singham franchise, featuring Rohit Shetty's cop universe with high-octane action and drama.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: War 2 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/PZrwXZnG/war2.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">War 2</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Action • Spy • Thriller</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.
              </div>
            </div>
          </div>

          <!-- Additional Upcoming: Baahubali 3 -->
          <div class="movie upcoming-movie">
            <div class="poster" style="background-image:url('https://i.ibb.co/cXsDK2hG/baahubali3.jpg')"></div>
            <div class="upcoming-overlay">
              <span class="coming-soon-badge">Coming December 2025</span>
              <div class="movie-title" style="color: white; font-size: 28px; margin-bottom: 8px;">Baahubali 3</div>
              <div style="font-size: 18px; margin-bottom: 12px; opacity: 0.9;">Epic • Fantasy • Action</div>
              <div style="font-size: 16px; opacity: 0.8; line-height: 1.5;">
                The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- MOVIE DETAILS -->
    <section id="page-details" class="hidden">
      <div class="page-section">
        <div class="card" id="details-card">
          <div style="display:flex;gap:24px;align-items:flex-start">
            <div style="width:240px;height:340px;background-size:cover;background-position:center;border-radius:12px; box-shadow: var(--card-shadow);" id="details-poster"></div>
            <div style="flex:1">
              <h2 id="details-title" style="margin:0; font-size: 32px;">Movie Title</h2>
              <div class="muted" id="details-meta" style="font-size: 18px; margin: 10px 0;">Meta</div>
              <p id="details-desc" style="font-size: 16px; line-height: 1.6;">Description</p>
              <div style="margin-top:20px">
                <a class="btn btn-large" id="details-book">Book Tickets</a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- BOOK TICKETS -->
    <section id="page-book" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;" id="book-title">Book Tickets</h2>
          <div class="row">
            <div style="flex:2; min-width: 500px;">
              <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
                <div>
                  <label>Show Timing</label>
                  <select id="show-timing">
                    <option>10:00 AM</option>
                    <option>01:30 PM</option>
                    <option>05:00 PM</option>
                    <option>08:30 PM</option>
                  </select>
                </div>

                <div>
                  <label>Seat Category</label>
                  <select id="seat-category">
                    <option value="platinum">Platinum (Rows A-B) — ₹350</option>
                    <option value="gold">Gold (Row C) — ₹250</option>
                    <option value="silver">Silver (Rows D-E) — ₹180</option>
                  </select>
                </div>
              </div>

              <div class="spacer"></div>

              <div style="margin-top:16px">
                <div style="font-weight:700; font-size: 20px;">Select Seats</div>
                <div class="muted" style="font-size: 16px;">Click seats to select/deselect</div>
                
                <div class="legend">
                  <div><span class="box" style="background:#b6f0c8"></span>Available</div>
                  <div><span class="box" style="background:#ff7070"></span>Selected</div>
                  <div><span class="box" style="background:#ccc"></span>Booked/Unavailable</div>
                </div>

                <div style="height:16px"></div>
                
                <div class="screen">SCREEN</div>

                <div class="seat-map card" id="seat-map" style="padding:20px"></div>

              </div>

            </div>

            <div style="flex:1; min-width: 350px;">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Booking Summary</h3>
                <div class="summary">
                  <div style="margin-bottom: 12px;">Movie: <strong id="sum-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Timing: <strong id="sum-time" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="sum-seats" style="float: right;">None</strong></div>
                  <div style="margin-bottom: 12px;">Price per seat: <strong id="sum-price" style="float: right;">₹0</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <span id="sum-total" style="float: right;">₹0</span></div>
                  <div style="height:20px"></div>
                  <label for="email-input">Email for Confirmation</label>
                  <input type="email" id="email-input" placeholder="you@example.com" />
                  <div style="height:16px"></div>
                  <div style="display:flex;gap:12px">
                    <a class="btn" id="proceed-payment" style="flex: 2;">Proceed to Payment</a>
                    <a class="btn secondary" id="clear-selection" style="flex: 1;">Clear</a>
                  </div>
                </div>
              </div>
            </div>
          </div>
        </div> 
      </div>
    </section>

    <!-- PAYMENT -->
    <section id="page-payment" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">Payment</h2>
          <div class="row">
            <div style="flex:1;min-width:500px">
              <div style="margin-bottom:20px">
                <label>Card / UPI Name</label>
                <input type="text" id="pay-name" placeholder="Avaneesh Thakur" value="Avaneesh Thakur" />
              </div>
              <div style="margin-bottom:20px">
                <label>Card / UPI Number</label>
                <input type="text" id="pay-number" placeholder="Enter 12-16 digit card or UPI id" />
              </div>
              <div style="display:grid; grid-template-columns: 1fr 1fr; gap: 16px;">
                <div>
                  <label>Expiry</label>
                  <input type="text" id="pay-expiry" placeholder="MM/YY" />
                </div>
                <div>
                  <label>CVV</label>
                  <input type="text" id="pay-cvv" placeholder="123" />
                </div>
              </div>
              <div style="height:20px"></div>
              <div class="muted">This is a simulated payment — no real transaction will occur.</div>
              <div style="height:20px"></div>
              <div style="display:flex;gap:12px">
                <a class="btn btn-large" id="confirm-pay">Pay & Confirm</a>
                <a class="btn secondary btn-large" id="back-to-book">Back</a>
              </div>
            </div>
            <div style="width:400px">
              <div class="card">
                <h3 style="margin:0; font-size: 22px;">Payment Summary</h3>
                <div style="margin-top:16px">
                  <div style="margin-bottom: 12px;">Movie: <strong id="pay-movie" style="float: right;">-</strong></div>
                  <div style="margin-bottom: 12px;">Seats: <strong id="pay-seats" style="float: right;">-</strong></div>
                  <div style="height:12px; border-bottom: 1px solid #eee; margin-bottom: 12px;"></div>
                  <div style="font-weight:800;font-size:24px">Total: <strong id="pay-total" style="float: right;">₹0</strong></div>
                  <div style="height:16px"></div>
                  <div class="muted">A confirmation email will be "sent" to the address you provided (simulated).</div>
                </div>
              </div>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- CONFIRMATION -->
    <section id="page-confirm" class="hidden">
      <div class="page-section">
        <div class="card" style="max-width: 800px; margin: 0 auto;">
          <h2 style="font-size: 32px; text-align: center;">Booking Confirmed! 🎉</h2>
          <p style="font-size: 18px; text-align: center;">Thank you, <strong>Avaneesh Thakur</strong>. Your booking is confirmed.</p>
          <div class="card" style="background: #f8f9fa;">
            <div style="margin-bottom: 16px; font-size: 16px;">Movie: <strong id="conf-movie" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Timing: <strong id="conf-time" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Seats: <strong id="conf-seats" style="float: right;"></strong></div>
            <div style="margin-bottom: 16px; font-size: 16px;">Email: <strong id="conf-email" style="float: right;"></strong></div>
            <div style="height:12px; border-bottom: 1px solid #ddd; margin-bottom: 16px;"></div>
            <div style="font-weight:800;font-size:24px">Total Paid: <strong id="conf-total" style="float: right;"></strong></div>
          </div>
          <div style="height:20px"></div>
          <div style="display:flex;gap:12px; justify-content: center;">
            <a class="btn btn-large" id="conf-home">Return to Homepage</a>
            <a class="btn secondary btn-large" id="conf-bookings">My Bookings</a>
          </div>
        </div>
      </div>
    </section>

    <!-- MY BOOKINGS -->
    <section id="page-bookings" class="hidden">
      <div class="page-section">
        <div class="card">
          <h2 style="margin-top:0; font-size: 28px;">My Bookings</h2>
          <div id="bookings-list">
            <div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>
          </div>
        </div>
      </div>
    </section>

  </main>

  <footer>
    <div class="footer-small">Made by Avaneesh Thakur • BookMyShow Prototype • 2024</div>
  </footer>
</div>

<div id="notification" class="notification"></div>

<script>
// Update MOVIES object with all converted image URLs
const MOVIES = {
  jawan: {
    id:'jawan',
    title:'Jawan',
    poster:'https://i.ibb.co/qF9NzK5s/jawan.jpg',
    meta:'Action • 2h 30m • Hindi',
    desc:'A high-octane action entertainer with blockbuster moments. A man is driven by a personal vendetta to rectify the wrongs in society, while keeping a promise made years ago. He comes up against a monstrous outlaw with no fear, who has caused extreme suffering to many.',
  },
  animal: {
    id:'animal',
    title:'Animal',
    poster:'https://i.ibb.co/SXhwpsmS/animal.jpg',
    meta:'Action • 3h 21m • Hindi',
    desc:'A father, who is often away due to work, is unable to comprehend the intensity of his son\'s love. Ironically, this fervent love and admiration for his father and family creates conflict between the father and son, which leads the son to become a threat to many.',
  },
  salaar: {
    id:'salaar',
    title:'Salaar',
    poster:'https://i.ibb.co/VY593nHz/salaar.jpg',
    meta:'Action • 2h 55m • Telugu/Hindi',
    desc:'A gang leader tries to keep a promise made to his dying friend and takes on the other criminal gangs. He is forced to become the saviour of the people when the city becomes a battleground for powerful and ruthless criminal forces.',
  },
  dunki: {
    id:'dunki',
    title:'Dunki',
    poster:'https://i.ibb.co/Y72KD1jn/dunki.jpg',
    meta:'Drama • 2h 41m • Hindi',
    desc:'A heartwarming story of four friends who dream of settling abroad and their journey through the illegal immigration method called Donkey Flight. The film explores themes of friendship, dreams, and the price of ambition.',
  },
  fighter: {
    id:'fighter',
    title:'Fighter',
    poster:'https://i.ibb.co/htpkP74/fighter.jpg',
    meta:'Action • 2h 26m • Hindi',
    desc:'An action-packed aerial drama that follows the journey of Air Force pilots as they face their toughest mission yet. The film showcases breathtaking aerial combat sequences and the personal sacrifices of our armed forces.',
  },
  kalki: {
    id:'kalki',
    title:'Kalki 2898 AD',
    poster:'https://i.ibb.co/CKjfmqTg/kalki.jpg',
    meta:'Sci-Fi • December 2025',
    desc:'A visionary sci-fi epic set far in the future where mythology meets science fiction in an unprecedented cinematic experience. The film promises to redefine Indian cinema with its scale and visual effects.',
  },
  pushpa: {
    id:'pushpa',
    title:'Pushpa 2: The Rule',
    poster:'https://i.ibb.co/Gv81f15k/pushpa2.jpg',
    meta:'Action • December 2025',
    desc:'The next installment of the mass-action saga continues the story of Pushpa Raj as he faces new challenges in the red sandalwood smuggling world. The film promises even more action and drama than its predecessor.',
  },
  devara: {
    id:'devara',
    title:'Devara',
    poster:'https://i.ibb.co/Y7Jz6L6v/devara.jpg',
    meta:'Action • December 2025',
    desc:'A period action drama set in coastal regions, featuring intense action sequences and powerful performances in a story of revenge and redemption.',
  },
  singham: {
    id:'singham',
    title:'Singham Again',
    poster:'https://i.ibb.co/v4gcmmpk/singham-again.jpg',
    meta:'Action • December 2025',
    desc:'The third installment in the popular Singham franchise, featuring Rohit Shetty\'s cop universe with high-octane action and drama.',
  },
  war2: {
    id:'war2',
    title:'War 2',
    poster:'https://i.ibb.co/PZrwXZnG/war2.jpg',
    meta:'Action • December 2025',
    desc:'The sequel to the blockbuster spy thriller featuring high-stakes international espionage and breathtaking action sequences across global locations.',
  },
  baahubali3: {
    id:'baahubali3',
    title:'Baahubali 3',
    poster:'https://i.ibb.co/cXsDK2hG/baahubali3.jpg',
    meta:'Epic • December 2025',
    desc:'The next chapter in the legendary Baahubali saga continues the epic story of the Mahishmati kingdom with even grander visuals and storytelling.',
  }
};

// Rest of the JavaScript code remains exactly the same as previous version

/* Single-page navigation */
const pages = {
  home: document.getElementById('page-home'),
  showing: document.getElementById('page-showing'),
  upcoming: document.getElementById('page-upcoming'),
  details: document.getElementById('page-details'),
  book: document.getElementById('page-book'),
  payment: document.getElementById('page-payment'),
  confirm: document.getElementById('page-confirm'),
  bookings: document.getElementById('page-bookings'),
};

function showPage(name){
  for(const p of Object.values(pages)) p.classList.add('hidden');
  pages[name].classList.remove('hidden');
  // navbar active states
  document.querySelectorAll('nav a').forEach(a=>a.classList.remove('active'));
  if(name==='home') document.getElementById('nav-home').classList.add('active');
  if(name==='showing') document.getElementById('nav-showing').classList.add('active');
  if(name==='upcoming') document.getElementById('nav-upcoming').classList.add('active');
  if(name==='bookings') document.getElementById('nav-bookings').classList.add('active');
}

/* Seat map state */
const ROWS = ['A','B','C','D','E'];
const SEATS_PER_ROW = 10;
let bookedSeats = {}; // { 'jawan_10:00_A3': true } to mark booked for simulation
let selectedSeats = []; // list of seat ids e.g., 'A3'
let currentMovie = 'jawan';
let currentTiming = '10:00 AM';

const PRICES = {
  platinum:350,
  gold:250,
  silver:180
};

function showNotification(message, isError = false) {
  const notification = document.getElementById('notification');
  notification.textContent = message;
  notification.className = 'notification';
  if (isError) {
    notification.classList.add('error');
  }
  notification.classList.add('show');
  
  setTimeout(() => {
    notification.classList.remove('show');
  }, 3000);
}

function init(){
  // nav handlers
  document.getElementById('nav-home').addEventListener('click', e=>{ e.preventDefault(); showPage('home')});
  document.getElementById('nav-showing').addEventListener('click', e=>{ e.preventDefault(); showPage('showing')});
  document.getElementById('nav-upcoming').addEventListener('click', e=>{ e.preventDefault(); showPage('upcoming')});
  document.getElementById('nav-bookings').addEventListener('click', e=>{ e.preventDefault(); renderBookings(); showPage('bookings')});
  document.getElementById('nav-logout').addEventListener('click', e=>{ e.preventDefault(); logout(); });

  document.getElementById('btn-see-showing').addEventListener('click', ()=>{ showPage('showing')});
  document.getElementById('btn-see-upcoming').addEventListener('click', ()=>{ showPage('upcoming')});
  document.getElementById('home-book-jawan').addEventListener('click', ()=> goToBook('jawan'));
  document.getElementById('home-book-animal').addEventListener('click', ()=> goToBook('animal'));
  document.getElementById('home-book-salaar').addEventListener('click', ()=> goToBook('salaar'));
  document.getElementById('quick-book').addEventListener('click', ()=> {
    const m = document.getElementById('quick-movie').value;
    goToBook(m);
  });

  // details/book buttons
  document.getElementById('details-book').addEventListener('click', ()=> goToBook(currentMovie));

  // book page controls
  document.getElementById('seat-category').addEventListener('change', updateSummary);
  document.getElementById('show-timing').addEventListener('change', ()=> { 
    currentTiming = document.getElementById('show-timing').value; 
    updateSummary();
    applyBookedVisuals();
  });
  document.getElementById('clear-selection').addEventListener('click', clearSelection);
  document.getElementById('proceed-payment').addEventListener('click', proceedToPayment);

  // payment page controls
  document.getElementById('confirm-pay').addEventListener('click', confirmPayment);
  document.getElementById('back-to-book').addEventListener('click', ()=> showPage('book'));

  // confirmation page buttons
  document.getElementById('conf-home').addEventListener('click', ()=> showPage('home'));
  document.getElementById('conf-bookings').addEventListener('click', ()=> { renderBookings(); showPage('bookings'); });

  // initial render
  renderSeatMap();
  populateDetails('jawan');
  updateSummary();
  restoreBookedFromLocal();
}

function goToDetails(movieId){
  currentMovie = movieId;
  populateDetails(movieId);
  showPage('details');
}
function goToBook(movieId){
  currentMovie = movieId;
  // reset selection
  clearSelection();
  populateDetails(movieId);
  showPage('book');
}

/* Populate details card */
function populateDetails(movieId){
  const m = MOVIES[movieId];
  document.getElementById('details-poster').style.backgroundImage = `url('${m.poster}')`;
  document.getElementById('details-title').innerText = m.title;
  document.getElementById('details-meta').innerText = m.meta;
  document.getElementById('details-desc').innerText = m.desc;
  document.getElementById('book-title').innerText = `Book Tickets - ${m.title}`;
  document.getElementById('sum-movie').innerText = m.title;
  document.getElementById('pay-movie').innerText = m.title;
}

/* Seat map rendering */
function renderSeatMap(){
  const container = document.getElementById('seat-map');
  container.innerHTML = '';
  for(const row of ROWS){
    const rowEl = document.createElement('div');
    rowEl.className='seat-row';
    const label = document.createElement('div');
    label.style.width='28px';
    label.style.fontWeight='700';
    label.innerText = row;
    rowEl.appendChild(label);
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const seatEl = document.createElement('div');
      seatEl.className='seat available';
      seatEl.id = `seat-${seatId}`;
      seatEl.innerText = i;
      seatEl.dataset.seat = seatId;
      // assign category styles visually but selection price uses mapping
      if(row==='A' || row==='B') seatEl.title='Platinum';
      else if(row==='C') seatEl.title='Gold';
      else seatEl.title='Silver';

      seatEl.addEventListener('click', ()=> onSeatClick(seatId));
      rowEl.appendChild(seatEl);
    }
    container.appendChild(rowEl);
  }
  // mark already booked seats (simulate)
  applyBookedVisuals();
}

/* simulate restoring booked seats (from localStorage bookings) */
function restoreBookedFromLocal(){
  const b = JSON.parse(localStorage.getItem('bookings') || '[]');
  // mark as booked for current movie/timeslot by id
  bookedSeats = {}; // reset
  for(const item of b){
    if(!item.movieId || !item.timing || !item.seats) continue;
    // for simplicity mark seats as booked for that movie across times
    item.seats.forEach(seat=>{
      const key = `${item.movieId}_${item.timing}_${seat}`;
      bookedSeats[key] = true;
    });
  }
  applyBookedVisuals();
}

/* Check if a seat is booked for current movie/time */
function isSeatBookedGlobal(seatId){
  const key = `${currentMovie}_${currentTiming}_${seatId}`;
  return !!bookedSeats[key];
}

function applyBookedVisuals(){
  for(const row of ROWS){
    for(let i=1;i<=SEATS_PER_ROW;i++){
      const seatId = `${row}${i}`;
      const el = document.getElementById(`seat-${seatId}`);
      if(!el) continue;
      if(isSeatBookedGlobal(seatId)){
        el.classList.remove('available','selected');
        el.classList.add('booked');
      } else {
        el.classList.remove('booked');
        el.classList.add('available');
      }
    }
  }
}

/* Seat click handler */
function onSeatClick(seatId){
  // can't select if booked
  if(isSeatBookedGlobal(seatId)) {
    showNotification('This seat is already booked!', true);
    return;
  }
  const el = document.getElementById(`seat-${seatId}`);
  if(!el) return;
  if(el.classList.contains('selected')){
    el.classList.remove('selected');
    selectedSeats = selectedSeats.filter(s=>s!==seatId);
    showNotification(`Seat ${seatId} removed from selection`);
  } else {
    el.classList.add('selected');
    selectedSeats.push(seatId);
    showNotification(`Seat ${seatId} added to selection`);
  }
  updateSummary();
}

/* Clear selection */
function clearSelection(){
  selectedSeats = [];
  document.querySelectorAll('.seat.selected').forEach(s => s.classList.remove('selected'));
  updateSummary();
  showNotification('Selection cleared');
}

/* Update summary panel */
function updateSummary(){
  // movie and time
  document.getElementById('sum-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('sum-time').innerText = document.getElementById('show-timing').value;
  currentTiming = document.getElementById('show-timing').value;
  // price per seat depends on selected category
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  document.getElementById('sum-price').innerText = `₹${price}`;
  // seats
  document.getElementById('sum-seats').innerText = selectedSeats.length ? selectedSeats.join(', ') : 'None';
  // total
  const total = selectedSeats.length * price;
  document.getElementById('sum-total').innerText = `₹${total}`;
  // payment summary fields
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ') || '-';
  document.getElementById('pay-total').innerText = `₹${total}`;
}

/* Proceed to payment validation (email required) */
function proceedToPayment(){
  const email = document.getElementById('email-input').value.trim();
  if(selectedSeats.length===0){
    showNotification('Please select at least one seat.', true);
    return;
  }
  if(!validateEmail(email)){
    showNotification('Please enter a valid email address.', true);
    return;
  }
  // populate payment page summary
  document.getElementById('pay-movie').innerText = MOVIES[currentMovie].title;
  document.getElementById('pay-seats').innerText = selectedSeats.join(', ');
  const cat = document.getElementById('seat-category').value;
  const price = PRICES[cat] || 0;
  const total = selectedSeats.length * price;
  document.getElementById('pay-total').innerText = `₹${total}`;
  // store interim booking info in session (page-level)
  window.currentBooking = {
    movieId: currentMovie,
    movieTitle: MOVIES[currentMovie].title,
    timing: currentTiming,
    seats: selectedSeats.slice(),
    category: cat,
    pricePerSeat: price,
    total: total,
    email: email,
    user: 'Avaneesh Thakur'
  };
  showPage('payment');
}

/* Payment confirmation (simulated) */
function confirmPayment(){
  const name = document.getElementById('pay-name').value.trim();
  const number = document.getElementById('pay-number').value.trim();
  const expiry = document.getElementById('pay-expiry').value.trim();
  const cvv = document.getElementById('pay-cvv').value.trim();
  // simple validations
  if(!name){ showNotification('Please enter name for payment.', true); return; }
  if(!number || number.length < 6){ showNotification('Please enter a valid card/UPI number (simulated).', true); return; }
  if(!expiry || !/^\d{2}\/\d{2}$/.test(expiry)){ showNotification('Expiry must be in MM/YY format.', true); return; }
  if(!cvv || cvv.length<3){ showNotification('Enter CVV (3 digits).', true); return; }

  const b = window.currentBooking || {};
  b.payMethod = 'Simulated Card/UPI';
  b.paymentInfo = {name, numberMasked: maskNumber(number), date: new Date().toISOString()};
  b.confirmationId = 'BMS' + Math.floor(Math.random()*900000 + 100000);

  // save to localStorage bookings array
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  bookings.push(b);
  localStorage.setItem('bookings', JSON.stringify(bookings));

  // mark seats as booked
  for(const s of b.seats){
    const key = `${b.movieId}_${b.timing}_${s}`;
    bookedSeats[key] = true;
  }
  applyBookedVisuals();

  // populate confirmation page
  document.getElementById('conf-movie').innerText = b.movieTitle;
  document.getElementById('conf-time').innerText = b.timing;
  document.getElementById('conf-seats').innerText = b.seats.join(', ');
  document.getElementById('conf-email').innerText = b.email;
  document.getElementById('conf-total').innerText = `₹${b.total}`;

  // clear selected seats for the UI
  selectedSeats = [];
  updateSummary();

  // move to confirmation
  showPage('confirm');
}

/* Booking list rendering */
function renderBookings(){
  const container = document.getElementById('bookings-list');
  const bookings = JSON.parse(localStorage.getItem('bookings') || '[]');
  container.innerHTML = '';
  if(!bookings.length){
    container.innerHTML = '<div class="muted" style="text-align: center; padding: 40px; font-size: 18px;">No bookings yet.</div>';
    return;
  }
  for(const bk of bookings.slice().reverse()){
    const el = document.createElement('div');
    el.className = 'booking-item';
    el.innerHTML = `
      <div style="display:flex;justify-content:space-between;align-items:center">
        <div>
          <div style="font-weight:800; font-size: 18px;">${escapeHtml(bk.movieTitle)}</div>
          <div class="muted" style="margin-top: 8px;">${escapeHtml(bk.timing)} • Seats: ${escapeHtml(bk.seats.join(', '))}</div>
          <div class="muted" style="margin-top: 4px;">Email: ${escapeHtml(bk.email)}</div>
        </div>
        <div style="text-align:right">
          <div style="font-weight:800; font-size: 20px;">₹${bk.total}</div>
          <div class="muted" style="margin-top: 4px;">Conf: ${escapeHtml(bk.confirmationId || '')}</div>
        </div>
      </div>
    `;
    container.appendChild(el);
  }
}

/* Helpers */
function maskNumber(num){
  if(num.length<=4) return '****';
  const last = num.slice(-4);
  return '**** **** **** ' + last;
}
function validateEmail(email){
  if(!email) return false;
  // simple regex
  const re = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return re.test(email);
}
function escapeHtml(text){
  if(!text) return '';
  return text.replace(/&/g,'&amp;').replace(/</g,'&lt;').replace(/>/g,'&gt;');
}

/* Logout function clears temporary bookings and selections (and local storage if desired) */
function logout(){
  if(confirm('Logout will clear session-selected data (bookings persist). Proceed?')){
    selectedSeats = [];
    window.currentBooking = null;
    // Optionally clear localStorage bookings: commented out to preserve bookings
    // localStorage.removeItem('bookings');
    // just go to home
    showPage('home');
    showNotification('Logged out successfully');
  }
}

/* Utility to set initial booking page movie/time on load if needed */
document.addEventListener('DOMContentLoaded', init);

/* ensure quick UI functions for links that call from html attributes */
function goToBook(movieId){ currentMovie = movieId; populateDetails(movieId); clearSelection(); renderSeatMap(); showPage('book'); }
function goToDetails(movieId){ currentMovie = movieId; populateDetails(movieId); showPage('details'); }

// ensure show-page when clicking elements by id
document.getElementById('btn-see-showing').addEventListener('click', ()=> showPage('showing'));
document.getElementById('btn-see-upcoming').addEventListener('click', ()=> showPage('upcoming'));

// Keep seat visuals consistent when timing or movie changes
document.getElementById('show-timing').addEventListener('change', ()=> { currentTiming = document.getElementById('show-timing').value; applyBookedVisuals(); });

// In case someone navigates directly to payment without currentBooking, prevent errors:
window.addEventListener('beforeunload', ()=>{}); // placeholder

</script>
</body>
</html>
"""

# write html file
p = Path(HTML_FILENAME)
p.write_text(html_code, encoding='utf-8')
print(f"Written {HTML_FILENAME} ({p.stat().st_size} bytes)")

# Serve the file using a simple HTTP server in a background thread
class QuietHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, format, *args):
        # suppress console spam from the server; comment out if you want logs
        return

def start_server(port=PORT):
    Handler = QuietHTTPRequestHandler
    os.chdir(os.path.dirname(os.path.abspath(HTML_FILENAME)) or ".")
    with socketserver.TCPServer(("", port), Handler) as httpd:
        print(f"Serving HTTP on 0.0.0.0 port {port} (http://localhost:{port}/{HTML_FILENAME}) ...")
        httpd.serve_forever()

# start server in background thread to keep notebook interactive
server_thread = threading.Thread(target=start_server, args=(PORT,), daemon=True)
server_thread.start()

# open the page in default browser (give the server a moment to start)
time.sleep(0.6)
url = f"http://localhost:{PORT}/{HTML_FILENAME}"
print(f"Opening {url} in your web browser...")
webbrowser.open(url)

Exception in thread Thread-14 (start_server):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/opt/anaconda3/lib/python3.13/threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/gr/87c8xy3d21q8n0tlcwllwc800000gn/T/ipykernel_13874/3138275605.py", line 1576, in start_server
    with socketserver.TCPServer(("", port), Handler) as httpd:
         ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 457, in __init__
    self.server_bind()
    ~~~~~~~~~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/socketserver.py", line 478, in server_bind
    self.socket.bind(self.server_address)

Written bookmyshow.html (58670 bytes)
Opening http://localhost:8000/bookmyshow.html in your web browser...


True